<center><br><br>
<font size=6>🎓 <b>Advanced Deep Learning - NLP Final Project</b></font><br>
<font size=6>📊 <b>mDeBERTa-v3-base Training (EDA)</b></font><br>
<font size=5>👥 <b>Group W</b></font><br><br>
<b>Adi Shalit</b>, ID: <code>206628885</code><br>
<b>Gal Gussarsky</b>, ID: <code>206453540</code><br><br>
<font size=4>📘 Course ID: <code>05714184</code></font><br>
<font size=4>📅 Spring 2025</font>
<br><br>
<hr style="width:60%; border:1px solid gray;"></center>


# 📑 Table of Contents

- [Training EX4](#Training-EX4)
- [Load best Model & Test EX4](#Load-Best-Model-EX4)
- [Training EX5](#Training-EX5)
- [Load best Model & Test EX5](#Load-Best-Model-EX5)




# 🚀 Training **mDeBERTa-v3-base** (Exercise 4 Style)

In this stage, we fine-tune **mDeBERTa-v3-base** for **5-class sentiment classification**.  
DeBERTa improves upon BERT/RoBERTa by introducing **disentangled attention** (separating content & position) and **relative position embeddings**, which allow it to capture context and word order more effectively.  

---

## 🎯 **What we do here**
- ⚡ **Pretrained Backbone**: Start from `microsoft/mdeberta-v3-base`.
- 🔒 **Layer Freezing**: Freeze most layers, unfreezing only the **last *k*** (with *k* optimized by Optuna).
- 🏷 **Classification Head**: Add a linear classifier for **5 sentiment classes**.
- 🎛 **Hyperparameter Tuning**: Use **Optuna** to search learning rate, weight decay, batch size, etc.
- ⏳ **Early Stopping**: Stop training when validation **F1** stops improving.
- 📊 **Experiment Tracking**: Log all runs with **Weights & Biases (W&B)**.
- ⚙️ **Training Tricks**: Mixed precision + gradient clipping for faster and more stable convergence.  

---



In [1]:
# import os
# os.environ["TRANSFORMERS_NO_TF"] = "1"   # force-disable TensorFlow backend
# os.environ["TRANSFORMERS_NO_FLAX"] = "1" # optional: also disable Flax/JAX


# Training EX4

In [1]:
# --- Cell 1: setup & speed knobs ---
# !pip  optuna==3.6.1 wandb==0.17.5

import os, math, random, time
from typing import Dict, Tuple
import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

import optuna
import wandb
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup
)


# -------- RTX 4090 performance switches --------
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")
if torch.cuda.is_available():
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    try:
        torch.set_float32_matmul_precision("high")
    except Exception:
        pass

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42
def set_seed(s=42):
    random.seed(s); np.random.seed(s)
    torch.manual_seed(s); torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = False   # allow fast kernels
    torch.backends.cudnn.benchmark = True
set_seed(SEED)

# --- W&B non-blocking defaults ---
os.environ.setdefault("WANDB_MODE", "online")  # set to "offline" if you prefer
WANDB_PROJECT = "adv-dl-deberta-v3"


C:\Users\adishalit1\AppData\Local\anaconda3\envs\dl4090\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 🌍 mDeBERTa-v3-base Description

After first experimenting with **RoBERTa-base**, we now move to a stronger multilingual model:  
**mDeBERTa-v3-base** (multilingual DeBERTa v3).  

---

## 🔎 Why mDeBERTa?
- 🧠 **Better architecture** → DeBERTa introduces **disentangled attention** (separating content vs. position info) and **relative position embeddings**, which help the model better capture context and word order.  
- ⚡ **DeBERTa v3** → improves pretraining with **ELECTRA-style gradient-disentangled embedding sharing**, making training more efficient and boosting accuracy.  
- 🌐 **Multilingual support** → pretrained on **CC100 dataset** (~2.5T tokens, 250k vocab), enabling cross-lingual transfer.  
- 📏 **Model size** → 12 Transformer layers, hidden size 768, ~279M parameters (all trainable in our setup).  

---

## 🏋️ Training Plan
Initially, we considered training also on the **original untranslated dataset**, but due to time and complexity we decided to **focus only on the clean translated version**.  
Thus, the final plan is:  

- ✅ **Train** on the **clean translated training data**.  
- ✅ **Evaluate** on the **clean translated test set** (same conditions as training).  

✨ This ensures a controlled, consistent training pipeline, while letting us directly compare mDeBERTa’s performance with RoBERTa under the same clean setup.  


In [2]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/mdeberta-v3-base", num_labels=5
).to(DEVICE)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable params: {trainable_params:,} / {total_params:,} "
      f"({100.0*trainable_params/total_params:.2f}%)")


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/mdeberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable params: 278,813,189 / 278,813,189 (100.00%)


C:\Users\adishalit1\AppData\Local\anaconda3\envs\dl4090\Lib\site-packages\torch\nn\modules\module.py:1329: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  return t.to(


In [3]:
model

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(251000, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

# 📊 First Training Results – Hyperparameter Insights

https://wandb.ai/adishalit1-tel-aviv-university/adv-dl-p2?nw=nwuseradishalit1

In the **first training phase** with **mDeBERTa-v3-base**, we used **Optuna** to explore a wide hyperparameter space.  
Although prior research suggests that DeBERTa works best with learning rates around **1e-5 – 6e-5**,  
we intentionally started with **much wider bounds (1e-6 – 5e-5)** to test stability and robustness.

---

## 🏆 Key Observations
- **⭐ Best Trial (Trial 2):**  
  - Validation F1 = **0.88022**  
  - Learning rate = **3.5e-5**  
  - Batch size = **8**  
  - Weight decay = **9.4e-5**  
  - **All 12 encoder layers unfrozen**  

  🔹 This score is almost identical to the **real-world English benchmark performance (~88.2 F1)**, confirming our setup reproduces strong results.

---

### 🔑 Hyperparameter Insights
- **Learning rate:**  
  - Best range: **3e-5 – 9e-5**  
  - Too high → unstable training  
  - Too low (<1e-5) → stable but plateaued

- **Unfrozen layers:**  
  - **9–12 layers unfrozen** performed best  
  - Full fine-tuning > partial freezing

- **Weight decay:**  
  - Small values (<1e-5) gave stability  
  - Very large → degraded performance

- **Batch size:**  
  - **8–16** = stable  
  - Larger sizes didn’t improve performance

---

## 🚀 Next Steps
- 🔎 Narrow LR search to **1e-5 – 5e-5** (sweet spot).  
- ✅ Stick with **full fine-tuning (12 layers)**.  
- 📉 Use **low weight decay (<1e-5)**.  
- 🔄 Retrain with **refined bounds** for a cleaner checkpoint and more reliable results.


In [3]:
# # =========================
# # ADV DL – Part B: Monolingual baseline (RoBERTa) – Exercise-4 style
# # Custom loop + early stopping + W&B + Optuna ONLY; freeze base, unfreeze last k layers
# # Uses df_train / df_test with columns: OriginalTweet (str), Sentiment (str)
# # =========================

# import os, math, random, time, json
# from typing import Dict, List, Tuple

# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix
# import torch
# from torch.utils.data import Dataset, DataLoader
# from torch.cuda.amp import autocast, GradScaler

# # ---- deps ----
# # !pip -q install transformers==4.43.3 optuna==3.6.1 wandb==0.17.5 >/dev/null

# import transformers
# from transformers import (
#     AutoTokenizer, AutoModelForSequenceClassification,
#     DataCollatorWithPadding, get_linear_schedule_with_warmup
# )

# import optuna
# import wandb

# # # from google.colab import drive
# # drive.mount("/content/drive")
# # DRIVE_OUT_DIR = "/content/drive/MyDrive/adv_dl_models"
# # os.makedirs(DRIVE_OUT_DIR, exist_ok=True)

# # -------------------------
# # Constants (no CFG, Optuna-only workflow)
# # -------------------------
# MODEL_NAME = "microsoft/mdeberta-v3-base"
# MAX_LEN = 512
# BATCH_SIZE = 16
# WARMUP_RATIO = 0.06
# GRAD_CLIP = 1.0
# USE_AMP = True
# PROJECT = "adv-dl-p2"
# BASE_RUN_NAME = "microsoft/mdeberta-v3-base_full_ex_4"
# TRIALS = 12
# SEED = 42
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# def set_seed(seed=42):
#     random.seed(seed); np.random.seed(seed)
#     torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
#     torch.backends.cudnn.deterministic = True
#     torch.backends.cudnn.benchmark = False

# set_seed(SEED)

# # -------------------------
# # Label mapping (5-way sentiment)
# # -------------------------
# CANON = {
#     "extremely negative": "extremely negative",
#     "negative": "negative",
#     "neutral": "neutral",
#     "positive": "positive",
#     "extremely positive": "extremely positive",
# }
# ORDER = ["extremely negative","negative","neutral","positive","extremely positive"]
# LABEL2ID = {lab: i for i, lab in enumerate(ORDER)}
# ID2LABEL = {i: lab for lab, i in LABEL2ID.items()}

# def normalize_label(s: str) -> str:
#     s = str(s).strip().lower()
#     s = s.replace("very negative", "extremely negative")
#     s = s.replace("very positive", "extremely positive")
#     s = s.replace("extreme negative", "extremely negative")
#     s = s.replace("extreme positive", "extremely positive")
#     return CANON.get(s, s)

# # -------------------------
# # Expect df_train, df_test in memory
# # -------------------------
# assert "OriginalTweet" in df_train.columns and "Sentiment" in df_train.columns, "df_train missing required columns"
# assert "OriginalTweet" in df_test.columns and "Sentiment" in df_test.columns, "df_test missing required columns"

# def prep_df(df: pd.DataFrame) -> pd.DataFrame:
#     df = df.copy()
#     df = df.dropna(subset=["OriginalTweet", "Sentiment"])
#     df["text"] = df["OriginalTweet"].astype(str).str.strip()
#     df["label_name"] = df["Sentiment"].apply(normalize_label)
#     df = df[df["label_name"].isin(ORDER)].reset_index(drop=True)
#     df["label"] = df["label_name"].map(LABEL2ID)
#     return df[["text", "label", "label_name"]]

# dftrain_ = prep_df(df_train)
# dftest_  = prep_df(df_test)

# train_df, val_df = train_test_split(
#     dftrain_, test_size=0.1, stratify=dftrain_["label"], random_state=SEED
# )
# print(f"Train/Val/Test sizes: {len(train_df)}/{len(val_df)}/{len(dftest_)}")

# # -------------------------
# # Dataset & Collator
# # -------------------------
# class TweetDataset(Dataset):
#     def __init__(self, df: pd.DataFrame, tokenizer: transformers.PreTrainedTokenizerBase, max_len: int):
#         self.texts = df["text"].tolist()
#         self.labels = df["label"].tolist()
#         self.tok = tokenizer
#         self.max_len = max_len
#     def __len__(self): return len(self.texts)
#     def __getitem__(self, idx):
#         enc = self.tok(self.texts[idx], truncation=True, max_length=self.max_len, padding=False)
#         enc["labels"] = self.labels[idx]
#         return {k: torch.tensor(v) for k, v in enc.items()}

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
# train_ds = TweetDataset(train_df, tokenizer, MAX_LEN)
# val_ds   = TweetDataset(val_df, tokenizer, MAX_LEN)
# test_ds  = TweetDataset(dftest_, tokenizer, MAX_LEN)

# collate_fn = DataCollatorWithPadding(tokenizer=tokenizer)
# train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  collate_fn=collate_fn, num_workers=2, pin_memory=True)
# val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn, num_workers=2, pin_memory=True)
# test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn, num_workers=2, pin_memory=True)

# # -------------------------
# # Model & Freeze/Unfreeze strategy
# # -------------------------
# def build_model(num_unfreeze_last_layers: int = 4):
#     model = AutoModelForSequenceClassification.from_pretrained(
#         MODEL_NAME, num_labels=len(ORDER), id2label=ID2LABEL, label2id=LABEL2ID
#     )
#     base = getattr(model, "roberta", None) or getattr(model, "bert", None) or getattr(model, "deberta", None)
#     if base is not None:
#         for p in base.parameters(): p.requires_grad = False
#         if hasattr(base, "encoder") and hasattr(base.encoder, "layer"):
#             k = num_unfreeze_last_layers
#             if k > 0:
#                 for layer in base.encoder.layer[-k:]:
#                     for p in layer.parameters(): p.requires_grad = True
#     for p in model.classifier.parameters(): p.requires_grad = True
#     return model.to(DEVICE)

# # -------------------------
# # Train / Eval utilities
# # -------------------------
# def get_optimizer_scheduler(model, num_training_steps: int, lr: float, weight_decay: float):
#     no_decay = ["bias", "LayerNorm.weight"]
#     optimizer_grouped_parameters = [
#         {"params": [p for n, p in model.named_parameters() if p.requires_grad and not any(nd in n for nd in no_decay)], "weight_decay": weight_decay},
#         {"params": [p for n, p in model.named_parameters() if p.requires_grad and any(nd in n for nd in no_decay)],  "weight_decay": 0.0},
#     ]
#     optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=lr)
#     num_warmup = int(num_training_steps * WARMUP_RATIO)
#     scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup, num_training_steps=num_training_steps)
#     return optimizer, scheduler

# def evaluate(model, loader) -> Dict[str, float]:
#     model.eval()
#     preds, labels = [], []
#     with torch.no_grad():
#         for batch in loader:
#             batch = {k: v.to(DEVICE) for k, v in batch.items()}
#             logits = model(**batch).logits
#             preds.extend(torch.argmax(logits, dim=-1).detach().cpu().tolist())
#             labels.extend(batch["labels"].detach().cpu().tolist())
#     acc = accuracy_score(labels, preds)
#     p, r, f1, _ = precision_recall_fscore_support(labels, preds, average="macro", zero_division=0)
#     return {"acc": acc, "precision": p, "recall": r, "f1": f1}

# def train_one_run(hp: Dict) -> Tuple[str, Dict[str, float]]:
#     """
#     hp keys: run_name, num_unfreeze_last_layers, lr, weight_decay, epochs, patience, trial_number
#     """
#     run_name = hp["run_name"]
#     num_unfreeze = int(hp["num_unfreeze_last_layers"])
#     lr = float(hp["lr"])
#     wd = float(hp["weight_decay"])
#     epochs = int(hp["epochs"])
#     patience = int(hp["patience"])

#     model = build_model(num_unfreeze)
#     total_steps = int(math.ceil(len(train_loader) * epochs))
#     optimizer, scheduler = get_optimizer_scheduler(model, total_steps, lr, wd)

#     scaler = GradScaler(enabled=(DEVICE == "cuda" and USE_AMP))
#     best_metric = -1.0
#     best_path = f"best_{run_name}.pt"
#     no_improve = 0
#     wandb_run = wandb.init(
#         project=PROJECT,
#         name=run_name,
#         config={
#             "model": MODEL_NAME,
#             "max_len": MAX_LEN,
#             "batch_size": BATCH_SIZE,
#             "epochs": epochs,
#             "lr": lr,
#             "weight_decay": wd,
#             "warmup_ratio": WARMUP_RATIO,
#             "grad_clip": GRAD_CLIP,
#             "num_unfreeze_last_layers": num_unfreeze,
#             # >>> ADDED — purely for tracking the Optuna suggestion (even if not used by loaders)
#             "suggested_batch_size": hp.get("batch_size", BATCH_SIZE),
#             "trial_number": hp.get("trial_number", -1),
#         },
#         reinit=True,
#     )

#     # >>> ADDED — define step metrics so W&B charts look great
#     wandb.define_metric("epoch")
#     wandb.define_metric("step")
#     wandb.define_metric("train/*", step_metric="step")
#     wandb.define_metric("val/*",   step_metric="epoch")

#     # >>> ADDED — visibility of trainable params
#     total_params     = sum(p.numel() for p in model.parameters())
#     trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
#     print(f"Trainable params: {trainable_params:,} / {total_params:,} "
#           f"({100.0*trainable_params/total_params:.2f}%) ; unfreeze_last_k={num_unfreeze}")
#     wandb.log({
#         "params/total": total_params,
#         "params/trainable": trainable_params,
#         "params/ratio": trainable_params / max(1, total_params),
#     }, step=0)



#     for epoch in range(epochs):
#         model.train()
#         t0 = time.time()
#         running_loss = 0.0
#         for step, batch in enumerate(train_loader):
#             batch = {k: v.to(DEVICE) for k, v in batch.items()}
#             optimizer.zero_grad(set_to_none=True)
#             with autocast(enabled=(DEVICE == "cuda" and USE_AMP)):
#                 outputs = model(**batch)
#                 loss = outputs.loss
#             scaler.scale(loss).backward()
#             if GRAD_CLIP is not None:
#                 scaler.unscale_(optimizer)
#                 torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
#             scaler.step(optimizer); scaler.update(); scheduler.step()
#             running_loss += loss.item()

#             if step % 20 == 0:
#                 wandb.log({"train/loss": loss.item(), "step": step + 1, "epoch": epoch + 1})

#         # epoch-end validation
#         val_metrics = evaluate(model, val_loader)
#         elapsed = time.time() - t0

#         epoch_loss = running_loss / max(1, len(train_loader))
#         current_lr = scheduler.get_last_lr()[0]
#         wandb.log({
#             "train/epoch_loss": epoch_loss,
#             "val/acc": val_metrics["acc"],
#             "val/precision": val_metrics["precision"],
#             "val/recall": val_metrics["recall"],
#             "val/f1": val_metrics["f1"],
#             "lr": current_lr,
#             "time/epoch_sec": elapsed,
#             "epoch": epoch + 1,
#         })

#         # Early stopping on val f1
#         if val_metrics["f1"] > best_metric:
#             best_metric = val_metrics["f1"]
#             torch.save(model.state_dict(), best_path)
#             no_improve = 0
#             wandb_run.summary["best_val_f1"] = best_metric
#             wandb_run.summary["best_checkpoint_path"] = best_path
#         else:
#             no_improve += 1
#             if no_improve >= patience:
#                 print(f"Early stopping at epoch {epoch+1}")
#                 break

#         print(f"Epoch {epoch+1}/{epochs} | "
#               f"loss={epoch_loss:.4f} | "
#               f"val_acc={val_metrics['acc']:.4f} | val_f1={val_metrics['f1']:.4f} | time={elapsed:.1f}s")

#     wandb.finish()

#     # Load best and return path + metrics on val for reference
#     model.load_state_dict(torch.load(best_path, map_location=DEVICE))
#     final_val = evaluate(model, val_loader)
#     return best_path, final_val

# # -------------------------
# # Optuna hyperparameter tuning (ALWAYS ON)
# # -------------------------

# # Constants
# FIXED_EPOCHS = 12
# FIXED_PATIENCE = 4


# def objective(trial: optuna.trial.Trial):
#     params = {
#         "run_name": f"{BASE_RUN_NAME}_optuna_trial_{trial.number}",
#         "num_unfreeze_last_layers": trial.suggest_int("num_unfreeze_last_layers", 4, 12),
#         "lr": trial.suggest_float("lr", 1e-6, 5e-5, log=True),
#         "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-1, log=True),
#         "batch_size": trial.suggest_categorical("batch_size", [4, 8, 16, 32, 64]),
#         "epochs": FIXED_EPOCHS,
#         "patience": FIXED_PATIENCE,
#         "trial_number": trial.number,
#     }
#     path, val_metrics = train_one_run(params)
#     # report intermediate value for pruning if enabled
#     trial.report(val_metrics["f1"], step=1)
#     return val_metrics["f1"]

# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=TRIALS, show_progress_bar=True)
# print("Best trial:", study.best_trial.number, "F1:", study.best_value)
# best_params = {"run_name": f"{BASE_RUN_NAME}_best_optuna", **study.best_trial.params}

# # Retrain best config to get a clean checkpoint
# best_ckpt, _ = train_one_run(best_params)
# best_path = best_ckpt

# # -------------------------
# # Final evaluation on TEST (+ W&B logging)
# # -------------------------
# model = build_model(best_params["num_unfreeze_last_layers"])
# model.load_state_dict(torch.load(best_path, map_location=DEVICE))
# model.eval()

# all_preds, all_labels = [], []
# with torch.no_grad():
#     for batch in test_loader:
#         batch = {k: v.to(DEVICE) for k, v in batch.items()}
#         logits = model(**batch).logits
#         all_preds.extend(torch.argmax(logits, dim=-1).detach().cpu().tolist())
#         all_labels.extend(batch["labels"].detach().cpu().tolist())

# acc = accuracy_score(all_labels, all_preds)
# p, r, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="macro", zero_division=0)
# print(f"\nTEST | acc={acc:.4f} | f1_macro={f1:.4f} | precision_macro={p:.4f} | recall_macro={r:.4f}\n")

# print("Per-class report (ids map to labels):")
# print(ID2LABEL)
# report = classification_report(
#     all_labels, all_preds,
#     target_names=[ID2LABEL[i] for i in range(len(ORDER))],
#     zero_division=0, output_dict=True
# )
# print(classification_report(
#     all_labels, all_preds,
#     target_names=[ID2LABEL[i] for i in range(len(ORDER))],
#     zero_division=0
# ))

# # ---- W&B: log test metrics, per-class scores, and confusion matrix ----
# test_run = wandb.init(project=PROJECT, name=f"{BASE_RUN_NAME}_test", resume="allow", reinit=True)
# log_payload = {
#     "test/acc": acc,
#     "test/precision_macro": p,
#     "test/recall_macro": r,
#     "test/f1_macro": f1,
# }
# for cls_name in ORDER:
#     if cls_name in report:
#         log_payload[f"test/{cls_name}/precision"] = report[cls_name]["precision"]
#         log_payload[f"test/{cls_name}/recall"]    = report[cls_name]["recall"]
#         log_payload[f"test/{cls_name}/f1"]        = report[cls_name]["f1-score"]

# wandb.log(log_payload)

# cm = confusion_matrix(all_labels, all_preds, labels=list(range(len(ORDER))))
# wandb.log({
#     "test/confusion_matrix": wandb.plot.confusion_matrix(
#         y_true=all_labels,
#         preds=all_preds,
#         class_names=[ID2LABEL[i] for i in range(len(ID2LABEL))]
#     )
# })
# test_run.summary["best_checkpoint_path"] = best_path
# test_run.summary["test_f1_macro"] = f1
# wandb.finish()

# 🔧 Second Hyperparameter Tuning – Testing Bigger Batches & Higher LRs

From the **first tuning round**, we figured out that:  
- ✅ Unfreezing **all 12 layers** works best  
- ✅ LR around **3e-5** was the sweet spot  

But we weren’t fully sure how far we can push things, so in this round we tried a **wider search** to see what happens.

---

## ⚙️ Changes we made
- **Batch size:** tried `[4, 8, 16, 32, 64]` → to check if bigger batches give more stable training, or if small ones still generalize better.  
- **Learning rate:** expanded up to `1e-2` 🤯 → just to test if more aggressive fine-tuning can still converge.  
- **Unfreezing:** kept the range at **8–12 layers**, since deeper unfreezing looked clearly better last time.  
- **Weight decay:** limited to `1e-6 – 1e-4`, because we saw high values ruin performance.  

---

## 🎯 Why we did this
- To see if **large batches** speed things up without hurting F1.  
- To test whether **big LRs** can still work with proper scheduling.  
- To confirm if **full unfreezing** keeps outperforming partial freezing.  

---

## 📝 What we expect
Honestly, we think the **lower LR region (1e-5 – 1e-4)** will still win 🏆,  
but this round should give us a better sense of the **stability limits** when scaling both **batch size** and **learning rate**.  


In [4]:
from sklearn.model_selection import train_test_split
# Load CSVs (columns: ['UserName','ScreenName','Location','TweetAt','OriginalTweet','Sentiment'])
TRAIN_CSV = "Corona_NLP_train_cleaned_translated.csv"   # or "Corona_NLP_train.csv"
TEST_CSV  = "Corona_NLP_test_cleaned_translated.csv"    # or "Corona_NLP_test.csv"


df_train = pd.read_csv(TRAIN_CSV, encoding="utf-8", engine="python")
df_test  = pd.read_csv(TEST_CSV,  encoding="utf-8", engine="python")

In [5]:
# =========================
# ADV DL – Part B: Monolingual baseline (RoBERTa) – Exercise-4 style
# Custom loop + early stopping + W&B + Optuna ONLY; freeze base, unfreeze last k layers
# Uses df_train / df_test with columns: OriginalTweet (str), Sentiment (str)
# =========================

import os, math, random, time, json
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix
import torch
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler

# ---- deps ----
# !pip -q install transformers==4.43.3 optuna==3.6.1 wandb==0.17.5 >/dev/null

import transformers
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    DataCollatorWithPadding, get_linear_schedule_with_warmup
)
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"

import optuna
import wandb

# -------------------------
# Constants (no CFG, Optuna-only workflow)
# -------------------------
MODEL_NAME = "microsoft/mdeberta-v3-base"
MAX_LEN = 512
BATCH_SIZE = 16
WARMUP_RATIO = 0.06
GRAD_CLIP = 1.0
USE_AMP = True
# PROJECT = "adv-dl-p2"
PROJECT = "adv-dl-p2-deberta-full"

BASE_RUN_NAME = "microsoft/mdeberta-v3-base_full_ex_4"
TRIALS = 20
SEED = 42
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(SEED)

# ---- GPU perf toggles (Windows-safe) ----
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
try:
    torch.set_float32_matmul_precision("high")
except Exception:
    pass

# -------------------------
# Label mapping (5-way sentiment)
# -------------------------
CANON = {
    "extremely negative": "extremely negative",
    "negative": "negative",
    "neutral": "neutral",
    "positive": "positive",
    "extremely positive": "extremely positive",
}
ORDER = ["extremely negative","negative","neutral","positive","extremely positive"]
LABEL2ID = {lab: i for i, lab in enumerate(ORDER)}
ID2LABEL = {i: lab for lab, i in LABEL2ID.items()}

def normalize_label(s: str) -> str:
    s = str(s).strip().lower()
    s = s.replace("very negative", "extremely negative")
    s = s.replace("very positive", "extremely positive")
    s = s.replace("extreme negative", "extremely negative")
    s = s.replace("extreme positive", "extremely positive")
    return CANON.get(s, s)

# -------------------------
# Expect df_train, df_test in memory
# -------------------------
assert "OriginalTweet" in df_train.columns and "Sentiment" in df_train.columns, "df_train missing required columns"
assert "OriginalTweet" in df_test.columns and "Sentiment" in df_test.columns, "df_test missing required columns"

def prep_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df = df.dropna(subset=["OriginalTweet", "Sentiment"])
    df["text"] = df["OriginalTweet"].astype(str).str.strip()
    df["label_name"] = df["Sentiment"].apply(normalize_label)
    df = df[df["label_name"].isin(ORDER)].reset_index(drop=True)
    df["label"] = df["label_name"].map(LABEL2ID)
    return df[["text", "label", "label_name"]]

dftrain_ = prep_df(df_train)
dftest_  = prep_df(df_test)

train_df, val_df = train_test_split(
    dftrain_, test_size=0.1, stratify=dftrain_["label"], random_state=SEED
)
print(f"Train/Val/Test sizes: {len(train_df)}/{len(val_df)}/{len(dftest_)}")

# -------------------------
# Dataset & Collator
# -------------------------
class TweetDataset(Dataset):
    def __init__(self, df: pd.DataFrame, tokenizer: transformers.PreTrainedTokenizerBase, max_len: int):
        self.texts = df["text"].tolist()
        self.labels = df["label"].tolist()
        self.tok = tokenizer
        self.max_len = max_len
    def __len__(self): return len(self.texts)
    def __getitem__(self, idx):
        enc = self.tok(self.texts[idx], truncation=True, max_length=self.max_len, padding=False)
        enc["labels"] = self.labels[idx]
        return {k: torch.tensor(v) for k, v in enc.items()}

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
train_ds = TweetDataset(train_df, tokenizer, MAX_LEN)
val_ds   = TweetDataset(val_df, tokenizer, MAX_LEN)
test_ds  = TweetDataset(dftest_, tokenizer, MAX_LEN)


BATCH_SIZE=16
# ---- pad_to_multiple_of=8 for Tensor Cores; Windows: workers=0 is often faster ----
collate_fn = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=8)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  collate_fn=collate_fn, num_workers=0, pin_memory=True, persistent_workers=False)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn, num_workers=0, pin_memory=True, persistent_workers=False)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn, num_workers=0, pin_memory=True, persistent_workers=False)

# -------------------------
# Model & Freeze/Unfreeze strategy
# -------------------------
def build_model(num_unfreeze_last_layers: int = 4):
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME, num_labels=len(ORDER), id2label=ID2LABEL, label2id=LABEL2ID
    )
    base = getattr(model, "roberta", None) or getattr(model, "bert", None) or getattr(model, "deberta", None)
    if base is not None:
        for p in base.parameters(): p.requires_grad = False
        if hasattr(base, "encoder") and hasattr(base.encoder, "layer"):
            k = num_unfreeze_last_layers
            if k > 0:
                for layer in base.encoder.layer[-k:]:
                    for p in layer.parameters(): p.requires_grad = True
    for p in model.classifier.parameters(): p.requires_grad = True
    return model.to(DEVICE)

# -------------------------
# Train / Eval utilities
# -------------------------
def get_optimizer_scheduler(model, num_training_steps: int, lr: float, weight_decay: float):
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {"params": [p for n, p in model.named_parameters() if p.requires_grad and not any(nd in n for nd in no_decay)], "weight_decay": weight_decay},
        {"params": [p for n, p in model.named_parameters() if p.requires_grad and any(nd in n for nd in no_decay)],  "weight_decay": 0.0},
    ]
    # try fused AdamW on CUDA (faster step) — falls back if unavailable
    try:
        optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=lr, weight_decay=weight_decay, fused=(DEVICE=="cuda"))
    except TypeError:
        optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=lr, weight_decay=weight_decay)
    num_warmup = int(num_training_steps * WARMUP_RATIO)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup, num_training_steps=num_training_steps)
    return optimizer, scheduler

def evaluate(model, loader) -> Dict[str, float]:
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for batch in loader:
            batch = {k: v.to(DEVICE, non_blocking=True) for k, v in batch.items()}
            # AMP autocast for faster eval math
            with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),
                          enabled=(DEVICE == "cuda" and USE_AMP)):
                logits = model(**batch).logits
            preds.extend(torch.argmax(logits, dim=-1).detach().cpu().tolist())
            labels.extend(batch["labels"].detach().cpu().tolist())
    acc = accuracy_score(labels, preds)
    p, r, f1, _ = precision_recall_fscore_support(labels, preds, average="macro", zero_division=0)
    return {"acc": acc, "precision": p, "recall": r, "f1": f1}

def train_one_run(hp: Dict) -> Tuple[str, Dict[str, float]]:
    """
    hp keys: run_name, num_unfreeze_last_layers, lr, weight_decay, epochs, patience, trial_number
    """
    run_name = hp["run_name"]
    num_unfreeze = int(hp["num_unfreeze_last_layers"])
    lr = float(hp["lr"])
    wd = float(hp["weight_decay"])
    # epochs = int(hp["epochs"])
    # patience = int(hp["patience"])
    epochs   = int(hp.get("epochs",   FIXED_EPOCHS))
    patience = int(hp.get("patience", FIXED_PATIENCE))
    model = build_model(num_unfreeze)
    total_steps = int(math.ceil(len(train_loader) * epochs))
    optimizer, scheduler = get_optimizer_scheduler(model, total_steps, lr, wd)

    scaler = GradScaler(enabled=(DEVICE == "cuda" and USE_AMP))
    best_metric = -1.0
    # best_path = f"best_{run_name}.pt"
    no_improve = 0
    # make run_name safe for filesystem paths and ensure a folder exists
    safe_run_name = run_name.replace("/", "__").replace("\\", "__")
    ckpt_dir = "checkpoints"
    os.makedirs(ckpt_dir, exist_ok=True)
    best_path = os.path.join(ckpt_dir, f"best_{safe_run_name}.pt")

    wandb_run = wandb.init(
        project=PROJECT,
        name=run_name,
        config={
            "model": MODEL_NAME,
            "max_len": MAX_LEN,
            "batch_size": BATCH_SIZE,
            "epochs": epochs,
            "lr": lr,
            "weight_decay": wd,
            "warmup_ratio": WARMUP_RATIO,
            "grad_clip": GRAD_CLIP,
            "num_unfreeze_last_layers": num_unfreeze,
            "trial_number": hp.get("trial_number", -1),
            "suggested_batch_size": hp.get("batch_size", BATCH_SIZE),
        },
        reinit=True,
    )

    # nicer W&B charts
    wandb.define_metric("epoch")
    wandb.define_metric("step")
    wandb.define_metric("train/*", step_metric="step")
    wandb.define_metric("val/*",   step_metric="epoch")

    # print + log trainable params
    total_params     = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Trainable params: {trainable_params:,} / {total_params:,} "
          f"({100.0*trainable_params/total_params:.2f}%) ; unfreeze_last_k={num_unfreeze}")
    wandb.log({"params/total": total_params,
               "params/trainable": trainable_params,
               "params/ratio": trainable_params/max(1,total_params)}, step=0)

    for epoch in range(epochs):
        model.train()
        t0 = time.time()
        running_loss = 0.0

        for step, batch in enumerate(train_loader):
            batch = {k: v.to(DEVICE, non_blocking=True) for k, v in batch.items()}
            optimizer.zero_grad(set_to_none=True)
            # use BF16 if supported; else FP16
            with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),
                          enabled=(DEVICE == "cuda" and USE_AMP)):
                outputs = model(**batch)
                loss = outputs.loss
            scaler.scale(loss).backward()
            if GRAD_CLIP is not None:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
            scaler.step(optimizer); scaler.update(); scheduler.step()
            running_loss += loss.item()

            if step % 20 == 0:
                wandb.log({"train/loss": loss.item(), "step": step + 1, "epoch": epoch + 1})

            # periodic console + throughput log (about 10x per epoch)
            if step % max(1, len(train_loader)//10) == 0 or step == 1:
                avg_loss = running_loss / max(1, (step + 1))
                elapsed  = time.time() - t0
                items    = (step + 1) * BATCH_SIZE
                itps     = items / max(elapsed, 1e-6)
                print(f"[e{epoch+1} b{step+1}/{len(train_loader)}] loss={loss.item():.4f} avg={avg_loss:.4f} it/s={itps:.1f}")
                wandb.log({"train/avg_loss_so_far": avg_loss,
                           "train/items_per_sec": itps,
                           "step": (epoch * len(train_loader)) + (step + 1),
                           "epoch": epoch + 1})

        # epoch-end validation
        val_metrics = evaluate(model, val_loader)
        elapsed = time.time() - t0

        epoch_loss = running_loss / max(1, len(train_loader))
        current_lr = scheduler.get_last_lr()[0]
        wandb.log({
            "train/epoch_loss": epoch_loss,
            "val/acc": val_metrics["acc"],
            "val/precision": val_metrics["precision"],
            "val/recall": val_metrics["recall"],
            "val/f1": val_metrics["f1"],
            "lr": current_lr,
            "time/epoch_sec": elapsed,
            "epoch": epoch + 1,
        })

        # Early stopping on val f1
        if val_metrics["f1"] > best_metric:
            best_metric = val_metrics["f1"]
            torch.save(model.state_dict(), best_path)
            no_improve = 0
            wandb_run.summary["best_val_f1"] = best_metric
            wandb_run.summary["best_checkpoint_path"] = best_path
            wandb.log({"val/best_f1_so_far": best_metric, "val/best_epoch": epoch + 1})
        else:
            no_improve += 1
            if no_improve >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

        print(f"Epoch {epoch+1}/{epochs} | "
              f"loss={epoch_loss:.4f} | "
              f"val_acc={val_metrics['acc']:.4f} | val_f1={val_metrics['f1']:.4f} | time={elapsed:.1f}s")

    wandb.finish()

    # Load best and return path + metrics on val for reference
    model.load_state_dict(torch.load(best_path, map_location=DEVICE))
    final_val = evaluate(model, val_loader)

    # store final val in W&B summary for quick sorting
    if wandb.run is not None:
        wandb.run.summary["final_val_acc"] = final_val["acc"]
        wandb.run.summary["final_val_precision"] = final_val["precision"]
        wandb.run.summary["final_val_recall"] = final_val["recall"]
        wandb.run.summary["final_val_f1"] = final_val["f1"]

    return best_path, final_val

# -------------------------
# Optuna hyperparameter tuning (ALWAYS ON)
# -------------------------

# Constants
FIXED_EPOCHS = 12
FIXED_PATIENCE = 4

def objective(trial: optuna.trial.Trial):
    params = {
        "run_name": f"{BASE_RUN_NAME}_optuna_trial_{trial.number}",
        "num_unfreeze_last_layers": trial.suggest_int("num_unfreeze_last_layers", 8, 12),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-4, log=True),
        "batch_size": trial.suggest_categorical("batch_size", [4, 8, 16, 32, 64]),
        "epochs": FIXED_EPOCHS,
        "patience": FIXED_PATIENCE,
        "trial_number": trial.number,
    }
    path, val_metrics = train_one_run(params)
    # console visibility per trial
    print(f"[Trial {trial.number}] f1={val_metrics['f1']:.4f} | "
          f"unfreeze_k={params['num_unfreeze_last_layers']} lr={params['lr']:.2e} "
          f"wd={params['weight_decay']:.1e} suggested_bs={params['batch_size']}")
    # report intermediate value for pruning if enabled
    trial.report(val_metrics["f1"], step=1)
    return val_metrics["f1"]



Train/Val/Test sizes: 37039/4116/3798


C:\Users\adishalit1\AppData\Local\anaconda3\envs\dl4090\Lib\site-packages\transformers\convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


### Run Study

In [5]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=TRIALS, show_progress_bar=True)
print("Best trial:", study.best_trial.number, "F1:", study.best_value)
best_params = {"run_name": f"{BASE_RUN_NAME}_best_optuna", **study.best_trial.params}



[I 2025-08-16 02:23:31,029] A new study created in memory with name: no-name-b0cc087a-195e-454e-8e52-0f0bae1b98f4
  0%|          | 0/20 [00:00<?, ?it/s]Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/mdeberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\adishalit1\AppData\Local\anaconda3\envs\dl4090\Lib\site-packages\torch\nn\modules\module.py:1329: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  return t.to(
C:\Users\adishalit1\AppData\Local\Temp\ipykernel_28084\2837239188.py:205: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
 

Trainable params: 85,648,901 / 278,813,189 (30.72%) ; unfreeze_last_k=12


C:\Users\adishalit1\AppData\Local\Temp\ipykernel_28084\2837239188.py:258: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),


[e1 b1/2315] loss=1.6808 avg=1.6808 it/s=38.9
[e1 b2/2315] loss=1.6321 avg=1.6564 it/s=60.4
[e1 b232/2315] loss=1.5153 avg=1.5628 it/s=329.1
[e1 b463/2315] loss=1.1529 avg=1.4154 it/s=340.2
[e1 b694/2315] loss=1.0651 avg=1.3113 it/s=349.1
[e1 b925/2315] loss=1.4492 avg=1.2348 it/s=352.0
[e1 b1156/2315] loss=0.6699 avg=1.1797 it/s=352.3
[e1 b1387/2315] loss=1.5224 avg=1.2216 it/s=338.3
[e1 b1618/2315] loss=1.5436 avg=1.2721 it/s=326.3
[e1 b1849/2315] loss=1.6322 avg=1.3118 it/s=329.8
[e1 b2080/2315] loss=1.6153 avg=1.3416 it/s=332.6
[e1 b2311/2315] loss=1.6319 avg=1.3653 it/s=333.9


C:\Users\adishalit1\AppData\Local\Temp\ipykernel_28084\2837239188.py:180: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),


...

C:\Users\adishalit1\AppData\Local\Temp\ipykernel_28084\2837239188.py:180: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Early stopping at epoch 6


epoch,▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▄▄▄▄▄▄▅▅▅▅▅▇▇▇▇▇▇▇███████
lr,█▇▅▄▂▁
params/ratio,▁
params/total,▁
params/trainable,▁
step,▁▁▁▂▂▂▂▁▁▁▂▂▁▁▁▁▁▄▂▂▂▂▂▁▁▂▂▂▁▂▂▂▇▂▂▂▂▂▂█
time/epoch_sec,▆█▅▆▁▁
train/avg_loss_so_far,▆▆▄▃▂▁▁▁▂█▆▆▆▆▆▆▆▆▆▆▅▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
train/epoch_loss,▁█████
train/items_per_sec,▁▆▆▇▇▇▇▇██▇▇▇▇▇▆▇▇▇▇▇▆▆▇▇▇▆█▇▇██▇█▇▇▇▇▇▇
train/loss,▄▃▄▃▁▆▁▇█▇▇▆▅▇▇▆▆▆▇▆▇▇▇▆▆▆▇▇▆▆▆▆▇▇▇▇▇▆█▆


C:\Users\adishalit1\AppData\Local\Temp\ipykernel_28084\2837239188.py:180: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),
Best trial: 13. Best value: 0.873252: 100%|██████████| 20/20 [3:47:06<00:00, 681.32s/it]

[Trial 19] f1=0.0869 | unfreeze_k=9 lr=1.91e-04 wd=2.5e-06 suggested_bs=8
[I 2025-08-16 06:10:37,417] Trial 19 finished with value: 0.08687713959680486 and parameters: {'num_unfreeze_last_layers': 9, 'lr': 0.00019148701280271666, 'weight_decay': 2.4790993754320025e-06, 'batch_size': 8}. Best is trial 13 with value: 0.8732520507590781.
Best trial: 13 F1: 0.8732520507590781


# 📊 Discussion of Results

Looking at the full Optuna study, we can clearly split trials into two groups:

- ✅ **Good trials**: ~6 runs went the distance (12 epochs) and reached **val/F1 > 0.86**  
- ❌ **Bad trials**: the rest crashed early with **F1 ≈ 0.08**, meaning some hyperparameter combos are just not stable for fine-tuning mDeBERTa  

---

## 🔍 Patterns we noticed
- **Learning rate (LR):**  
  - Best runs are always in the **1e-5 – 1e-4 range**  
  - Larger LRs → quick divergence and failure  
- **Weight decay:**  
  - When **WD > 1e-5**, performance dropped  
  - Small WD values (≈ `1e-5` or less) worked much better  
- **Unfreezing layers:**  
  - Strong runs came from **8–12 layers unfrozen**  
  - The **absolute best run** unfreezed **all 12 layers**, showing deeper fine-tuning pays off for this dataset  

---

## 🏆 Best run
The top score came from **Trial 2 (first run)** with:  
- **val/F1 = 0.88022**  
- LR = `3.5e-5`  
- Batch size = `8`  
- Weight decay = `9.4e-5`  
- Unfrozen layers = **12** (full encoder)  

💡 This result is basically on par with the **official English benchmark for mDeBERTa (~88.2 F1)** → so our setup is hitting the model’s expected ceiling.  

---

## 🚀 Next steps
- Zoom in on the **LR sweet spot (1e-5 – 5e-5)**  
- Stick with **low weight decay (<1e-5)**  
- Keep **full unfreezing (12 layers)**, or at least ≥9  
- Retrain in this refined space to get a more stable checkpoint → then move on to the **clean vs noisy test comparison**  


# 💾 Saving and Reloading Best Hyperparameters

After finishing the Optuna search, we don’t want to lose the best hyperparameters.  
To make the process reproducible, we **save them into a JSON file** so we can load them again later.  


In [7]:
import json, os
os.makedirs("checkpoints", exist_ok=True)

# best hparams from Optuna (only suggested ones live here)
best_hparams = study.best_trial.params

# if your train_one_run expects epochs/patience and they were fixed (not suggested),
# add them explicitly:
best_hparams_complete = {
    **best_hparams,
    "epochs": FIXED_EPOCHS,       # or whatever you used
    "patience": FIXED_PATIENCE,   # "
}
# hp_path = os.path.join("checkpoints", "best_hparams_optuna.json")
hp_path = os.path.join("checkpoints", "best_hparams_optuna_2.json")
with open(hp_path, "w") as f:
    json.dump(best_hparams_complete, f, indent=2)
print("Saved best hparams to:", hp_path)
print("Best trial number:", study.best_trial.number, " value:", study.best_value)


Saved best hparams to: checkpoints\best_hparams_optuna_2.json
Best trial number: 13  value: 0.8732520507590781


## 🚀Load & Train Best Hyperparameters

Now we load the saved hyperparameters from the best Optuna run.  
The best trial reached **val/F1 = 0.88022** (trial 2), with LR ≈ 3.5e-5, weight decay ≈ 9.4e-5, batch size 8, and all 12 layers unfrozen.  
We will use these settings to retrain the model in a clean run and later evaluate it on the test set.


# Load Best Model EX4

In [7]:
import json, os
BASE_RUN_NAME = "microsoft/mdeberta-v3-base_full_ex_4"
with open(os.path.join("checkpoints", "best_hparams_optuna.json")) as f:
    best_hparams = json.load(f)

# give a distinct name for the final run
best_hparams["run_name"] = f"{BASE_RUN_NAME}_best_optuna_retrain"

# (optional) bump epochs here; see guidance below
best_hparams["epochs"] = 10      # higher cap
best_hparams["patience"] = 4     # unchanged

best_hparams


{'num_unfreeze_last_layers': 12,
 'lr': 3.496909962515421e-05,
 'weight_decay': 9.403805231949854e-05,
 'batch_size': 8,
 'epochs': 10,
 'patience': 4,
 'run_name': 'microsoft/mdeberta-v3-base_full_ex_4_best_optuna_retrain'}

In [9]:

# Retrain best config to get a clean checkpoint
best_ckpt, _ = train_one_run(best_hparams)
best_path = best_ckpt

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/mdeberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\adishalit1\AppData\Local\Temp\ipykernel_35636\1725332257.py:207: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=(DEVICE == "cuda" and USE_AMP))
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: adishalit1 (adishalit1-tel-aviv-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' 

Trainable params: 85,648,901 / 278,813,189 (30.72%) ; unfreeze_last_k=12


C:\Users\adishalit1\AppData\Local\Temp\ipykernel_35636\1725332257.py:260: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),


[e1 b1/2315] loss=1.5977 avg=1.5977 it/s=30.4
[e1 b2/2315] loss=1.6135 avg=1.6056 it/s=44.5
[e1 b232/2315] loss=1.5696 avg=1.5877 it/s=315.9
[e1 b463/2315] loss=1.3081 avg=1.5419 it/s=332.9
[e1 b694/2315] loss=1.2160 avg=1.4361 it/s=341.6
[e1 b925/2315] loss=1.0303 avg=1.3292 it/s=345.2
[e1 b1156/2315] loss=1.4365 avg=1.2427 it/s=346.7
[e1 b1387/2315] loss=1.0453 avg=1.1761 it/s=349.4
[e1 b1618/2315] loss=0.6425 avg=1.1249 it/s=350.5
[e1 b1849/2315] loss=0.3707 avg=1.0793 it/s=349.3
[e1 b2080/2315] loss=0.5823 avg=1.0368 it/s=347.1
[e1 b2311/2315] loss=0.6791 avg=1.0018 it/s=346.2


C:\Users\adishalit1\AppData\Local\Temp\ipykernel_35636\1725332257.py:182: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),


...

[e10 b232/2315] loss=0.0031 avg=0.1081 it/s=352.8
[e10 b463/2315] loss=0.0052 avg=0.1049 it/s=358.1
[e10 b694/2315] loss=0.0366 avg=0.1007 it/s=352.3
[e10 b925/2315] loss=0.0026 avg=0.1015 it/s=351.9
[e10 b1156/2315] loss=0.0023 avg=0.1016 it/s=352.2
[e10 b1387/2315] loss=0.0039 avg=0.1013 it/s=348.6
[e10 b1618/2315] loss=0.4062 avg=0.1009 it/s=347.3
[e10 b1849/2315] loss=0.0101 avg=0.1008 it/s=348.6
[e10 b2080/2315] loss=0.1211 avg=0.1012 it/s=349.4
[e10 b2311/2315] loss=0.5247 avg=0.0987 it/s=350.6


C:\Users\adishalit1\AppData\Local\Temp\ipykernel_35636\1725332257.py:182: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Epoch 10/10 | loss=0.0987 | val_acc=0.8635 | val_f1=0.8672 | time=110.3s


epoch,▁▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▆▆▆▆▆▆▆▆▆▆▆▇▇███
lr,█▇▆▆▅▄▃▃▂▁
params/ratio,▁
params/total,▁
params/trainable,▁
step,▁▁▂▂▂▁▁▁▂▂▂▁▁▁▁▂▁▁▂▅▁▁▂▂▁▂▇▁▂▂█▁▂▂▁▂▂▁▁▂
time/epoch_sec,▄▃▂▃█▂▁▇▇▃
train/avg_loss_so_far,██▇▇▆▄▃▃▃▃▃▃▃▃▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▃▁▁▁
train/epoch_loss,█▄▃▃▂▂▂▁▁▁
train/items_per_sec,▁▁▆▆▆▆▆▆▆▅▆▆▆▆▆▆▅▆▆▆▆▆▆▆▆▆▆▆▆▆▆█▇▆▆▆▆▆▆▆
train/loss,██▃▄▄▄▄▃▁▃▃▃▂▂▄▂▁▃▂▃▂▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▅▂


C:\Users\adishalit1\AppData\Local\Temp\ipykernel_35636\1725332257.py:182: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),


## 💾Save Best Model & Test

After loading the best hyperparameters, we retrained the model from scratch to get a **clean checkpoint**.  
This ensures that the final model is trained only with the best settings found by Optuna (val/F1 ≈ 0.88022).  

For evaluation, we tested on the **clean translated test set**, since the model was also trained on **cleaned and translated data**.  
This way, the evaluation setup matches the training conditions.  

### 📊 Final Results (Validation / Test)
- **Best validation F1:** 0.87125 (epoch 9)  
- **Validation Accuracy:** 0.86346  
- **Validation Precision:** 0.86194  
- **Validation Recall:** 0.87403  
- **Validation F1 (last epoch):** 0.86723  

The best checkpoint was saved at:  
`checkpoints/best_microsoft_mdeberta-v3-base_full_ex_4_best_optuna_retrain.pt`  

### Training Stats
- Trainable parameters: **85.6M / 278.8M** (~30.7%)  
- Average training loss (per epoch): **0.0987**  
- Throughput: ~**350 items/sec**  
- Time per epoch: ~**110 sec**  

These results confirm that our setup is close to the reported performance of **mDeBERTa-v3-base on English (≈88.2 F1)** and provides a strong baseline.
___

## Test

In [11]:
# # Retrain best config to get a clean checkpoint
# best_ckpt, _ = train_one_run(best_params)
# best_path = best_ckpt
best_params=best_hparams
# -------------------------
# Final evaluation on TEST (+ W&B logging)
# -------------------------
model = build_model(best_params["num_unfreeze_last_layers"])
model.load_state_dict(torch.load(best_path, map_location=DEVICE))
model.eval()

all_preds, all_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(DEVICE, non_blocking=True) for k, v in batch.items()}
        with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),
                      enabled=(DEVICE == "cuda" and USE_AMP)):
            logits = model(**batch).logits
        all_preds.extend(torch.argmax(logits, dim=-1).detach().cpu().tolist())
        all_labels.extend(batch["labels"].detach().cpu().tolist())

acc = accuracy_score(all_labels, all_preds)
p, r, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="macro", zero_division=0)
print(f"\nTEST | acc={acc:.4f} | f1_macro={f1:.4f} | precision_macro={p:.4f} | recall_macro={r:.4f}\n")

print("Per-class report (ids map to labels):")
print(ID2LABEL)
report = classification_report(
    all_labels, all_preds,
    target_names=[ID2LABEL[i] for i in range(len(ORDER))],
    zero_division=0, output_dict=True
)
print(classification_report(
    all_labels, all_preds,
    target_names=[ID2LABEL[i] for i in range(len(ORDER))],
    zero_division=0
))

# # ---- W&B: log test metrics, per-class scores, and confusion matrix ----
# test_run = wandb.init(project=PROJECT, name=f"{BASE_RUN_NAME}_test", resume="allow", reinit=True)
# log_payload = {
#     "test/acc": acc,
#     "test/precision_macro": p,
#     "test/recall_macro": r,
#     "test/f1_macro": f1,
# }
# for cls_name in ORDER:
#     if cls_name in report:
#         log_payload[f"test/{cls_name}/precision"] = report[cls_name]["precision"]
#         log_payload[f"test/{cls_name}/recall"]    = report[cls_name]["recall"]
#         log_payload[f"test/{cls_name}/f1"]        = report[cls_name]["f1-score"]
#
# wandb.log(log_payload)
#
# cm = confusion_matrix(all_labels, all_preds, labels=list(range(len(ORDER))))
# wandb.log({
#     "test/confusion_matrix": wandb.plot.confusion_matrix(
#         y_true=all_labels,
#         preds=all_preds,
#         class_names=[ID2LABEL[i] for i in range(len(ID2LABEL))]
#     )
# })
# test_run.summary["best_checkpoint_path"] = best_path
# test_run.summary["test_f1_macro"] = f1
# wandb.finish()


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/mdeberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\adishalit1\AppData\Local\Temp\ipykernel_35636\582332933.py:16: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),



TEST | acc=0.8662 | f1_macro=0.8685 | precision_macro=0.8680 | recall_macro=0.8697

Per-class report (ids map to labels):
{0: 'extremely negative', 1: 'negative', 2: 'neutral', 3: 'positive', 4: 'extremely positive'}
                    precision    recall  f1-score   support

extremely negative       0.89      0.89      0.89       592
          negative       0.85      0.88      0.87      1041
           neutral       0.85      0.87      0.86       619
          positive       0.88      0.81      0.84       947
extremely positive       0.87      0.89      0.88       599

          accuracy                           0.87      3798
         macro avg       0.87      0.87      0.87      3798
      weighted avg       0.87      0.87      0.87      3798



# ✅ Test Results – Clean Translated Set

When evaluating on the **clean translated test set** (same distribution as training data), the model reached:

- 🎯 **Accuracy:** 0.8662  
- 📊 **Macro F1:** 0.8685  
- 🧮 **Macro Precision:** 0.8680  
- 🔄 **Macro Recall:** 0.8697  

---

### 🔎 Per-class breakdown
- 😡 **Extremely Negative:** F1 = **0.89** → very strong & consistent  
- 🙁 **Negative:** F1 = **0.87**  
- 😐 **Neutral:** F1 = **0.86**  
- 🙂 **Positive:** F1 = **0.84** → slightly weaker, recall drop  
- 🤩 **Extremely Positive:** F1 = **0.88** → very strong & consistent  

---

### 📌 Takeaway
The model performs **best on the extreme sentiment classes** (extremely negative / extremely positive), where the signal is more clear.  
Performance is a bit weaker for **positive vs neutral**, which makes sense since these classes are often harder to separate in real tweets.  


## Export: Model, Tokenizer, Metrics & Artifacts (Exercise 4)

We export the **retrained best model** and all artifacts so the run is fully reproducible and easy to reload elsewhere.

**What gets saved:**
- 🤗 **HF model + tokenizer** (config + weights + tokenizer files)
- 🧠 **Raw best state_dict** snapshot (`best_state_dict.pt`) from early stopping
- 📊 **Test metrics** (`test_metrics.json`)
- 📈 **Classification report** CSV and **confusion matrix** CSV
- 🏷️ **Label mapping** (`labels.json`) with `order`, `label2id`, `id2label`
- ⚙️ **Best hyperparameters** (`best_hparams_ex4.json`)
- 📄 **README** with reload instructions

**Output directory:**
`ex_4_model/microsoft__mdeberta-v3-base_full_ex_4_YYYYMMDD_HHMMSS`

This makes it simple to reload the exact model and tokenizer later, or to load the raw `state_dict` into a fresh model with the same head and label mapping.


In [12]:
# === Save full EX.4 export (model, tokenizer, metrics, hparams) ===
import os, json, time, shutil
from pathlib import Path
import pandas as pd
from sklearn.metrics import confusion_matrix

# Root folder + unique subdir (so you can export multiple times safely)
timestamp = time.strftime("%Y%m%d_%H%M%S")
run_stub  = (BASE_RUN_NAME if 'BASE_RUN_NAME' in globals() else 'ex4').replace("/", "__").replace("\\", "__")
export_dir = os.path.join("ex_4_model", f"{run_stub}_{timestamp}")
os.makedirs(export_dir, exist_ok=True)

# 1) Save the model in Hugging Face format + tokenizer
# (Your current `model` already has id2label/label2id in its config from build_model)
model.save_pretrained(export_dir)          # writes config.json + pytorch_model.bin
tokenizer.save_pretrained(export_dir)      # writes tokenizer files into same folder

# Also keep the raw state_dict snapshot that early stopping wrote
try:
    shutil.copy2(best_path, os.path.join(export_dir, "best_state_dict.pt"))
except Exception as e:
    print(f"[export] Warning: couldn't copy raw state_dict from {best_path}: {e}")

# 2) Save test artifacts (report CSV, confusion matrix CSV, metrics JSON)
rep_df = pd.DataFrame(report).transpose()
cm     = confusion_matrix(all_labels, all_preds, labels=list(range(len(ORDER))))
cm_df  = pd.DataFrame(cm, index=[f"true_{c}" for c in ORDER], columns=[f"pred_{c}" for c in ORDER])

rep_path   = os.path.join(export_dir, "classification_report_test.csv")
cm_path    = os.path.join(export_dir, "confusion_matrix_test.csv")
metrics_js = os.path.join(export_dir, "test_metrics.json")

rep_df.to_csv(rep_path, index=True)
cm_df.to_csv(cm_path, index=True)
with open(metrics_js, "w", encoding="utf-8") as f:
    json.dump(
        {
            "test_accuracy": float(acc),
            "test_precision_macro": float(p),
            "test_recall_macro": float(r),
            "test_f1_macro": float(f1),
        },
        f,
        indent=2,
    )

# 3) Save label mapping + order (useful when reloading elsewhere)
labels_js = os.path.join(export_dir, "labels.json")
with open(labels_js, "w", encoding="utf-8") as f:
    json.dump(
        {
            "order": ORDER,
            "label2id": LABEL2ID,
            "id2label": {int(k): v for k, v in ID2LABEL.items()},
        },
        f,
        indent=2,
    )

# 4) Save best hyperparameters that produced the checkpoint
hparams = {}
if 'best_params'   in globals(): hparams = best_params
elif 'best_hparams' in globals(): hparams = best_hparams

hparams_js = os.path.join(export_dir, "best_hparams_ex4.json")
with open(hparams_js, "w", encoding="utf-8") as f:
    json.dump(hparams, f, indent=2)

# 5) Tiny README with reload instructions
readme_txt = os.path.join(export_dir, "README.txt")
with open(readme_txt, "w", encoding="utf-8") as f:
    f.write(
        "Exercise-4 export\n"
        f"Model: {MODEL_NAME}\n"
        f"Exported at: {timestamp}\n"
        f"Run stub: {run_stub}\n"
        f"Labels (order): {ORDER}\n"
        f"Best hparams: {hparams}\n"
        "\n"
        "How to reload:\n"
        "----------------\n"
        "from transformers import AutoModelForSequenceClassification, AutoTokenizer\n"
        f"model = AutoModelForSequenceClassification.from_pretrained(r\"{export_dir}\")\n"
        f"tokenizer = AutoTokenizer.from_pretrained(r\"{export_dir}\")\n"
        "\n"
        "You can also load the raw state_dict (best_state_dict.pt) into a fresh model created\n"
        "with the same head and id2label/label2id mapping.\n"
    )

print("\n=== EX.4 EXPORT DONE ===")
print("Saved to:", export_dir)
print(" • HF model + tokenizer")
print(" • Original best state_dict snapshot →", os.path.join(export_dir, "best_state_dict.pt"))
print(" • Test metrics JSON →", metrics_js)
print(" • Classification report CSV →", rep_path)
print(" • Confusion matrix CSV →", cm_path)
print(" • Label mapping JSON →", labels_js)
print(" • Best hparams JSON →", hparams_js)



=== EX.4 EXPORT DONE ===
Saved to: ex_4_model\microsoft__mdeberta-v3-base_full_ex_4_20250817_133620
 • HF model + tokenizer
 • Original best state_dict snapshot → ex_4_model\microsoft__mdeberta-v3-base_full_ex_4_20250817_133620\best_state_dict.pt
 • Test metrics JSON → ex_4_model\microsoft__mdeberta-v3-base_full_ex_4_20250817_133620\test_metrics.json
 • Classification report CSV → ex_4_model\microsoft__mdeberta-v3-base_full_ex_4_20250817_133620\classification_report_test.csv
 • Confusion matrix CSV → ex_4_model\microsoft__mdeberta-v3-base_full_ex_4_20250817_133620\confusion_matrix_test.csv
 • Label mapping JSON → ex_4_model\microsoft__mdeberta-v3-base_full_ex_4_20250817_133620\labels.json
 • Best hparams JSON → ex_4_model\microsoft__mdeberta-v3-base_full_ex_4_20250817_133620\best_hparams_ex4.json


In [10]:
# ---- W&B: log test metrics, per-class scores, and confusion matrix ----
test_run = wandb.init(project=PROJECT, name=f"{BASE_RUN_NAME}_test", resume="allow", reinit=True)
log_payload = {
    "test/acc": acc,
    "test/precision_macro": p,
    "test/recall_macro": r,
    "test/f1_macro": f1,
}
for cls_name in ORDER:
    if cls_name in report:
        log_payload[f"test/{cls_name}/precision"] = report[cls_name]["precision"]
        log_payload[f"test/{cls_name}/recall"]    = report[cls_name]["recall"]
        log_payload[f"test/{cls_name}/f1"]        = report[cls_name]["f1-score"]

wandb.log(log_payload)

cm = confusion_matrix(all_labels, all_preds, labels=list(range(len(ORDER))))
wandb.log({
    "test/confusion_matrix": wandb.plot.confusion_matrix(
        y_true=all_labels,
        preds=all_preds,
        class_names=[ID2LABEL[i] for i in range(len(ID2LABEL))]
    )
})
test_run.summary["best_checkpoint_path"] = best_path
test_run.summary["test_f1_macro"] = f1
wandb.finish()


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


test/acc,▁
test/extremely negative/f1,▁
test/extremely negative/precision,▁
test/extremely negative/recall,▁
test/extremely positive/f1,▁
test/extremely positive/precision,▁
test/extremely positive/recall,▁
test/f1_macro,▁
test/negative/f1,▁
test/negative/precision,▁
test/negative/recall,▁


# 🚀 Exercise 5 – Training with HuggingFace Libraries

In this part we re-implement our training pipeline using the **HuggingFace `transformers` library**.  
Instead of writing a custom training loop, we use the built-in **`Trainer` API**, which makes fine-tuning large models simpler and more reproducible.  

### 🔑 What’s different here
- Use of **`TrainingArguments`** and **`Trainer`** for training, evaluation, and logging.  
- Automatic support for **mixed precision (fp16/bf16)**, **gradient accumulation**, and **checkpointing**.  
- Direct integration with **evaluation metrics (accuracy, F1, precision, recall)**.  
- Cleaner and more modular workflow compared to Exercise 4.  

### 🎯 Goal
Train the **mDeBERTa-v3-base** model on our **clean translated dataset**, with the best hyperparameters found earlier, and validate that the HuggingFace training pipeline gives consistent results with our custom Optuna-based loop.  
___

# 🚀 Ex.5 – First HF Trainer Run (Best Hyperparameters from Before)

In this section we move from our **custom training loop** to the **HuggingFace `Trainer` API**,  
using the *best hyperparameters we found earlier with Optuna*:

- Model: **mDeBERTa-v3-base**  
- Batch size = 16  
- LR = 3e-5  
- Weight decay = 0.05  
- Epochs = 12  
- Unfreeze last **10 layers**  

We also keep all the nice features:
- ✅ `wandb` logging  
- ✅ Early stopping via patience  
- ✅ Mixed precision (fp16/bf16)  
- ✅ Print-friendly callback (for live tracking)  

👉 The goal here is to reproduce the strong setup we had before, but now inside HuggingFace’s official training loop.  
Later, we will **discuss why the results weren’t as good as expected** and what might explain the gap compared to our previous loop.  



# Training EX5

In [ ]:
# --- Ex.5: HF Trainer version with W&B + prints (Windows/RTX 4090 friendly) ---

import os, math, random, time, json
from typing import Dict, Any

import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix

import wandb
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    DataCollatorWithPadding, Trainer, TrainingArguments
)
from transformers.trainer_callback import TrainerCallback, TrainerState, TrainerControl

# ---- Fast CUDA defaults (4090) ----
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")
if torch.cuda.is_available():
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    try:
        torch.set_float32_matmul_precision("high")
    except Exception:
        pass

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42
def set_seed(s=42):
    random.seed(s); np.random.seed(s)
    torch.manual_seed(s); torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True
set_seed(SEED)

# ---- W&B defaults ----
os.environ.setdefault("WANDB_MODE", "online")
os.environ.setdefault("WANDB_PROJECT", "adv-dl-p2")
os.environ.setdefault("WANDB_NOTEBOOK_NAME", "ex5_trainer.ipynb")
WANDB_PROJECT = os.environ["WANDB_PROJECT"]

# ---- Constants ----
MODEL_NAME = "microsoft/mdeberta-v3-base"
BASE_RUN_NAME = MODEL_NAME.replace("/", "__") + "_ex5_trainer"
MAX_LEN = 512
BATCH_SIZE = 16
EPOCHS = 12
PATIENCE = 4
LR = 3e-5
WEIGHT_DECAY = 0.05
WARMUP_RATIO = 0.06
GRAD_ACCUM = 1
NUM_WORKERS = 0  # Windows-safe; raise to 2 if stable

# ---- Label mapping (5-way) ----
CANON = {
    "extremely negative": "extremely negative",
    "negative": "negative",
    "neutral": "neutral",
    "positive": "positive",
    "extremely positive": "extremely positive",
}
ORDER = ["extremely negative","negative","neutral","positive","extremely positive"]
LABEL2ID = {lab: i for i, lab in enumerate(ORDER)}
ID2LABEL = {i: lab for lab, i in LABEL2ID.items()}

def normalize_label(s: str) -> str:
    s = str(s).strip().lower()
    s = s.replace("very negative", "extremely negative")
    s = s.replace("very positive", "extremely positive")
    s = s.replace("extreme negative", "extremely negative")
    s = s.replace("extreme positive", "extremely positive")
    return CANON.get(s, s)

# ---- Prep dataframes from df_train/df_test already in memory ----
assert "OriginalTweet" in df_train.columns and "Sentiment" in df_train.columns
assert "OriginalTweet" in df_test.columns and "Sentiment" in df_test.columns

def prep_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.dropna(subset=["OriginalTweet", "Sentiment"]).copy()
    df["text"] = df["OriginalTweet"].astype(str).str.strip()
    df["label_name"] = df["Sentiment"].apply(normalize_label)
    df = df[df["label_name"].isin(ORDER)].reset_index(drop=True)
    df["labels"] = df["label_name"].map(LABEL2ID)
    return df[["text", "labels", "label_name"]]

dftrain_ = prep_df(df_train)
dftest_  = prep_df(df_test)

train_df, val_df = train_test_split(
    dftrain_, test_size=0.1, stratify=dftrain_["labels"], random_state=SEED
)

print(f"Train/Val/Test sizes: {len(train_df)}/{len(val_df)}/{len(dftest_)}")


In [14]:
from datasets import Dataset
from transformers import DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

def tok_fn(batch):
    return tokenizer(batch["text"], padding=False, truncation=True, max_length=MAX_LEN)

def keep_only(ds, cols):
    # Use select_columns if available, else remove the others
    if hasattr(ds, "select_columns"):
        return ds.select_columns(cols)
    drop = [c for c in ds.column_names if c not in cols]
    return ds.remove_columns(drop)

# Build datasets (avoid adding __index_level_0__ with preserve_index=False)
train_ds = Dataset.from_pandas(train_df,  preserve_index=False)
val_ds   = Dataset.from_pandas(val_df,    preserve_index=False)
test_ds  = Dataset.from_pandas(dftest_,   preserve_index=False)

# Keep just text + labels before tokenization
cols_keep = ["text", "labels"]
train_ds = keep_only(train_ds, cols_keep)
val_ds   = keep_only(val_ds,   cols_keep)
test_ds  = keep_only(test_ds,  cols_keep)

# Tokenize
train_ds = train_ds.map(tok_fn, batched=True)
val_ds   = val_ds.map(tok_fn,   batched=True)
test_ds  = test_ds.map(tok_fn,  batched=True)

# Remove raw text after tokenization, keep labels + token IDs
train_ds = train_ds.remove_columns(["text"])
val_ds   = val_ds.remove_columns(["text"])
test_ds  = test_ds.remove_columns(["text"])

# Ensure final columns are exactly what Trainer expects
final_cols = ["input_ids", "attention_mask", "labels"]
train_ds = keep_only(train_ds, final_cols)
val_ds   = keep_only(val_ds,   final_cols)
test_ds  = keep_only(test_ds,  final_cols)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=8)


C:\Users\adishalit1\AppData\Local\anaconda3\envs\dl4090\Lib\site-packages\transformers\convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Map: 100%|██████████| 3798/3798 [00:00<00:00, 36229.75 examples/s]


In [15]:
from transformers import AutoModelForSequenceClassification

UNFREEZE_LAST_K = 10  # 1..12 are sensible for DeBERTa-v3-base

def build_model(unfreeze_last_k=UNFREEZE_LAST_K):
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=len(ORDER),
        id2label=ID2LABEL,
        label2id=LABEL2ID,
        torch_dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else None),
        use_safetensors=True,
    )
    base = getattr(model, "deberta", None) or getattr(model, "roberta", None) or getattr(model, "bert", None)
    if base is not None and hasattr(base, "encoder") and hasattr(base.encoder, "layer"):
        # freeze all
        for p in base.parameters(): p.requires_grad = False
        # unfreeze last k transformer blocks
        for layer in base.encoder.layer[-int(unfreeze_last_k):]:
            for p in layer.parameters(): p.requires_grad = True
    # classifier head always trainable
    for p in model.classifier.parameters(): p.requires_grad = True
    return model.to(DEVICE)

model = build_model()
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable:,} / {total:,} ({100*trainable/total:.2f}%) ; unfreeze_last_k={UNFREEZE_LAST_K}")


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/mdeberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable params: 71,473,157 / 278,813,189 (25.63%) ; unfreeze_last_k=10


In [16]:
import evaluate
metric_acc = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    acc = metric_acc.compute(predictions=preds, references=labels)["accuracy"]
    p, r, f1, _ = precision_recall_fscore_support(labels, preds, average="macro", zero_division=0)
    return {"accuracy": acc, "precision": p, "recall": r, "f1": f1}

class PrintAndWBCallback(TrainerCallback):
    def __init__(self, print_every=20):
        self.print_every = print_every
        self.steps_per_epoch = None
        self.last_print_step = -1

    def on_train_begin(self, args, state: TrainerState, control: TrainerControl, **kwargs):
        # steps/epoch computed via train_dataloader length
        trainer = kwargs.get("model")  # not available here; compute later in on_step_begin
        print(f"[Run] epochs={args.num_train_epochs} bs={args.per_device_train_batch_size} "
              f"lr={args.learning_rate:.2e} wd={args.weight_decay:.1e} "
              f"warmup_ratio={args.warmup_ratio} grad_accum={args.gradient_accumulation_steps}")

    def on_step_begin(self, args, state: TrainerState, control: TrainerControl, **kwargs):
        # set steps_per_epoch once we have dataloader info exposed in state
        if self.steps_per_epoch is None and state.max_steps is not None and state.num_train_epochs:
            approx_steps_total = state.max_steps
            self.steps_per_epoch = max(1, int(approx_steps_total / math.ceil(state.num_train_epochs)))

    def on_log(self, args, state: TrainerState, control: TrainerControl, logs=None, **kwargs):
        if logs is None: return
        # mimic: [e1 b123/...] loss=... it/s=...
        if self.steps_per_epoch:
            step_in_epoch = (state.global_step % self.steps_per_epoch) or self.steps_per_epoch
            if step_in_epoch % self.print_every == 0 and state.global_step != self.last_print_step:
                loss = logs.get("loss", logs.get("train_loss", None))
                lr = logs.get("learning_rate", None)
                sps = logs.get("train_samples_per_second", None) or logs.get("samples_per_second", None)
                txt = f"[e{int(state.epoch or 0)} b{step_in_epoch}/{self.steps_per_epoch}]"
                if loss is not None: txt += f" loss={loss:.4f}"
                if lr   is not None: txt += f" lr={lr:.2e}"
                if sps  is not None: txt += f" samp/s={sps:.1f}"
                print(txt, flush=True)
                self.last_print_step = state.global_step

    def on_evaluate(self, args, state: TrainerState, control: TrainerControl, metrics, **kwargs):
        if metrics:
            msg = (f"[val @ epoch {int(state.epoch or 0)}] "
                   f"acc={metrics.get('eval_accuracy',0):.4f} "
                   f"f1={metrics.get('eval_f1',0):.4f} "
                   f"p={metrics.get('eval_precision',0):.4f} "
                   f"r={metrics.get('eval_recall',0):.4f}")
            print(msg, flush=True)

safe_name = BASE_RUN_NAME
out_dir = os.path.join("hf_ckpts", safe_name)
os.makedirs(out_dir, exist_ok=True)

bf16_ok = torch.cuda.is_available() and torch.cuda.is_bf16_supported()

training_args = TrainingArguments(
    output_dir=out_dir,
    run_name=safe_name,
    report_to=["wandb"],
    learning_rate=LR,
    weight_decay=WEIGHT_DECAY,
    warmup_ratio=WARMUP_RATIO,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,
    dataloader_num_workers=NUM_WORKERS,
    fp16=not bf16_ok and torch.cuda.is_available(),
    bf16=bf16_ok,
    logging_strategy="steps",
    logging_steps=20,
    eval_strategy="epoch",            # (new name replacing evaluation_strategy)
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    save_total_limit=2,
)

wandb_run = wandb.init(
    project=WANDB_PROJECT,
    name=safe_name,
    config={
        "model": MODEL_NAME,
        "max_len": MAX_LEN,
        "batch_size": BATCH_SIZE,
        "epochs": EPOCHS,
        "lr": LR,
        "weight_decay": WEIGHT_DECAY,
        "warmup_ratio": WARMUP_RATIO,
        "grad_accum": GRAD_ACCUM,
        "unfreeze_last_k": UNFREEZE_LAST_K,
    },
    reinit=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[PrintAndWBCallback(print_every=20)],
)

print(f"[Trainer] Starting fine-tune → output_dir={out_dir}")
train_out = trainer.train()
print(train_out)

# best checkpoint path on disk
best_path = trainer.state.best_model_checkpoint
print("Best checkpoint dir:", best_path)
wandb_run.summary["best_checkpoint_dir"] = best_path
wandb.finish()


C:\Users\adishalit1\AppData\Local\Temp\ipykernel_21192\3450032475.py:102: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


[Trainer] Starting fine-tune → output_dir=hf_ckpts\microsoft__mdeberta-v3-base_ex5_trainer
[Run] epochs=12 bs=16 lr=3.00e-05 wd=5.0e-02 warmup_ratio=0.06 grad_accum=1


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.213200,1.232292,0.482750,0.500943,0.507310,0.496841
2,1.080600,1.058841,0.561467,0.584520,0.592781,0.577479
3,0.962200,0.983838,0.596210,0.615364,0.619935,0.611747
4,0.895000,0.967778,0.605199,0.623989,0.630168,0.619882
5,0.954000,0.949300,0.618319,0.632191,0.640670,0.632150
6,0.939800,0.952293,0.613703,0.634493,0.636548,0.628234
7,0.953200,0.948852,0.611516,0.631063,0.634211,0.626338
8,0.999500,0.942999,0.613703,0.630220,0.638575,0.628589
9,0.902200,0.946464,0.616861,0.636011,0.638370,0.630863
10,0.953400,0.946236,0.615889,0.633949,0.638359,0.630118


[e0 b20/2315] loss=1.6226 lr=3.42e-07
[e0 b40/2315] loss=1.6173 lr=7.02e-07
[e0 b60/2315] loss=1.6075 lr=1.06e-06
[e0 b80/2315] loss=1.6150 lr=1.42e-06
[e0 b100/2315] loss=1.6179 lr=1.78e-06
[e0 b120/2315] loss=1.6180 lr=2.14e-06
[e0 b140/2315] loss=1.6063 lr=2.50e-06
[e0 b160/2315] loss=1.6032 lr=2.86e-06
[e0 b180/2315] loss=1.6073 lr=3.22e-06
[e0 b200/2315] loss=1.6209 lr=3.58e-06
[e0 b220/2315] loss=1.6040 lr=3.94e-06
[e0 b240/2315] loss=1.6119 lr=4.30e-06
[e0 b260/2315] loss=1.6084 lr=4.66e-06
[e0 b280/2315] loss=1.5997 lr=5.02e-06
[e0 b300/2315] loss=1.6078 lr=5.38e-06
[e0 b320/2315] loss=1.6032 lr=5.74e-06
[e0 b340/2315] loss=1.6153 lr=6.10e-06
[e0 b360/2315] loss=1.6092 lr=6.46e-06
[e0 b380/2315] loss=1.6029 lr=6.82e-06
[e0 b400/2315] loss=1.6028 lr=7.18e-06
[e0 b420/2315] loss=1.5912 lr=7.54e-06
[e0 b440/2315] loss=1.5979 lr=7.90e-06
[e0 b460/2315] loss=1.6232 lr=8.26e-06
[e0 b480/2315] loss=1.5998 lr=8.62e-06
[e0 b500/2315] loss=1.6050 lr=8.98e-06
[e0 b520/2315] loss=1.5995 lr

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


TrainOutput(global_step=27780, training_loss=1.0007773465952314, metrics={'train_runtime': 1182.5271, 'train_samples_per_second': 375.863, 'train_steps_per_second': 23.492, 'total_flos': 1.887617915188579e+16, 'train_loss': 1.0007773465952314, 'epoch': 12.0})
Best checkpoint dir: hf_ckpts\microsoft__mdeberta-v3-base_ex5_trainer\checkpoint-11575


eval/accuracy,▁▅▇▇████████
eval/f1,▁▅▇▇████████
eval/loss,█▄▂▂▁▁▁▁▁▁▁▁
eval/precision,▁▅▇▇████████
eval/recall,▁▅▇▇████████
eval/runtime,▂▁▅▅▇▅▄▅▂▇█▂
eval/samples_per_second,▇█▄▄▂▄▅▃▇▂▁▆
eval/steps_per_second,▇█▄▄▂▄▅▃▇▂▁▆
train/epoch,▁▁▁▁▁▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇███
train/global_step,▁▁▁▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇█
train/grad_norm,▁▁▁▁▂▂▂█▂▂▂▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▃▂▄▂▂▂▂▂▂▂▂▂▂


## HP Tuning Optuna

# 🧪 Ex.5 — Hyperparameter Tuning with HuggingFace `Trainer` (+ Optuna)

Now that the basic HF `Trainer` pipeline works, we’re doing a **focused HP search** directly on top of it.  
Goal: keep the Trainer setup clean and stable while letting **Optuna** explore a *small, sensible* space around our earlier best settings.

---

## 🔧 What we’re tuning (student-style plan)
- **Learning rate**: `1e-5 → 1e-4` (log scale)  
  *Why*: this is the sweet spot we found before; we’ll confirm it under the Trainer loop.
- **Weight decay**: `1e-6 → 1e-4`  
  *Why*: larger values hurt before; we constrain to the “safe” zone.
- **Unfreezing depth**: **8–12** last layers  
  *Why*: deeper unfreezing won in Ex.4; we test if full (12) still dominates here.
- **Batch size**: **{4, 8, 16, 32}**  
  
---

## 🏗️ How we run it
- **Trainer factory per trial** → builds a fresh model and `TrainingArguments` for the sampled HPs.
- **Callbacks**:  
  - Custom print logger (lighter than full tqdm).  
  - **EarlyStoppingCallback** with patience = **4**.  
- **bf16/fp16** auto-detection for speed, dynamic padding (pad to 8), and fused AdamW where possible.
- **W&B**: each trial is a separate run, grouped under a single **study** for easy comparison.

---

## 🎯 What we expect
- Best trials should cluster again around **LR ≈ few×1e-5**, **low weight decay**, and **deep unfreezing** (≥9, often 12).  
- Smaller batches (**8–16**) typically remain more stable than very large ones.

---

## 📦 After the study
We **save the best params to JSON** (`hf_ckpts/best_params_optuna{i}.json`) so we can retrain cleanly later  
(with possibly more epochs) without depending on the Optuna object.
 ---
## Wadb Link
https://wandb.ai/adishalit1-tel-aviv-university/adv-dl-p2-ex-5-deberta-16-08-try

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
# Load CSVs (your files have columns: ['UserName','ScreenName','Location','TweetAt','OriginalTweet','Sentiment'])
TRAIN_CSV = "Corona_NLP_train_cleaned_translated.csv"   # or "Corona_NLP_train.csv"
TEST_CSV  = "Corona_NLP_test_cleaned_translated.csv"    # or "Corona_NLP_test.csv"


df_train = pd.read_csv(TRAIN_CSV, encoding="utf-8", engine="python")
df_test  = pd.read_csv(TEST_CSV,  encoding="utf-8", engine="python")

In [4]:
# --- Ex.5: HF Trainer version with W&B + prints (Windows/RTX 4090 friendly) ---

import os, math, random, time, json
from typing import Dict, Any

import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix

import wandb
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    DataCollatorWithPadding, Trainer, TrainingArguments
)
from transformers.trainer_callback import TrainerCallback, TrainerState, TrainerControl

# ---- Fast CUDA defaults (4090) ----
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")
if torch.cuda.is_available():
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    try:
        torch.set_float32_matmul_precision("high")
    except Exception:
        pass

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42
def set_seed(s=42):
    random.seed(s); np.random.seed(s)
    torch.manual_seed(s); torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True
set_seed(SEED)

# ---- W&B defaults ----
os.environ.setdefault("WANDB_MODE", "online")
os.environ.setdefault("WANDB_PROJECT", "adv-dl-p2-ex-5-deberta-16-08-try")
os.environ.setdefault("WANDB_NOTEBOOK_NAME", "ex5_trainer_new.ipynb")
WANDB_PROJECT = os.environ["WANDB_PROJECT"]

# ---- Constants ----
MODEL_NAME = "microsoft/mdeberta-v3-base"
BASE_RUN_NAME = MODEL_NAME.replace("/", "__") + "_ex5_trainer-try"
MAX_LEN = 512
BATCH_SIZE = 16
EPOCHS = 12
PATIENCE = 4
LR = 3e-5
WEIGHT_DECAY = 0.05
WARMUP_RATIO = 0.06
GRAD_ACCUM = 1
NUM_WORKERS = 0  # Windows-safe; raise to 2 if stable

# ---- Label mapping (5-way) ----
CANON = {
    "extremely negative": "extremely negative",
    "negative": "negative",
    "neutral": "neutral",
    "positive": "positive",
    "extremely positive": "extremely positive",
}
ORDER = ["extremely negative","negative","neutral","positive","extremely positive"]
LABEL2ID = {lab: i for i, lab in enumerate(ORDER)}
ID2LABEL = {i: lab for lab, i in LABEL2ID.items()}

def normalize_label(s: str) -> str:
    s = str(s).strip().lower()
    s = s.replace("very negative", "extremely negative")
    s = s.replace("very positive", "extremely positive")
    s = s.replace("extreme negative", "extremely negative")
    s = s.replace("extreme positive", "extremely positive")
    return CANON.get(s, s)

# ---- Prep dataframes from df_train/df_test already in memory ----
assert "OriginalTweet" in df_train.columns and "Sentiment" in df_train.columns
assert "OriginalTweet" in df_test.columns and "Sentiment" in df_test.columns

def prep_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.dropna(subset=["OriginalTweet", "Sentiment"]).copy()
    df["text"] = df["OriginalTweet"].astype(str).str.strip()
    df["label_name"] = df["Sentiment"].apply(normalize_label)
    df = df[df["label_name"].isin(ORDER)].reset_index(drop=True)
    df["labels"] = df["label_name"].map(LABEL2ID)
    return df[["text", "labels", "label_name"]]

dftrain_ = prep_df(df_train)
dftest_  = prep_df(df_test)

train_df, val_df = train_test_split(
    dftrain_, test_size=0.1, stratify=dftrain_["labels"], random_state=SEED
)

print(f"Train/Val/Test sizes: {len(train_df)}/{len(val_df)}/{len(dftest_)}")

from datasets import Dataset
from transformers import DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

def tok_fn(batch):
    return tokenizer(batch["text"], padding=False, truncation=True, max_length=MAX_LEN)

def keep_only(ds, cols):
    # Use select_columns if available, else remove the others
    if hasattr(ds, "select_columns"):
        return ds.select_columns(cols)
    drop = [c for c in ds.column_names if c not in cols]
    return ds.remove_columns(drop)

# Build datasets (avoid adding __index_level_0__ with preserve_index=False)
train_ds = Dataset.from_pandas(train_df,  preserve_index=False)
val_ds   = Dataset.from_pandas(val_df,    preserve_index=False)
test_ds  = Dataset.from_pandas(dftest_,   preserve_index=False)

# Keep just text + labels before tokenization
cols_keep = ["text", "labels"]
train_ds = keep_only(train_ds, cols_keep)
val_ds   = keep_only(val_ds,   cols_keep)
test_ds  = keep_only(test_ds,  cols_keep)

# Tokenize
train_ds = train_ds.map(tok_fn, batched=True)
val_ds   = val_ds.map(tok_fn,   batched=True)
test_ds  = test_ds.map(tok_fn,  batched=True)

# Remove raw text after tokenization, keep labels + token IDs
train_ds = train_ds.remove_columns(["text"])
val_ds   = val_ds.remove_columns(["text"])
test_ds  = test_ds.remove_columns(["text"])

# Ensure final columns are exactly what Trainer expects
final_cols = ["input_ids", "attention_mask", "labels"]
train_ds = keep_only(train_ds, final_cols)
val_ds   = keep_only(val_ds,   final_cols)
test_ds  = keep_only(test_ds,  final_cols)

# # --- Speed tip: length stats + length column for bucketing ---
# def _len_map(batch):
#     return {"input_length": [len(x) for x in batch["input_ids"]]}
#
# train_ds = train_ds.map(_len_map, batched=True)
# val_ds   = val_ds.map(_len_map,   batched=True)
# test_ds  = test_ds.map(_len_map,  batched=True)

# Optional: quick distribution print so you can decide a smaller MAX_LEN later
import numpy as np
# lens = np.array(train_ds["input_length"])
# print(f"[len] p50={np.percentile(lens,50)} p90={np.percentile(lens,90)} "
#       f"p95={np.percentile(lens,95)} max={lens.max()}")

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=8)

from transformers import AutoModelForSequenceClassification

UNFREEZE_LAST_K = 10  # 1..12 are sensible for DeBERTa-v3-base

def build_model(unfreeze_last_k=UNFREEZE_LAST_K):
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=len(ORDER),
        id2label=ID2LABEL,
        label2id=LABEL2ID,
        torch_dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else None),
        use_safetensors=True,
    )
    base = getattr(model, "deberta", None) or getattr(model, "roberta", None) or getattr(model, "bert", None)
    if base is not None and hasattr(base, "encoder") and hasattr(base.encoder, "layer"):
        # freeze all
        for p in base.parameters(): p.requires_grad = False
        # unfreeze last k transformer blocks
        for layer in base.encoder.layer[-int(unfreeze_last_k):]:
            for p in layer.parameters(): p.requires_grad = True
    # classifier head always trainable
    for p in model.classifier.parameters(): p.requires_grad = True
    return model.to(DEVICE)

model = build_model()

# # after model = build_model()
# if torch.cuda.is_available():
#     model.gradient_checkpointing_enable()          # compute–memory tradeoff → larger batch


trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable:,} / {total:,} ({100*trainable/total:.2f}%) ; unfreeze_last_k={UNFREEZE_LAST_K}")
import evaluate
metric_acc = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    acc = metric_acc.compute(predictions=preds, references=labels)["accuracy"]
    p, r, f1, _ = precision_recall_fscore_support(labels, preds, average="macro", zero_division=0)
    return {"accuracy": acc, "precision": p, "recall": r, "f1": f1}

class PrintAndWBCallback(TrainerCallback):
    def __init__(self, print_every=100):
        self.print_every = print_every
        self.steps_per_epoch = None
        self.last_print_step = -1

    def on_train_begin(self, args, state: TrainerState, control: TrainerControl, **kwargs):
        # steps/epoch computed via train_dataloader length
        trainer = kwargs.get("model")  # not available here; compute later in on_step_begin
        print(f"[Run] epochs={args.num_train_epochs} bs={args.per_device_train_batch_size} "
              f"lr={args.learning_rate:.2e} wd={args.weight_decay:.1e} "
              f"warmup_ratio={args.warmup_ratio} grad_accum={args.gradient_accumulation_steps}")

    def on_step_begin(self, args, state: TrainerState, control: TrainerControl, **kwargs):
        # set steps_per_epoch once we have dataloader info exposed in state
        if self.steps_per_epoch is None and state.max_steps is not None and state.num_train_epochs:
            approx_steps_total = state.max_steps
            self.steps_per_epoch = max(1, int(approx_steps_total / math.ceil(state.num_train_epochs)))

    def on_log(self, args, state: TrainerState, control: TrainerControl, logs=None, **kwargs):
        if logs is None: return
        # mimic: [e1 b123/...] loss=... it/s=...
        if self.steps_per_epoch:
            step_in_epoch = (state.global_step % self.steps_per_epoch) or self.steps_per_epoch
            if step_in_epoch % self.print_every == 0 and state.global_step != self.last_print_step:
                loss = logs.get("loss", logs.get("train_loss", None))
                lr = logs.get("learning_rate", None)
                sps = logs.get("train_samples_per_second", None) or logs.get("samples_per_second", None)
                txt = f"[e{int(state.epoch or 0)} b{step_in_epoch}/{self.steps_per_epoch}]"
                if loss is not None: txt += f" loss={loss:.4f}"
                if lr   is not None: txt += f" lr={lr:.2e}"
                if sps  is not None: txt += f" samp/s={sps:.1f}"
                print(txt, flush=True)
                self.last_print_step = state.global_step

    def on_evaluate(self, args, state: TrainerState, control: TrainerControl, metrics, **kwargs):
        if metrics:
            msg = (f"[val @ epoch {int(state.epoch or 0)}] "
                   f"acc={metrics.get('eval_accuracy',0):.4f} "
                   f"f1={metrics.get('eval_f1',0):.4f} "
                   f"p={metrics.get('eval_precision',0):.4f} "
                   f"r={metrics.get('eval_recall',0):.4f}")
            print(msg, flush=True)





Train/Val/Test sizes: 37039/4116/3798


C:\Users\adishalit1\AppData\Local\anaconda3\envs\dl4090\Lib\site-packages\transformers\convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Map: 100%|██████████| 3798/3798 [00:00<00:00, 24327.51 examples/s]
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/mdeberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\adishalit1\AppData\Local\anaconda3\envs\d

Trainable params: 71,473,157 / 278,813,189 (25.63%) ; unfreeze_last_k=10


In [3]:
UNFREEZE_LAST_K = 10

In [4]:
    # --- Ex.5: Optuna study (12 trials × 12 epochs) around your current best HPs ---

import optuna
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

STUDY_GROUP   = BASE_RUN_NAME + "_study"
REFINE_EPOCHS = 8        # <- per your request
N_TRIALS      = 10        # <- per your request
LOG_STEPS     = 100

# Convenience: candidate batch sizes "around" your current BATCH_SIZE, but safe on 4090
def bs_candidates(bs):
    cands = {bs}
    if bs <= 16:
        cands.add(min(32, bs * 2))
    else:
        cands.add(max(8, bs // 2))
    # keep them reasonable
    return sorted({b for b in cands if 4 <= b <= 64})

# Trainer factory for a single trial
def build_trainer_for_trial(lr, wd, k_unfreeze, bs, run_name, epochs):
    model = build_model(unfreeze_last_k=int(k_unfreeze))
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total = sum(p.numel() for p in model.parameters())
    print(f"Trainable params: {trainable:,} / {total:,} ({100*trainable/total:.2f}%) ; unfreeze_last_k={k_unfreeze}")

    # bf16_ok = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
    safe_name = BASE_RUN_NAME
    out_dir = os.path.join("hf_ckpts", safe_name)
    os.makedirs(out_dir, exist_ok=True)

    bf16_ok = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
    args = TrainingArguments(
        output_dir=os.path.join("hf_ckpts", run_name),
        run_name=run_name,
        report_to=["wandb"],
        seed=SEED,
        learning_rate=float(lr),
        weight_decay=float(wd),
        warmup_ratio=WARMUP_RATIO,
        num_train_epochs=int(epochs),
        per_device_train_batch_size=int(bs),
        per_device_eval_batch_size=int(bs),
        gradient_accumulation_steps=GRAD_ACCUM,
        dataloader_num_workers=NUM_WORKERS,
        dataloader_pin_memory=True,
        dataloader_prefetch_factor=2 if NUM_WORKERS > 0 else None,
        fp16=(torch.cuda.is_available() and not bf16_ok),
        bf16=bf16_ok,
        optim="adamw_torch_fused" if torch.cuda.is_available() else "adamw_torch",
        save_safetensors=True,
        disable_tqdm=True,             # we print ourselves
        logging_strategy="steps",
        logging_steps=LOG_STEPS,
        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        callbacks=[
            PrintAndWBCallback(print_every=LOG_STEPS),
            EarlyStoppingCallback(early_stopping_patience=PATIENCE),
        ],
    )
    return trainer

# Objective: search narrowly around your current best HPs
def objective(trial: optuna.trial.Trial):
    lr = trial.suggest_float("lr", 1e-5, 1e-4, log=True)
    wd = trial.suggest_float("weight_decay",1e-6, 1e-4, log=True)
    k  = trial.suggest_int("unfreeze_last_k", 8,12)
    bs = trial.suggest_categorical("batch_size", [4, 8, 16, 32])

    run_name = f"{BASE_RUN_NAME}__t{trial.number}"

    # W&B run per trial (grouped under the study for easy comparison)
    wb = wandb.init(
        project=WANDB_PROJECT,
        name=run_name,
        group=STUDY_GROUP,
        job_type="optuna-trial",
        tags=["optuna","trainer","ex5"],
        config={
            "trial": trial.number, "model": MODEL_NAME,
            "lr": lr, "weight_decay": wd, "unfreeze_last_k": k,
            "batch_size": bs, "epochs": REFINE_EPOCHS,
            "warmup_ratio": WARMUP_RATIO, "max_len": MAX_LEN,
        },
        settings=wandb.Settings(start_method="thread"),
    )

    print(f"[TUNE] trial={trial.number} | epochs={REFINE_EPOCHS} bs={bs} lr={lr:.2e} wd={wd:.1e} k={k}")

    trainer = build_trainer_for_trial(lr, wd, k, bs, run_name, REFINE_EPOCHS)

    # Train + epoch evals (printed by callback; logged to W&B automatically)
    trainer.train()

    # Final eval for the objective score
    metrics = trainer.evaluate()
    f1 = float(metrics.get("eval_f1", 0.0))
    acc = float(metrics.get("eval_accuracy", 0.0))

    # Extra trial-level logging
    wandb.log({"trial/f1": f1, "trial/accuracy": acc})
    wb.summary["best_model_ckpt"] = trainer.state.best_model_checkpoint
    wb.summary["best_eval_f1"] = f1
    wb.summary["params"] = dict(lr=lr, weight_decay=wd, unfreeze_last_k=k, batch_size=bs, epochs=REFINE_EPOCHS)
    wandb.finish()

    print(f"[TUNE-END] trial={trial.number} f1={f1:.4f}")
    return f1

# Run the study
print(f"[Study] Starting Optuna: trials={N_TRIALS}, epochs/trial={REFINE_EPOCHS} ; centered at "
      f"lr={LR:.2e}, wd={WEIGHT_DECAY:.1e}, k={UNFREEZE_LAST_K}, bs={BATCH_SIZE}")
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=True)

# Persist best params
best = {
    "lr": float(study.best_trial.params["lr"]),
    "weight_decay": float(study.best_trial.params["weight_decay"]),
    "num_unfreeze_last_layers": int(study.best_trial.params["unfreeze_last_k"]),
    "batch_size": int(study.best_trial.params["batch_size"]),
    "epochs": int(EPOCHS),   # you can bump this later for final train if you want
}
os.makedirs("hf_ckpts", exist_ok=True)
with open("hf_ckpts/best_params_optuna1.json", "w") as f:
    json.dump(best, f, indent=2)

print("[Study best]", best)
print("Saved best params → hf_ckpts/best_params_optuna1.json")


[I 2025-08-16 21:41:10,602] A new study created in memory with name: no-name-3b13a031-3415-41ed-8e90-8e6b61be55de


[Study] Starting Optuna: trials=10, epochs/trial=8 ; centered at lr=3.00e-05, wd=5.0e-02, k=10, bs=16


  0%|          | 0/10 [00:00<?, ?it/s]wandb: WARNING `start_method` is deprecated and will be removed in a future version of wandb. This setting is currently non-functional and safely ignored.
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find ex5_trainer_new.ipynb.
wandb: Currently logged in as: adishalit1 (adishalit1-tel-aviv-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[TUNE] trial=0 | epochs=8 bs=8 lr=1.20e-05 wd=6.1e-06 k=12


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/mdeberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Trainable params: 85,648,901 / 278,813,189 (30.72%) ; unfreeze_last_k=12
[Run] epochs=8 bs=4 lr=4.21e-05 wd=1.9e-06 warmup_ratio=0.06 grad_accum=1
[e0 b100/9260] loss=1.6410 lr=9.37e-07
{'loss': 1.641, 'grad_norm': 6.0625, 'learning_rate': 9.368652170568546e-07, 'epoch': 0.01079913606911447}
[e0 b200/9260] loss=1.6500 lr=1.88e-06
{'loss': 1.65, 'grad_norm': 2.859375, 'learning_rate': 1.8831937191344855e-06, 'epoch': 0.02159827213822894}
[e0 b300/9260] loss=1.6472 lr=2.83e-06
{'loss': 1.6472, 'grad_norm': 7.15625, 'learning_rate': 2.8295222212121163e-06, 'epoch': 0.032397408207343416}
[e0 b400/9260] loss=1.6394 lr=3.78e-06
{'loss': 1.6394, 'grad_norm': 4.84375, 'learning_rate': 3.7758507232897473e-06, 'epoch': 0.04319654427645788}
[e0 b500/9260] loss=1.6493 lr=4.72e-06
{'loss': 1.6493, 'grad_norm': 6.15625, 'learning_rate': 4.722179225367378e-06, 'epoch': 0.05399568034557235}
[e0 b600/9260] loss=1.6426 lr=5.67e-06
{'loss': 1.6426, 'grad_norm': 4.8125, 'learning_rate': 5.668507727445009e

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


eval/accuracy,▁▅▇▇█████
eval/f1,▁▅▇▇█████
eval/loss,█▄▂▃▁▂▂▂▁
eval/precision,▁▆▇▇█████
eval/recall,▁▅▇██████
eval/runtime,▃▃▆▃▆█▄▁▆
eval/samples_per_second,▆▆▂▅▃▁▅█▃
eval/steps_per_second,▆▆▂▅▃▁▅█▃
train/epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇██
train/grad_norm,▁▁▂▂▄▄▅▄▃▆▃▃▃▂▄▅▄▆▆█▇█▄▄▄▅▂▅▄▂▃▂▃▂█▅▃▃▂▇


Best trial: 2. Best value: 0.852341: 100%|██████████| 10/10 [4:18:17<00:00, 1549.73s/it]

[TUNE-END] trial=9 f1=0.7334
[I 2025-08-17 01:59:27,931] Trial 9 finished with value: 0.7333747370018296 and parameters: {'lr': 4.2064301917350694e-05, 'weight_decay': 1.935643819405686e-06, 'unfreeze_last_k': 12, 'batch_size': 4}. Best is trial 2 with value: 0.8523413656247504.
[Study best] {'lr': 9.034601256162855e-05, 'weight_decay': 2.3378864159216274e-06, 'num_unfreeze_last_layers': 12, 'batch_size': 32, 'epochs': 12}
Saved best params → hf_ckpts/best_params_optuna1.json


# 📊 Ex.5 — Optuna Trials: First Results  

We ran **10 trials** with Optuna on top of the HuggingFace Trainer. The goal was to see if the “sweet spot” we found earlier (low LR, deep unfreezing) would still hold.  

---

### 🔑 Main Observations
- **Learning rate**:  
  The good runs are still **between `1e-5` and `9e-5`**.  
  Too small (≈`1e-5`) → model struggles, F1 < 0.6.  
  Too large (≥`1e-4`) → model basically collapses (trial 10 got F1 ≈ 0.08 😬).  

- **Unfreezing**:  
  Best trials need **10–12 layers unfrozen**.  
  When we unfreeze fewer layers (≤9), performance drops sharply.  

- **Overall performance**:  
  Only **two trials (2 and 5)** crossed **0.85 F1**, while most others stayed in the 0.59–0.74 range.  
  So we’re **on the right track** but not as strong/consistent as before.  

---

### 🏆 Best Trials
- **Trial 2** → LR ≈ 9e-5, WD ≈ 2e-6, batch size = 32, unfreeze = 12 → **F1 = 0.852**  
- **Trial 5** → LR ≈ 9.5e-5, WD ≈ 2e-5, batch size = 16, unfreeze = 11 → **F1 = 0.852**  

Both show that **higher LR (but still <1e-4)** + **deep unfreezing** works best.  

---

### 📌 Takeaways
- The **sweet spot (LR, unfreezing)** is consistent with earlier experiments.  
- But the results are **less stable** this time → some configs just collapse.  
- We should:  
  - Run **more trials (20–30)** to confirm the pattern.  
  - Maybe tune **warmup ratio/scheduler** for stability.  
  
  


# Another try with higher lr

In [4]:
    # --- Ex.5: Optuna study (12 trials × 12 epochs) around your current best HPs ---

import optuna
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

STUDY_GROUP   = BASE_RUN_NAME + "_study"
REFINE_EPOCHS = 8        # <- per your request
N_TRIALS      = 15        # <- per your request
LOG_STEPS     = 100



# Objective: search narrowly around your current best HPs
def objective(trial: optuna.trial.Trial):
    lr = trial.suggest_float("lr", 7e-5, 5e-4, log=True)
    wd = trial.suggest_float("weight_decay",1e-5, 1e-4, log=True)
    k  = trial.suggest_int("unfreeze_last_k", 8,12)
    bs = trial.suggest_categorical("batch_size", [8, 16, 32,64,128])

    run_name = f"{BASE_RUN_NAME}__t{trial.number+10}"

    # W&B run per trial (grouped under the study for easy comparison)
    wb = wandb.init(
        project=WANDB_PROJECT,
        name=run_name,
        group=STUDY_GROUP,
        job_type="optuna-trial",
        tags=["optuna","trainer","ex5"],
        config={
            "trial": trial.number+10, "model": MODEL_NAME,
            "lr": lr, "weight_decay": wd, "unfreeze_last_k": k,
            "batch_size": bs, "epochs": REFINE_EPOCHS,
            "warmup_ratio": WARMUP_RATIO, "max_len": MAX_LEN,
        },
        settings=wandb.Settings(start_method="thread"),
    )

    print(f"[TUNE] trial={trial.number+10} | epochs={REFINE_EPOCHS} bs={bs} lr={lr:.2e} wd={wd:.1e} k={k}")

    trainer = build_trainer_for_trial(lr, wd, k, bs, run_name, REFINE_EPOCHS)

    # Train + epoch evals (printed by callback; logged to W&B automatically)
    trainer.train()

    # Final eval for the objective score
    metrics = trainer.evaluate()
    f1 = float(metrics.get("eval_f1", 0.0))
    acc = float(metrics.get("eval_accuracy", 0.0))

    # Extra trial-level logging
    wandb.log({"trial/f1": f1, "trial/accuracy": acc})
    wb.summary["best_model_ckpt"] = trainer.state.best_model_checkpoint
    wb.summary["best_eval_f1"] = f1
    wb.summary["params"] = dict(lr=lr, weight_decay=wd, unfreeze_last_k=k, batch_size=bs, epochs=REFINE_EPOCHS)
    wandb.finish()

    print(f"[TUNE-END] trial={trial.number+10} f1={f1:.4f}")
    return f1

# Run the study
print(f"[Study] Starting Optuna: trials={N_TRIALS}, epochs/trial={REFINE_EPOCHS} ; centered at "
      f"lr={LR:.2e}, wd={WEIGHT_DECAY:.1e}, k={UNFREEZE_LAST_K}, bs={BATCH_SIZE}")
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=True)

# Persist best params
best = {
    "lr": float(study.best_trial.params["lr"]),
    "weight_decay": float(study.best_trial.params["weight_decay"]),
    "num_unfreeze_last_layers": int(study.best_trial.params["unfreeze_last_k"]),
    "batch_size": int(study.best_trial.params["batch_size"]),
    "epochs": int(EPOCHS),   # you can bump this later for final train if you want
}
os.makedirs("hf_ckpts", exist_ok=True)
with open("hf_ckpts/best_params_optuna_1closer.json", "w") as f:
    json.dump(best, f, indent=2)

print("[Study best]", best)
print("Saved best params → hf_ckpts/best_params_optuna1closer.json")


[I 2025-08-17 04:34:57,147] A new study created in memory with name: no-name-4f0d5460-fa46-45df-86f5-9b8bd36a09cf


[Study] Starting Optuna: trials=15, epochs/trial=8 ; centered at lr=3.00e-05, wd=5.0e-02, k=10, bs=16


  0%|          | 0/15 [00:00<?, ?it/s]wandb: WARNING `start_method` is deprecated and will be removed in a future version of wandb. This setting is currently non-functional and safely ignored.
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find ex5_trainer_new.ipynb.
wandb: Currently logged in as: adishalit1 (adishalit1-tel-aviv-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[TUNE] trial=10 | epochs=8 bs=16 lr=3.88e-04 wd=9.1e-06 k=11


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/mdeberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/mdeberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\adishalit1\AppData\Local\Temp\ipykernel_29804\3473936884.py:106: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Trainable params: 78,561,029 / 278,813,189 (28.18%) ; unfreeze_last_k=11
[Run] epochs=8 bs=128 lr=1.04e-04 wd=4.6e-05 warmup_ratio=0.06 grad_accum=1
[e0 b100/290] loss=1.5677 lr=7.38e-05
{'loss': 1.5677, 'grad_norm': 3.515625, 'learning_rate': 7.378932090650918e-05, 'epoch': 0.3448275862068966}
[e0 b200/290] loss=1.1213 lr=1.02e-04
{'loss': 1.1213, 'grad_norm': 11.5, 'learning_rate': 0.00010152442289861383, 'epoch': 0.6896551724137931}


Best trial: 2. Best value: 0.858824:  53%|█████▎    | 8/15 [1:49:57<1:36:13, 824.74s/it]


[W 2025-08-17 06:24:55,040] Trial 8 failed with parameters: {'lr': 0.00010434853461526551, 'weight_decay': 4.6344760009348534e-05, 'unfreeze_last_k': 11, 'batch_size': 128} because of the following error: RuntimeError('CUDA error: out of memory\nCUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.\nFor debugging consider passing CUDA_LAUNCH_BLOCKING=1\nCompile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.\n').
Traceback (most recent call last):
  File "C:\Users\adishalit1\AppData\Local\anaconda3\envs\dl4090\Lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\adishalit1\AppData\Local\Temp\ipykernel_29804\3473936884.py", line 151, in objective
    trainer.train()
  File "C:\Users\adishalit1\AppData\Local\anaconda3\envs\dl4090\Lib\site-packages\transformers\trainer.py", line 2238, in train
    return inner_t

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


# 📊 Ex.5 — Optuna Trials (11–18): Extended Results  

After the first 10 runs, we extended the Optuna search to more trials.  
Here’s what we observed:  

---

### ✅ Stronger Trials
- **Trial 12** → LR ≈ 1.0e-4, WD ≈ 5e-5, batch size 16, unfreeze 12 → **F1 = 0.859**  
  → This is our **best so far**. Confirms that pushing LR slightly above 9e-5 can work if weight decay is kept moderate.  
- **Trial 13** → LR ≈ 9.4e-5, WD ≈ 6e-5, batch size 8, unfreeze 11 → **F1 = 0.847**  
  → Stable, good consistency. Smaller batch helped here.  
- **Trial 11** → LR ≈ 1.2e-4, WD ≈ 8.6e-5, batch size 16, unfreeze 8 → **F1 = 0.836**  
  → Still decent, but shallower unfreezing (only 8 layers) seems to cap performance.  
- **Trial 17** → LR ≈ 8.6e-5, WD ≈ 1e-5, batch size 32, unfreeze 11 → **F1 = 0.832**  
  → Confirms the LR “sweet spot.” Batch size 32 still works fine.  

---

### ⚠️ Medium Results
- **Trial 14** → LR ≈ 9.1e-5, WD ≈ 1.3e-5, batch size 8, unfreeze 8 → **F1 = 0.814**  
  → Again, too shallow unfreezing limits the score.  

---

### ❌ Collapse Zone
Several trials **completely collapsed (F1 ≈ 0.086)**:  
- Trial 10, 15, 16, 11 (second run), 18 (no result)  
  - All used **very large LR (≥4e-4)**.  
  - This confirms that once LR crosses ~1.5e-4, the model becomes unstable and fails to learn.  

---

### 🧾 Takeaways
- **Best range** remains **LR ≈ 8e-5 → 1e-4**, with **weight decay ~1e-5 → 6e-5**.  
- **Deep unfreezing (10–12 layers)** is almost always required for high F1.  
- **Batch size flexibility**: both 16 and 32 work, 8 sometimes helps but not always.  
- **Large LR (≥2e-4)** → catastrophic collapse (F1 ~ 0.08).  

---

👉 Overall, Trials 12 and 13 show we can **push beyond 0.85 F1** with careful tuning.  
But the search space is still fragile — a small step in LR can cause collapse.  
Next step: maybe test **schedulers / warmup ratio tweaks** to stabilize training.  


# 🏁 Ex.5 — Final Training & Test Evaluation  

Now that we finished **hyperparameter tuning with Optuna**, it’s time for the **final run**:  
we take the **best HPs** (learning rate, weight decay, batch size, unfreezing depth, epochs)  
and retrain a **fresh model from scratch** with them.  

---

### 🔨 What happens in this stage:
1. **Load best hyperparameters** — either directly from the Optuna study in memory,  
   or from the saved JSON (`hf_ckpts/best_params_optuna1.json`).  
2. **Rebuild the model** — freeze everything, then unfreeze the best `k` last layers,  
   keeping the classifier head trainable.  
3. **Training setup** — longer training (15 epochs), early stopping, bf16/fp16 acceleration,  
   and fused AdamW optimizer for speed.  
4. **Train & save checkpoints** — log everything to W&B, keep the best model at the end.  
5. **Evaluate** — report metrics on both validation and the clean translated **test set**.  
   We include accuracy, precision, recall, macro-F1, plus full per-class report and confusion matrix.  

---

👉 This gives us the **final performance numbers** of our pipeline,  
and also saves the best model checkpoint so we can reuse it later.


# Load Best Model EX5

In [8]:
# ===== Final train on best HPs, save model, and evaluate on test =====
import os, json, time, importlib.util
import numpy as np
import pandas as pd
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix
import wandb
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback, AutoModelForSequenceClassification


# --- 1) Load best hyperparameters (from current study or from disk) ---
best_params_path = "hf_ckpts/best_params_optuna1.json"
if 'study' in globals() and study.best_trial is not None:
    bp = {
        "lr": float(study.best_trial.params["lr"]),
        "weight_decay": float(study.best_trial.params["weight_decay"]),
        "num_unfreeze_last_layers": int(study.best_trial.params["unfreeze_last_k"]),
        "batch_size": int(study.best_trial.params["batch_size"]),
        "epochs": int(15),  # keep your requested final epochs
    }
    print("[final] Using best params from in-memory study:", bp)
    os.makedirs(os.path.dirname(best_params_path), exist_ok=True)
    with open(best_params_path, "w") as f: json.dump(bp, f, indent=2)
else:
    with open(best_params_path, "r") as f:
        bp = json.load(f)
    print("[final] Loaded best params from file:", bp)

FINAL_EPOCHS   = int(bp.get("epochs", EPOCHS))
FINAL_LR       = float(bp["lr"])
FINAL_WD       = float(bp["weight_decay"])
FINAL_BS       = int(bp["batch_size"])
FINAL_UNFREEZE = int(bp["num_unfreeze_last_layers"])

# --- 2) Rebuild a fresh model with the chosen unfreezing plan ---
def build_model_for_final(unfreeze_last_k: int):
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=len(ORDER),
        id2label=ID2LABEL,
        label2id=LABEL2ID,
        torch_dtype=(torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else None),
        use_safetensors=True,
    )
    base = getattr(model, "deberta", None) or getattr(model, "roberta", None) or getattr(model, "bert", None)
    if base is not None and hasattr(base, "encoder") and hasattr(base.encoder, "layer"):
        # freeze all, then unfreeze last k transformer blocks
        for p in base.parameters(): p.requires_grad = False
        for layer in base.encoder.layer[-int(unfreeze_last_k):]:
            for p in layer.parameters(): p.requires_grad = True
    for p in model.classifier.parameters(): p.requires_grad = True

    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total = sum(p.numel() for p in model.parameters())
    print(f"[final] Trainable params: {trainable:,} / {total:,} ({100*trainable/total:.2f}%) ; unfreeze_last_k={unfreeze_last_k}")
    return model.to(DEVICE)

model = build_model_for_final(FINAL_UNFREEZE)

# --- 3) Final TrainingArguments (bf16 on 4090; fused AdamW; no torch.compile) ---
bf16_ok = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
timestamp = time.strftime("%Y%m%d_%H%M%S")
final_run_name = f"{BASE_RUN_NAME}__final_{timestamp}"
final_out_dir  = os.path.join("hf_ckpts", final_run_name)
os.makedirs(final_out_dir, exist_ok=True)

training_args = TrainingArguments(
    output_dir=final_out_dir,
    run_name=final_run_name,
    report_to=["wandb"],

    learning_rate=FINAL_LR,
    weight_decay=FINAL_WD,
    warmup_ratio=WARMUP_RATIO,
    num_train_epochs=FINAL_EPOCHS,

    per_device_train_batch_size=FINAL_BS,
    per_device_eval_batch_size=FINAL_BS,
    gradient_accumulation_steps=GRAD_ACCUM,

    dataloader_num_workers=max(0, NUM_WORKERS),   # keep your Windows-stable choice
    dataloader_pin_memory=True,
    dataloader_prefetch_factor=2 if NUM_WORKERS > 0 else None,

    fp16=(torch.cuda.is_available() and not bf16_ok),
    bf16=bf16_ok,
    optim=("adamw_torch_fused" if torch.cuda.is_available() else "adamw_torch"),
    save_safetensors=True,

    logging_strategy="steps",
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,

    torch_compile=False,   # keep off on Windows unless Triton is present & stable
    seed=SEED,
)

# --- 4) W&B run for the final train ---
wandb_run = wandb.init(
    project=WANDB_PROJECT,
    name=final_run_name,
    group=BASE_RUN_NAME + "_final",
    job_type="final-train",
    tags=["final","trainer","ex5"],
    config={
        "model": MODEL_NAME,
        "max_len": MAX_LEN,
        "epochs": FINAL_EPOCHS,
        "lr": FINAL_LR,
        "weight_decay": FINAL_WD,
        "warmup_ratio": WARMUP_RATIO,
        "batch_size": FINAL_BS,
        "grad_accum": GRAD_ACCUM,
        "unfreeze_last_k": FINAL_UNFREEZE,
        "num_workers": NUM_WORKERS,
        "bf16": bf16_ok,
    },
)

print(f"[final] device={DEVICE} bf16={bf16_ok} fp16={training_args.fp16} | out_dir={final_out_dir}")
print(f"[final] HP → bs={FINAL_BS} lr={FINAL_LR:.2e} wd={FINAL_WD:.1e} epochs={FINAL_EPOCHS} unfreeze_last_k={FINAL_UNFREEZE}")

# --- 5) Trainer + train ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,                 # ok; deprecates in v5 (processing_class later)
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[PrintAndWBCallback(print_every=100),
               EarlyStoppingCallback(early_stopping_patience=PATIENCE)],
)

print("[final] Starting training …")
train_out = trainer.train()
print("[final] TrainOutput:", train_out)
print("[final] Best checkpoint:", trainer.state.best_model_checkpoint)

# --- 6) Evaluate on validation & test, save detailed reports ---
print("[final] Evaluating on validation set …")
val_metrics = trainer.evaluate(eval_dataset=val_ds)
print("[final][val] →", val_metrics)
wandb.log({f"final/val_{k}": v for k, v in val_metrics.items()})

print("[final] Evaluating on TEST set …")
test_pred = trainer.predict(test_ds)
test_logits = test_pred.predictions
test_labels = test_pred.label_ids
test_preds  = np.argmax(test_logits, axis=-1)

test_acc = accuracy_score(test_labels, test_preds)
p, r, f1, _ = precision_recall_fscore_support(test_labels, test_preds, average="macro", zero_division=0)
test_metrics = {"test_accuracy": float(test_acc), "test_precision_macro": float(p), "test_recall_macro": float(r), "test_f1_macro": float(f1)}
print("[final][test] →", test_metrics)
wandb.log({f"final/{k}": v for k, v in test_metrics.items()})

# per-class report & confusion matrix
report_dict = classification_report(test_labels, test_preds, target_names=ORDER, digits=4, output_dict=True)
cm = confusion_matrix(test_labels, test_preds, labels=list(range(len(ORDER))))



[final] Loaded best params from file: {'lr': 0.00010491108358128862, 'weight_decay': 5.114700076417737e-05, 'num_unfreeze_last_layers': 12, 'batch_size': 16, 'epochs': 12}


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/mdeberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[final] Trainable params: 85,648,901 / 278,813,189 (30.72%) ; unfreeze_last_k=12


train/epoch,▁▁▂▂▃▃▃▄▄▅▅▆▆▆▇▇██
train/global_step,▁▁▂▂▃▃▃▄▄▅▅▆▆▆▇▇██
train/grad_norm,▁▁▁▂▃▂▃▆▄▇▄▆▆▄▄█▄▅
train/learning_rate,▁▁▂▂▃▃▄▄▅▅▅▆▆▇▇███
train/loss,████▇▇▆▅▄▄▃▂▂▂▁▂▁▁
train/epoch,0.77754
train/global_step,1800
train/grad_norm,13.4375
train/learning_rate,0.0001
train/loss,0.8664


[final] device=cuda bf16=True fp16=False | out_dir=hf_ckpts\microsoft__mdeberta-v3-base_ex5_trainer-try__final_20250817_104013
[final] HP → bs=16 lr=1.05e-04 wd=5.1e-05 epochs=12 unfreeze_last_k=12
[final] Starting training …


C:\Users\adishalit1\AppData\Local\Temp\ipykernel_30920\2488745322.py:139: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


[Run] epochs=12 bs=16 lr=1.05e-04 wd=5.1e-05 warmup_ratio=0.06 grad_accum=1


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.743100,0.713724,0.729106,0.761852,0.721231,0.736664
2,0.468500,0.510003,0.827017,0.843368,0.824059,0.831787
3,0.442800,0.587996,0.811710,0.811926,0.834801,0.814972
4,0.340900,0.469016,0.850826,0.852043,0.858283,0.854500
5,0.281400,0.487433,0.853984,0.856803,0.861751,0.857645
6,0.275800,0.460352,0.861516,0.860913,0.870439,0.864878
7,0.248500,0.500496,0.863946,0.862110,0.875398,0.867273
8,0.243500,0.506808,0.860787,0.857589,0.872901,0.863752
9,0.249600,0.497311,0.863946,0.862095,0.874121,0.867178
10,0.199400,0.514455,0.859572,0.855997,0.872231,0.862454


[e0 b100/2315] loss=1.6420 lr=6.23e-06
[e0 b200/2315] loss=1.6353 lr=1.25e-05
[e0 b300/2315] loss=1.6074 lr=1.88e-05
[e0 b400/2315] loss=1.5622 lr=2.51e-05
[e0 b500/2315] loss=1.5431 lr=3.14e-05
[e0 b600/2315] loss=1.4967 lr=3.77e-05
[e0 b700/2315] loss=1.4243 lr=4.40e-05
[e0 b800/2315] loss=1.3462 lr=5.03e-05
[e0 b900/2315] loss=1.2049 lr=5.66e-05
[e0 b1000/2315] loss=1.1781 lr=6.29e-05
[e0 b1100/2315] loss=1.0698 lr=6.92e-05
[e0 b1200/2315] loss=1.0031 lr=7.55e-05
[e0 b1300/2315] loss=0.9656 lr=8.18e-05
[e0 b1400/2315] loss=0.9317 lr=8.80e-05
[e0 b1500/2315] loss=0.9780 lr=9.43e-05
[e0 b1600/2315] loss=0.9174 lr=1.01e-04
[e0 b1700/2315] loss=0.8946 lr=1.05e-04
[e0 b1800/2315] loss=0.8692 lr=1.04e-04
[e0 b1900/2315] loss=0.8118 lr=1.04e-04
[e0 b2000/2315] loss=0.8220 lr=1.04e-04
[e0 b2100/2315] loss=0.7980 lr=1.03e-04
[e0 b2200/2315] loss=0.7537 lr=1.03e-04
[e0 b2300/2315] loss=0.7431 lr=1.02e-04
[val @ epoch 1] acc=0.7291 f1=0.7367 p=0.7619 r=0.7212
[val @ epoch 2] acc=0.8270 f1=0.83

[val @ epoch 11] acc=0.8639 f1=0.8673 p=0.8621 r=0.8754
[final][val] → {'eval_loss': 0.5004957914352417, 'eval_accuracy': 0.8639455782312925, 'eval_precision': 0.8621102868494844, 'eval_recall': 0.8753979815762152, 'eval_f1': 0.8672727518751845, 'eval_runtime': 4.4479, 'eval_samples_per_second': 925.389, 'eval_steps_per_second': 58.005, 'epoch': 11.0}
[final] Evaluating on TEST set …
[final][test] → {'test_accuracy': 0.8433385992627699, 'test_precision_macro': 0.8464462108232367, 'test_recall_macro': 0.8524153135872158, 'test_f1_macro': 0.8476267133422806}


# 📊 Ex.5 — Final Results (Validation & Test)

After retraining the model with the **best hyperparameters** from Optuna,  
we get the following performance:

---

## ✅ Validation set
- **Accuracy:** 0.8640  
- **Macro F1:** 0.8673  
- **Macro Precision:** 0.8621  
- **Macro Recall:** 0.8754  

> The model is very consistent on validation, reaching ~86–87% across all main metrics.  

---

## 🧪 Test set (clean translated)
- **Accuracy:** 0.8433  
- **Macro F1:** 0.8476  
- **Macro Precision:** 0.8464  
- **Macro Recall:** 0.8524  

> On the unseen test set, performance drops slightly (≈2% lower than validation),  
> but results are still strong and balanced across classes.  

---

### 🎓 Takeaway
- Compared to our **Ex.4 training**, where the test **Macro F1 was 0.8685**,  
  this run is a bit **weaker (~0.85 F1)**.  
- Still, the model shows **good generalization and stable performance**.  
- The Validation ↔ Test gap remains small → no major overfitting.  
- Overall, with **macro F1 ~0.85**, the model captures all sentiment classes fairly well.
---
After finishing the final training and test evaluation,  
we saved **all important artifacts** for reproducibility and later analysis.  



### 📂 What we saved
- **Model + Tokenizer** (best checkpoint) → can be reloaded for inference or fine-tuning.  
- **Test metrics JSON** → contains summary scores (accuracy, precision, recall, macro F1).  
- **Classification report (CSV)** → per-class metrics (precision, recall, F1, support).  
- **Confusion matrix (CSV)** → class-wise confusion analysis.  


In [10]:
# --- 6) Save evaluation artifacts (Windows-safe: use W&B Artifacts, no symlinks) ---
import time
rep_df = pd.DataFrame(report_dict).transpose()
cm_df  = pd.DataFrame(cm, index=[f"true_{c}" for c in ORDER], columns=[f"pred_{c}" for c in ORDER])

rep_path  = os.path.join(final_out_dir, "classification_report_test.csv")
cm_path   = os.path.join(final_out_dir, "confusion_matrix_test.csv")
json_path = os.path.join(final_out_dir, "test_metrics.json")

rep_df.to_csv(rep_path, index=True)
cm_df.to_csv(cm_path, index=True)
with open(json_path, "w") as f:
    json.dump(test_metrics, f, indent=2)

# ⛔️ Remove wandb.save(...) — it uses symlinks on Windows and fails.
# Use an Artifact instead:
ts = time.strftime("%Y%m%d_%H%M%S")
eval_art = wandb.Artifact(f"{BASE_RUN_NAME}-eval-{ts}", type="evaluation")
eval_art.add_file(rep_path,  name="classification_report_test.csv")
eval_art.add_file(cm_path,   name="confusion_matrix_test.csv")
eval_art.add_file(json_path, name="test_metrics.json")
wandb_run.log_artifact(eval_art)

# --- 7) Save the final model (best weights) + tokenizer ---
save_dir = os.path.join(final_out_dir, "best_model")
os.makedirs(save_dir, exist_ok=True)
trainer.save_model(save_dir)        # saves model + trainer state
tokenizer.save_pretrained(save_dir) # saves tokenizer

# a tiny README for the checkpoint
with open(os.path.join(save_dir, "README.txt"), "w", encoding="utf-8") as f:
    f.write(
        f"Model: {MODEL_NAME}\n"
        f"Labels: {ORDER}\n"
        f"hp: lr={FINAL_LR}, weight_decay={FINAL_WD}, batch_size={FINAL_BS}, "
        f"epochs={FINAL_EPOCHS}, unfreeze_last_k={FINAL_UNFREEZE}\n"
        f"Val metrics: {val_metrics}\n"
        f"Test metrics: {test_metrics}\n"
    )

# (optional but nice) Log the whole model directory as a model artifact too
model_art = wandb.Artifact(f"{BASE_RUN_NAME}-model-{ts}", type="model")
model_art.add_dir(save_dir)
wandb_run.log_artifact(model_art)

# Summaries + finish
wandb_run.summary["best_checkpoint_dir"] = trainer.state.best_model_checkpoint
wandb_run.summary.update({f"final/{k}": v for k, v in test_metrics.items()})
wandb.finish()

print("\n=== DONE ===")
print("Saved:")
print(" • Model + tokenizer →", save_dir)
print(" • Test metrics JSON →", json_path)
print(" • Classification report CSV →", rep_path)
print(" • Confusion matrix CSV →", cm_path)


wandb: Adding directory to artifact (.\hf_ckpts\microsoft__mdeberta-v3-base_ex5_trainer-try__final_20250817_104013\best_model)... Done. 0.9s
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


eval/accuracy,▁▆▅▇▇███████
eval/f1,▁▆▅▇▇███████
eval/loss,█▂▅▁▂▁▂▂▂▂▃▂
eval/precision,▁▇▄▇████████
eval/recall,▁▆▆▇▇███████
eval/runtime,▂▁█▂▅█▅▇▂▃▂▄
eval/samples_per_second,▇█▁▇▄▁▄▂▇▆▇▄
eval/steps_per_second,▇█▁▇▄▁▄▂▇▆▇▄
final/test_accuracy,▁
final/test_f1_macro,▁
final/test_precision_macro,▁



=== DONE ===
Saved:
 • Model + tokenizer → hf_ckpts\microsoft__mdeberta-v3-base_ex5_trainer-try__final_20250817_104013\best_model
 • Test metrics JSON → hf_ckpts\microsoft__mdeberta-v3-base_ex5_trainer-try__final_20250817_104013\test_metrics.json
 • Classification report CSV → hf_ckpts\microsoft__mdeberta-v3-base_ex5_trainer-try__final_20250817_104013\classification_report_test.csv
 • Confusion matrix CSV → hf_ckpts\microsoft__mdeberta-v3-base_ex5_trainer-try__final_20250817_104013\confusion_matrix_test.csv


In [11]:
cm_df

,pred_extremely negative,pred_negative,pred_neutral,pred_positive,pred_extremely positive
true_extremely negative,538,50,2,2,0
true_negative,104,873,19,41,4
true_neutral,5,64,506,42,2
true_positive,4,64,32,733,114
true_extremely positive,1,1,0,44,553


In [12]:
pt_path = os.path.join(final_out_dir, "best_model_ex5.pt")
torch.save(trainer.model.state_dict(), pt_path)
print("Saved state_dict →", pt_path)


Saved state_dict → hf_ckpts\microsoft__mdeberta-v3-base_ex5_trainer-try__final_20250817_104013\best_model_ex5.pt


# ⚖️  Weighted Loss Experiment (mid-class emphasis)

In this extra experiment we re-run the **full training loop (Ex.4 style)**,  
but we make one important change: **the loss function now gives more weight to the “middle” classes**  
(`negative`, `neutral`, `positive`).  

---

### 🎯 Why we do this
- In earlier runs, the model was already strong on the **extreme classes** (`extremely negative`, `extremely positive`).  
- Performance on the **middle classes** was weaker (more confusion between neutral/positive/negative).  
- To balance this, we **boost the loss weight** for the mid-classes so that mistakes on them are penalized more.  
  → This should encourage the model to learn better boundaries between these subtle sentiments.  

---

### 🔧 How we apply it
- We start from the **inverse-frequency class weights** (baseline).  
- Then, we multiply the weights of the mid-classes by a fixed **boost factor (e.g. 1.5×)**.  
- Finally, we normalize so the average weight stays ~1.0, keeping training stable.  
- Loss = **Weighted CrossEntropy**, optionally with small label smoothing.  

---

### 📊 Hyperparameters
- For this **first run** we kept the HP search range the same as before:  
  - `lr` in **8e-5 – 1e-3** (log scale)  
  - `weight_decay` in **1e-6 – 1e-4**  
  - Unfreeze depth: **8–12 layers**  
  - Batch size: {4, 8, 16, 32, 64}  
- The only difference is the **loss weighting strategy**.  

---

### 🔮 Next step
- After this first weighted-loss run, we may also include the **mid-class boost ratio itself**  
as a tunable hyperparameter in Optuna (instead of fixing it to 1.5).  
- That way, the study can find the **optimal balance** between extremes vs. middle classes.  

---

✅ So in short: same training process as Ex.4,  
but **loss re-weighting** to focus on the harder mid classes.


In [ ]:
# # =========================
# # ADV DL – Part B: Monolingual baseline (RoBERTa) – Exercise-4 style
# # Custom loop + early stopping + W&B + Optuna ONLY; freeze base, unfreeze last k layers
# # Uses df_train / df_test with columns: OriginalTweet (str), Sentiment (str)
# # =========================

# import os, math, random, time, json
# from typing import Dict, List, Tuple

# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix
# import torch
# from torch.utils.data import Dataset, DataLoader
# from torch.cuda.amp import autocast, GradScaler
# import torch.nn.functional as F
# from collections import Counter

# # ---- deps ----
# # !pip -q install transformers==4.43.3 optuna==3.6.1 wandb==0.17.5 >/dev/null

# import transformers
# from transformers import (
#     AutoTokenizer, AutoModelForSequenceClassification,
#     DataCollatorWithPadding, get_linear_schedule_with_warmup
# )
# import os
# os.environ["TRANSFORMERS_NO_TF"] = "1"
# os.environ["TRANSFORMERS_NO_FLAX"] = "1"

# import optuna
# import wandb

# # -------------------------
# # Constants (no CFG, Optuna-only workflow)
# # -------------------------
# MODEL_NAME = "microsoft/mdeberta-v3-base"
# MAX_LEN = 512
# BATCH_SIZE = 16
# WARMUP_RATIO = 0.06
# GRAD_CLIP = 1.0
# USE_AMP = True

# # ❗ New W&B project & run base (to keep things separate)
# PROJECT = "adv-dl-p2-deberta-midf1"
# BASE_RUN_NAME = "microsoft/mdeberta-v3-base_full_ex_4_midf1"

# TRIALS = 20
# SEED = 42
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# def set_seed(seed=42):
#     random.seed(seed); np.random.seed(seed)
#     torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
#     torch.backends.cudnn.deterministic = True
#     torch.backends.cudnn.benchmark = False

# set_seed(SEED)

# # ---- GPU perf toggles (Windows-safe) ----
# torch.backends.cuda.matmul.allow_tf32 = True
# torch.backends.cudnn.allow_tf32 = True
# try:
#     torch.set_float32_matmul_precision("high")
# except Exception:
#     pass

# # -------------------------
# # Label mapping (5-way sentiment)
# # -------------------------
# CANON = {
#     "extremely negative": "extremely negative",
#     "negative": "negative",
#     "neutral": "neutral",
#     "positive": "positive",
#     "extremely positive": "extremely positive",
# }
# ORDER = ["extremely negative","negative","neutral","positive","extremely positive"]
# LABEL2ID = {lab: i for i, lab in enumerate(ORDER)}
# ID2LABEL = {i: lab for lab, i in LABEL2ID.items()}

# def normalize_label(s: str) -> str:
#     s = str(s).strip().lower()
#     s = s.replace("very negative", "extremely negative")
#     s = s.replace("very positive", "extremely positive")
#     s = s.replace("extreme negative", "extremely negative")
#     s = s.replace("extreme positive", "extremely positive")
#     return CANON.get(s, s)

# # -------------------------
# # Expect df_train, df_test in memory
# # -------------------------
# assert "OriginalTweet" in df_train.columns and "Sentiment" in df_train.columns, "df_train missing required columns"
# assert "OriginalTweet" in df_test.columns and "Sentiment" in df_test.columns, "df_test missing required columns"

# def prep_df(df: pd.DataFrame) -> pd.DataFrame:
#     df = df.copy()
#     df = df.dropna(subset=["OriginalTweet", "Sentiment"])
#     df["text"] = df["OriginalTweet"].astype(str).str.strip()
#     df["label_name"] = df["Sentiment"].apply(normalize_label)
#     df = df[df["label_name"].isin(ORDER)].reset_index(drop=True)
#     df["label"] = df["label_name"].map(LABEL2ID)
#     return df[["text", "label", "label_name"]]

# dftrain_ = prep_df(df_train)
# dftest_  = prep_df(df_test)

# train_df, val_df = train_test_split(
#     dftrain_, test_size=0.1, stratify=dftrain_["label"], random_state=SEED
# )
# print(f"Train/Val/Test sizes: {len(train_df)}/{len(val_df)}/{len(dftest_)}")

# # # --- class weights for mid-class emphasis (computed from training split) ---
# # _counts = Counter(train_df["label"].tolist())
# # _num_classes = len(ORDER)
# # _total = sum(_counts.values())
# # # inverse-freq: N / (K * n_c)
# # CLASS_WEIGHTS = torch.tensor(
# #     [_total / (_num_classes * _counts[i]) for i in range(_num_classes)],
# #     dtype=torch.float,
# #     device=DEVICE,
# # )
# # --- class weights with explicit mid-class boost ---
# from collections import Counter
# import numpy as np
# import torch

# _counts = Counter(train_df["label"].tolist())
# _num_classes = len(ORDER)
# _total = sum(_counts.values())

# # inverse frequency baseline: N / (K * n_c)
# inv = np.array([_total / (_num_classes * max(1, _counts.get(i, 0))) for i in range(_num_classes)],
#                dtype=np.float32)

# # indices
# IDX_EXT_NEG = LABEL2ID["extremely negative"]
# IDX_NEG     = LABEL2ID["negative"]
# IDX_NEU     = LABEL2ID["neutral"]
# IDX_POS     = LABEL2ID["positive"]
# IDX_EXT_POS = LABEL2ID["extremely positive"]

# # multiplicative boost: mids > extremes (tune 1.25–2.0 as you like)
# MID_BOOST = 1.5
# mult = np.ones(_num_classes, dtype=np.float32)
# mult[[IDX_NEG, IDX_NEU, IDX_POS]] = MID_BOOST   # boost the mid classes

# weights = inv * mult

# # optional: renormalize so the mean weight is 1.0 (keeps loss scale stable)
# weights = weights * (_num_classes / weights.sum())

# CLASS_WEIGHTS = torch.tensor(weights, dtype=torch.float, device=DEVICE)

# print("[loss] Using class-weighted CrossEntropy; weights per label:")
# print({ID2LABEL[i]: float(CLASS_WEIGHTS[i].item()) for i in range(_num_classes)})


# # -------------------------
# # Dataset & Collator
# # -------------------------
# class TweetDataset(Dataset):
#     def __init__(self, df: pd.DataFrame, tokenizer: transformers.PreTrainedTokenizerBase, max_len: int):
#         self.texts = df["text"].tolist()
#         self.labels = df["label"].tolist()
#         self.tok = tokenizer
#         self.max_len = max_len
#     def __len__(self): return len(self.texts)
#     def __getitem__(self, idx):
#         enc = self.tok(self.texts[idx], truncation=True, max_length=self.max_len, padding=False)
#         enc["labels"] = self.labels[idx]
#         return {k: torch.tensor(v) for k, v in enc.items()}

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
# train_ds = TweetDataset(train_df, tokenizer, MAX_LEN)
# val_ds   = TweetDataset(val_df, tokenizer, MAX_LEN)
# test_ds  = TweetDataset(dftest_, tokenizer, MAX_LEN)

# BATCH_SIZE=16
# # ---- pad_to_multiple_of=8 for Tensor Cores; Windows: workers=0 is often faster ----
# collate_fn = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=8)
# train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  collate_fn=collate_fn, num_workers=0, pin_memory=True, persistent_workers=False)
# val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn, num_workers=0, pin_memory=True, persistent_workers=False)
# test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn, num_workers=0, pin_memory=True, persistent_workers=False)

# # -------------------------
# # Model & Freeze/Unfreeze strategy
# # -------------------------
# def build_model(num_unfreeze_last_layers: int = 4):
#     model = AutoModelForSequenceClassification.from_pretrained(
#         MODEL_NAME, num_labels=len(ORDER), id2label=ID2LABEL, label2id=LABEL2ID
#     )
#     base = getattr(model, "roberta", None) or getattr(model, "bert", None) or getattr(model, "deberta", None)
#     if base is not None:
#         for p in base.parameters(): p.requires_grad = False
#         if hasattr(base, "encoder") and hasattr(base.encoder, "layer"):
#             k = num_unfreeze_last_layers
#             if k > 0:
#                 for layer in base.encoder.layer[-k:]:
#                     for p in layer.parameters(): p.requires_grad = True
#     for p in model.classifier.parameters(): p.requires_grad = True
#     return model.to(DEVICE)

# # -------------------------
# # Train / Eval utilities
# # -------------------------
# def get_optimizer_scheduler(model, num_training_steps: int, lr: float, weight_decay: float):
#     no_decay = ["bias", "LayerNorm.weight"]
#     optimizer_grouped_parameters = [
#         {"params": [p for n, p in model.named_parameters() if p.requires_grad and not any(nd in n for nd in no_decay)], "weight_decay": weight_decay},
#         {"params": [p for n, p in model.named_parameters() if p.requires_grad and any(nd in n for nd in no_decay)],  "weight_decay": 0.0},
#     ]
#     # try fused AdamW on CUDA (faster step) — falls back if unavailable
#     try:
#         optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=lr, weight_decay=weight_decay, fused=(DEVICE=="cuda"))
#     except TypeError:
#         optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=lr, weight_decay=weight_decay)
#     num_warmup = int(num_training_steps * WARMUP_RATIO)
#     scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup, num_training_steps=num_training_steps)
#     return optimizer, scheduler

# MIDS = [LABEL2ID["negative"], LABEL2ID["neutral"], LABEL2ID["positive"]]

# def evaluate(model, loader) -> Dict[str, float]:
#     model.eval()
#     preds, labels = [], []
#     with torch.no_grad():
#         for batch in loader:
#             batch = {k: v.to(DEVICE, non_blocking=True) for k, v in batch.items()}
#             # AMP autocast for faster eval math
#             with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),
#                           enabled=(DEVICE == "cuda" and USE_AMP)):
#                 logits = model(**batch).logits
#             preds.extend(torch.argmax(logits, dim=-1).detach().cpu().tolist())
#             labels.extend(batch["labels"].detach().cpu().tolist())
#     acc = accuracy_score(labels, preds)
#     p, r, f1, _ = precision_recall_fscore_support(labels, preds, average="macro", zero_division=0)
#     # extra: mid-class F1 (negative/neutral/positive)
#     p_mid, r_mid, f1_mid, _ = precision_recall_fscore_support(
#         labels, preds, labels=MIDS, average="macro", zero_division=0
#     )
#     return {"acc": acc, "precision": p, "recall": r, "f1": f1, "f1_mid": f1_mid}

# def train_one_run(hp: Dict) -> Tuple[str, Dict[str, float]]:
#     """
#     hp keys: run_name, num_unfreeze_last_layers, lr, weight_decay, epochs, patience, trial_number
#     """
#     run_name = hp["run_name"]
#     num_unfreeze = int(hp["num_unfreeze_last_layers"])
#     lr = float(hp["lr"])
#     wd = float(hp["weight_decay"])
#     epochs   = int(hp.get("epochs",   FIXED_EPOCHS))
#     patience = int(hp.get("patience", FIXED_PATIENCE))
#     model = build_model(num_unfreeze)
#     total_steps = int(math.ceil(len(train_loader) * epochs))
#     optimizer, scheduler = get_optimizer_scheduler(model, total_steps, lr, wd)

#     scaler = GradScaler(enabled=(DEVICE == "cuda" and USE_AMP))
#     best_metric = -1.0
#     no_improve = 0

#     # ❗ save to a different folder + name to avoid collisions
#     safe_run_name = run_name.replace("/", "__").replace("\\", "__")
#     ckpt_dir = "checkpoints_midf1"
#     os.makedirs(ckpt_dir, exist_ok=True)
#     best_path = os.path.join(ckpt_dir, f"best_midf1_{safe_run_name}.pt")

#     wandb_run = wandb.init(
#         project=PROJECT,
#         name=run_name,
#         config={
#             "model": MODEL_NAME,
#             "max_len": MAX_LEN,
#             "batch_size": BATCH_SIZE,
#             "epochs": epochs,
#             "lr": lr,
#             "weight_decay": wd,
#             "warmup_ratio": WARMUP_RATIO,
#             "grad_clip": GRAD_CLIP,
#             "num_unfreeze_last_layers": num_unfreeze,
#             "trial_number": hp.get("trial_number", -1),
#             "suggested_batch_size": hp.get("batch_size", BATCH_SIZE),
#         },
#         reinit=True,
#     )

#     # nicer W&B charts
#     wandb.define_metric("epoch")
#     wandb.define_metric("step")
#     wandb.define_metric("train/*", step_metric="step")
#     wandb.define_metric("val/*",   step_metric="epoch")

#     # print + log trainable params
#     total_params     = sum(p.numel() for p in model.parameters())
#     trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
#     print(f"Trainable params: {trainable_params:,} / {total_params:,} "
#           f"({100.0*trainable_params/total_params:.2f}%) ; unfreeze_last_k={num_unfreeze}")
#     wandb.log({"params/total": total_params,
#                "params/trainable": trainable_params,
#                "params/ratio": trainable_params/max(1,total_params)}, step=0)

#     for epoch in range(epochs):
#         model.train()
#         t0 = time.time()
#         running_loss = 0.0

#         for step, batch in enumerate(train_loader):
#             batch = {k: v.to(DEVICE, non_blocking=True) for k, v in batch.items()}
#             labels = batch.pop("labels")  # we compute weighted loss ourselves

#             optimizer.zero_grad(set_to_none=True)
#             # use BF16 if supported; else FP16
#             with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),
#                           enabled=(DEVICE == "cuda" and USE_AMP)):
#                 outputs = model(**batch)
#                 logits = outputs.logits
#                 # weighted CE (+ smoothing if available)
#                 try:
#                     loss = F.cross_entropy(logits, labels, weight=CLASS_WEIGHTS, label_smoothing=0.05)
#                 except TypeError:
#                     loss = F.cross_entropy(logits, labels, weight=CLASS_WEIGHTS)

#             scaler.scale(loss).backward()
#             if GRAD_CLIP is not None:
#                 scaler.unscale_(optimizer)
#                 torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
#             scaler.step(optimizer); scaler.update(); scheduler.step()
#             running_loss += loss.item()

#             if step % 20 == 0:
#                 wandb.log({"train/loss": loss.item(), "step": step + 1, "epoch": epoch + 1})

#             # periodic console + throughput log (about 10x per epoch)
#             if step % max(1, len(train_loader)//10) == 0 or step == 1:
#                 avg_loss = running_loss / max(1, (step + 1))
#                 elapsed  = time.time() - t0
#                 items    = (step + 1) * BATCH_SIZE
#                 itps     = items / max(elapsed, 1e-6)
#                 print(f"[e{epoch+1} b{step+1}/{len(train_loader)}] loss={loss.item():.4f} avg={avg_loss:.4f} it/s={itps:.1f}")
#                 wandb.log({"train/avg_loss_so_far": avg_loss,
#                            "train/items_per_sec": itps,
#                            "step": (epoch * len(train_loader)) + (step + 1),
#                            "epoch": epoch + 1})

#         # epoch-end validation
#         val_metrics = evaluate(model, val_loader)
#         elapsed = time.time() - t0

#         epoch_loss = running_loss / max(1, len(train_loader))
#         current_lr = scheduler.get_last_lr()[0]
#         wandb.log({
#             "train/epoch_loss": epoch_loss,
#             "val/acc": val_metrics["acc"],
#             "val/precision": val_metrics["precision"],
#             "val/recall": val_metrics["recall"],
#             "val/f1": val_metrics["f1"],
#             "val/mid_f1": val_metrics["f1_mid"],   # extra log (format unchanged elsewhere)
#             "lr": current_lr,
#             "time/epoch_sec": elapsed,
#             "epoch": epoch + 1,
#         })

#         # Early stopping on mid-class F1 (prints stay the same)
#         target_metric = val_metrics["f1_mid"]
#         if target_metric > best_metric:
#             best_metric = target_metric
#             torch.save(model.state_dict(), best_path)
#             no_improve = 0
#             wandb_run.summary["best_val_f1"] = best_metric  # kept same key for compatibility
#             wandb_run.summary["best_val_mid_f1"] = best_metric
#             wandb_run.summary["best_checkpoint_path"] = best_path
#             wandb.log({"val/best_f1_so_far": best_metric, "val/best_epoch": epoch + 1})
#         else:
#             no_improve += 1
#             if no_improve >= patience:
#                 print(f"Early stopping at epoch {epoch+1}")
#                 break

#         # console print line unchanged
#         print(f"Epoch {epoch+1}/{epochs} | "
#               f"loss={epoch_loss:.4f} | "
#               f"val_acc={val_metrics['acc']:.4f} | val_f1={val_metrics['f1']:.4f} | time={elapsed:.1f}s")

#     wandb.finish()

#     # Load best and return path + metrics on val for reference
#     model.load_state_dict(torch.load(best_path, map_location=DEVICE))
#     final_val = evaluate(model, val_loader)

#     # store final val in W&B summary for quick sorting
#     if wandb.run is not None:
#         wandb.run.summary["final_val_acc"] = final_val["acc"]
#         wandb.run.summary["final_val_precision"] = final_val["precision"]
#         wandb.run.summary["final_val_recall"] = final_val["recall"]
#         wandb.run.summary["final_val_f1"] = final_val["f1"]
#         wandb.run.summary["final_val_mid_f1"] = final_val["f1_mid"]

#     return best_path, final_val

# # -------------------------
# # Optuna hyperparameter tuning (ALWAYS ON)
# # -------------------------

# # Constants
# FIXED_EPOCHS = 12
# FIXED_PATIENCE = 4

# def objective(trial: optuna.trial.Trial):
#     params = {
#         "run_name": f"{BASE_RUN_NAME}_optuna_trial_{trial.number}",
#         "num_unfreeze_last_layers": trial.suggest_int("num_unfreeze_last_layers", 8, 12),
#         "lr": trial.suggest_float("lr", 8e-5, 1e-3, log=True),
#         "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-4, log=True),
#         "batch_size": trial.suggest_categorical("batch_size", [4, 8, 16, 32, 64]),
#         "epochs": FIXED_EPOCHS,
#         "patience": FIXED_PATIENCE,
#         "trial_number": trial.number,
#     }
#     path, val_metrics = train_one_run(params)
#     # console visibility per trial (unchanged)
#     print(f"[Trial {trial.number}] f1={val_metrics['f1']:.4f} | "
#           f"unfreeze_k={params['num_unfreeze_last_layers']} lr={params['lr']:.2e} "
#           f"wd={params['weight_decay']:.1e} suggested_bs={params['batch_size']}")
#     # report intermediate value for pruning if enabled (keep macro f1 for study objective)
#     trial.report(val_metrics["f1"], step=1)
#     return val_metrics["f1"]

# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=TRIALS, show_progress_bar=True)
# print("Best trial:", study.best_trial.number, "F1:", study.best_value)
# best_params = {"run_name": f"{BASE_RUN_NAME}_best_optuna", **study.best_trial.params}

# # also persist best params to a different folder/name
# os.makedirs("checkpoints_midf1", exist_ok=True)
# with open(os.path.join("checkpoints_midf1", "best_hparams_optuna_midf1.json"), "w") as f:
#     json.dump({**best_params, "epochs": FIXED_EPOCHS, "patience": FIXED_PATIENCE}, f, indent=2)


In [ ]:
# import json, os
# os.makedirs("checkpoints", exist_ok=True)

# # best hparams from Optuna (only suggested ones live here)
# best_hparams = study.best_trial.params

# # if your train_one_run expects epochs/patience and they were fixed (not suggested),
# # add them explicitly:
# best_hparams_complete = {
#     **best_hparams,
#     "epochs": FIXED_EPOCHS,       # or whatever you used
#     "patience": FIXED_PATIENCE,   # "
# }
# # hp_path = os.path.join("checkpoints", "best_hparams_optuna.json")
# hp_path = os.path.join("checkpoints", "best_hparams_optuna_2.json")
# with open(hp_path, "w") as f:
#     json.dump(best_hparams_complete, f, indent=2)
# print("Saved best hparams to:", hp_path)
# print("Best trial number:", study.best_trial.number, " value:", study.best_value)

In [2]:
from sklearn.model_selection import train_test_split
# Load CSVs (your files have columns: ['UserName','ScreenName','Location','TweetAt','OriginalTweet','Sentiment'])
TRAIN_CSV = "Corona_NLP_train_cleaned_translated.csv"   # or "Corona_NLP_train.csv"
TEST_CSV  = "Corona_NLP_test_cleaned_translated.csv"    # or "Corona_NLP_test.csv"


df_train = pd.read_csv(TRAIN_CSV, encoding="utf-8", engine="python")
df_test  = pd.read_csv(TEST_CSV,  encoding="utf-8", engine="python")

In [3]:
# =========================
# ADV DL – Part B: Monolingual baseline (RoBERTa) – Exercise-4 style
# Custom loop + early stopping + W&B + Optuna ONLY; freeze base, unfreeze last k layers
# Uses df_train / df_test with columns: OriginalTweet (str), Sentiment (str)
# =========================

import os, math, random, time, json
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix
import torch
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
import torch.nn.functional as F
from collections import Counter

# ---- deps ----
# !pip -q install transformers==4.43.3 optuna==3.6.1 wandb==0.17.5 >/dev/null

import transformers
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    DataCollatorWithPadding, get_linear_schedule_with_warmup
)
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"

import optuna
import wandb

# -------------------------
# Constants (no CFG, Optuna-only workflow)
# -------------------------
MODEL_NAME = "microsoft/mdeberta-v3-base"
MAX_LEN = 512
BATCH_SIZE = 16
WARMUP_RATIO = 0.06
GRAD_CLIP = 1.0
USE_AMP = True

# ❗ New W&B project & run base (to keep things separate)
PROJECT = "adv-dl-p2-deberta-midf1_new_study"
BASE_RUN_NAME = "microsoft/mdeberta-v3-base_full_ex_4_midf1"

TRIALS = 20
SEED = 42
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(SEED)

# ---- GPU perf toggles (Windows-safe) ----
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
try:
    torch.set_float32_matmul_precision("high")
except Exception:
    pass

# -------------------------
# Label mapping (5-way sentiment)
# -------------------------
CANON = {
    "extremely negative": "extremely negative",
    "negative": "negative",
    "neutral": "neutral",
    "positive": "positive",
    "extremely positive": "extremely positive",
}
ORDER = ["extremely negative","negative","neutral","positive","extremely positive"]
LABEL2ID = {lab: i for i, lab in enumerate(ORDER)}
ID2LABEL = {i: lab for lab, i in LABEL2ID.items()}

def normalize_label(s: str) -> str:
    s = str(s).strip().lower()
    s = s.replace("very negative", "extremely negative")
    s = s.replace("very positive", "extremely positive")
    s = s.replace("extreme negative", "extremely negative")
    s = s.replace("extreme positive", "extremely positive")
    return CANON.get(s, s)

# -------------------------
# Expect df_train, df_test in memory
# -------------------------
assert "OriginalTweet" in df_train.columns and "Sentiment" in df_train.columns, "df_train missing required columns"
assert "OriginalTweet" in df_test.columns and "Sentiment" in df_test.columns, "df_test missing required columns"

def prep_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df = df.dropna(subset=["OriginalTweet", "Sentiment"])
    df["text"] = df["OriginalTweet"].astype(str).str.strip()
    df["label_name"] = df["Sentiment"].apply(normalize_label)
    df = df[df["label_name"].isin(ORDER)].reset_index(drop=True)
    df["label"] = df["label_name"].map(LABEL2ID)
    return df[["text", "label", "label_name"]]

dftrain_ = prep_df(df_train)
dftest_  = prep_df(df_test)

train_df, val_df = train_test_split(
    dftrain_, test_size=0.1, stratify=dftrain_["label"], random_state=SEED
)
print(f"Train/Val/Test sizes: {len(train_df)}/{len(val_df)}/{len(dftest_)}")



IDX_EXT_NEG = LABEL2ID["extremely negative"]
IDX_NEG     = LABEL2ID["negative"]
IDX_NEU     = LABEL2ID["neutral"]
IDX_POS     = LABEL2ID["positive"]
IDX_EXT_POS = LABEL2ID["extremely positive"]

_num_classes = len(ORDER)

# knobs: make mids heavier than extremes
MID_WEIGHT = 1.5   # applied to negative/neutral/positive
EXT_WEIGHT = 0.8   # applied to extremely negative/positive (less than MID_WEIGHT)

weights = np.full(_num_classes, EXT_WEIGHT, dtype=np.float32)
weights[[IDX_NEG, IDX_NEU, IDX_POS]] = MID_WEIGHT

# normalize so average weight is 1.0 (optional but recommended)
weights = weights * (_num_classes / weights.sum())

CLASS_WEIGHTS = torch.tensor(weights, dtype=torch.float, device=DEVICE)

print("[loss] Using tiered class weights; per label:")
print({ID2LABEL[i]: float(CLASS_WEIGHTS[i].item()) for i in range(_num_classes)})

# -------------------------
# Dataset & Collator
# -------------------------
class TweetDataset(Dataset):
    def __init__(self, df: pd.DataFrame, tokenizer: transformers.PreTrainedTokenizerBase, max_len: int):
        self.texts = df["text"].tolist()
        self.labels = df["label"].tolist()
        self.tok = tokenizer
        self.max_len = max_len
    def __len__(self): return len(self.texts)
    def __getitem__(self, idx):
        enc = self.tok(self.texts[idx], truncation=True, max_length=self.max_len, padding=False)
        enc["labels"] = self.labels[idx]
        return {k: torch.tensor(v) for k, v in enc.items()}

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
train_ds = TweetDataset(train_df, tokenizer, MAX_LEN)
val_ds   = TweetDataset(val_df, tokenizer, MAX_LEN)
test_ds  = TweetDataset(dftest_, tokenizer, MAX_LEN)

BATCH_SIZE=16
# ---- pad_to_multiple_of=8 for Tensor Cores; Windows: workers=0 is often faster ----
collate_fn = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=8)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  collate_fn=collate_fn, num_workers=0, pin_memory=True, persistent_workers=False)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn, num_workers=0, pin_memory=True, persistent_workers=False)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn, num_workers=0, pin_memory=True, persistent_workers=False)

# -------------------------
# Model & Freeze/Unfreeze strategy
# -------------------------
def build_model(num_unfreeze_last_layers: int = 4):
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME, num_labels=len(ORDER), id2label=ID2LABEL, label2id=LABEL2ID
    )
    base = getattr(model, "roberta", None) or getattr(model, "bert", None) or getattr(model, "deberta", None)
    if base is not None:
        for p in base.parameters(): p.requires_grad = False
        if hasattr(base, "encoder") and hasattr(base.encoder, "layer"):
            k = num_unfreeze_last_layers
            if k > 0:
                for layer in base.encoder.layer[-k:]:
                    for p in layer.parameters(): p.requires_grad = True
    for p in model.classifier.parameters(): p.requires_grad = True
    return model.to(DEVICE)

# -------------------------
# Train / Eval utilities
# -------------------------
def get_optimizer_scheduler(model, num_training_steps: int, lr: float, weight_decay: float):
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {"params": [p for n, p in model.named_parameters() if p.requires_grad and not any(nd in n for nd in no_decay)], "weight_decay": weight_decay},
        {"params": [p for n, p in model.named_parameters() if p.requires_grad and any(nd in n for nd in no_decay)],  "weight_decay": 0.0},
    ]
    # try fused AdamW on CUDA (faster step) — falls back if unavailable
    try:
        optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=lr, weight_decay=weight_decay, fused=(DEVICE=="cuda"))
    except TypeError:
        optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=lr, weight_decay=weight_decay)
    num_warmup = int(num_training_steps * WARMUP_RATIO)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup, num_training_steps=num_training_steps)
    return optimizer, scheduler

MIDS = [LABEL2ID["negative"], LABEL2ID["neutral"], LABEL2ID["positive"]]

def evaluate(model, loader) -> Dict[str, float]:
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for batch in loader:
            batch = {k: v.to(DEVICE, non_blocking=True) for k, v in batch.items()}
            # AMP autocast for faster eval math
            with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),
                          enabled=(DEVICE == "cuda" and USE_AMP)):
                logits = model(**batch).logits
            preds.extend(torch.argmax(logits, dim=-1).detach().cpu().tolist())
            labels.extend(batch["labels"].detach().cpu().tolist())
    acc = accuracy_score(labels, preds)
    p, r, f1, _ = precision_recall_fscore_support(labels, preds, average="macro", zero_division=0)
    # extra: mid-class F1 (negative/neutral/positive)
    p_mid, r_mid, f1_mid, _ = precision_recall_fscore_support(
        labels, preds, labels=MIDS, average="macro", zero_division=0
    )
    return {"acc": acc, "precision": p, "recall": r, "f1": f1, "f1_mid": f1_mid}

def train_one_run(hp: Dict) -> Tuple[str, Dict[str, float]]:
    """
    hp keys: run_name, num_unfreeze_last_layers, lr, weight_decay, epochs, patience, trial_number
    """
    run_name = hp["run_name"]
    num_unfreeze = int(hp["num_unfreeze_last_layers"])
    lr = float(hp["lr"])
    wd = float(hp["weight_decay"])
    epochs   = int(hp.get("epochs",   FIXED_EPOCHS))
    patience = int(hp.get("patience", FIXED_PATIENCE))
    model = build_model(num_unfreeze)
    total_steps = int(math.ceil(len(train_loader) * epochs))
    optimizer, scheduler = get_optimizer_scheduler(model, total_steps, lr, wd)

    scaler = GradScaler(enabled=(DEVICE == "cuda" and USE_AMP))
    best_metric = -1.0
    no_improve = 0

    # ❗ save to a different folder + name to avoid collisions
    safe_run_name = run_name.replace("/", "__").replace("\\", "__")
    ckpt_dir = "checkpoints_midf1"
    os.makedirs(ckpt_dir, exist_ok=True)
    best_path = os.path.join(ckpt_dir, f"best_midf1_{safe_run_name}.pt")

    wandb_run = wandb.init(
        project=PROJECT,
        name=run_name,
        config={
            "model": MODEL_NAME,
            "max_len": MAX_LEN,
            "batch_size": BATCH_SIZE,
            "epochs": epochs,
            "lr": lr,
            "weight_decay": wd,
            "warmup_ratio": WARMUP_RATIO,
            "grad_clip": GRAD_CLIP,
            "num_unfreeze_last_layers": num_unfreeze,
            "trial_number": hp.get("trial_number", -1),
            "suggested_batch_size": hp.get("batch_size", BATCH_SIZE),
        },
        reinit=True,
    )

    # nicer W&B charts
    wandb.define_metric("epoch")
    wandb.define_metric("step")
    wandb.define_metric("train/*", step_metric="step")
    wandb.define_metric("val/*",   step_metric="epoch")

    # print + log trainable params
    total_params     = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Trainable params: {trainable_params:,} / {total_params:,} "
          f"({100.0*trainable_params/total_params:.2f}%) ; unfreeze_last_k={num_unfreeze}")
    wandb.log({"params/total": total_params,
               "params/trainable": trainable_params,
               "params/ratio": trainable_params/max(1,total_params)}, step=0)

    for epoch in range(epochs):
        model.train()
        t0 = time.time()
        running_loss = 0.0

        for step, batch in enumerate(train_loader):
            batch = {k: v.to(DEVICE, non_blocking=True) for k, v in batch.items()}
            labels = batch.pop("labels")  # we compute weighted loss ourselves

            optimizer.zero_grad(set_to_none=True)
            # use BF16 if supported; else FP16
            with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),
                          enabled=(DEVICE == "cuda" and USE_AMP)):
                outputs = model(**batch)
                logits = outputs.logits
                # weighted CE (+ smoothing if available)
                try:
                    loss = F.cross_entropy(logits, labels, weight=CLASS_WEIGHTS, label_smoothing=0.05)
                except TypeError:
                    loss = F.cross_entropy(logits, labels, weight=CLASS_WEIGHTS)

            scaler.scale(loss).backward()
            if GRAD_CLIP is not None:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
            scaler.step(optimizer); scaler.update(); scheduler.step()
            running_loss += loss.item()

            if step % 20 == 0:
                wandb.log({"train/loss": loss.item(), "step": step + 1, "epoch": epoch + 1})

            # periodic console + throughput log (about 10x per epoch)
            if step % max(1, len(train_loader)//10) == 0 or step == 1:
                avg_loss = running_loss / max(1, (step + 1))
                elapsed  = time.time() - t0
                items    = (step + 1) * BATCH_SIZE
                itps     = items / max(elapsed, 1e-6)
                print(f"[e{epoch+1} b{step+1}/{len(train_loader)}] loss={loss.item():.4f} avg={avg_loss:.4f} it/s={itps:.1f}")
                wandb.log({"train/avg_loss_so_far": avg_loss,
                           "train/items_per_sec": itps,
                           "step": (epoch * len(train_loader)) + (step + 1),
                           "epoch": epoch + 1})

        # epoch-end validation
        val_metrics = evaluate(model, val_loader)
        elapsed = time.time() - t0

        epoch_loss = running_loss / max(1, len(train_loader))
        current_lr = scheduler.get_last_lr()[0]
        wandb.log({
            "train/epoch_loss": epoch_loss,
            "val/acc": val_metrics["acc"],
            "val/precision": val_metrics["precision"],
            "val/recall": val_metrics["recall"],
            "val/f1": val_metrics["f1"],
            "val/mid_f1": val_metrics["f1_mid"],   # extra log (format unchanged elsewhere)
            "lr": current_lr,
            "time/epoch_sec": elapsed,
            "epoch": epoch + 1,
        })

        # Early stopping on mid-class F1 (prints stay the same)
        target_metric = val_metrics["f1_mid"]
        if target_metric > best_metric:
            best_metric = target_metric
            torch.save(model.state_dict(), best_path)
            no_improve = 0
            wandb_run.summary["best_val_f1"] = best_metric  # kept same key for compatibility
            wandb_run.summary["best_val_mid_f1"] = best_metric
            wandb_run.summary["best_checkpoint_path"] = best_path
            wandb.log({"val/best_f1_so_far": best_metric, "val/best_epoch": epoch + 1})
        else:
            no_improve += 1
            if no_improve >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

        # console print line unchanged
        print(f"Epoch {epoch+1}/{epochs} | "
              f"loss={epoch_loss:.4f} | "
              f"val_acc={val_metrics['acc']:.4f} | val_f1={val_metrics['f1']:.4f} | time={elapsed:.1f}s")

    wandb.finish()

    # Load best and return path + metrics on val for reference
    model.load_state_dict(torch.load(best_path, map_location=DEVICE))
    final_val = evaluate(model, val_loader)

    # store final val in W&B summary for quick sorting
    if wandb.run is not None:
        wandb.run.summary["final_val_acc"] = final_val["acc"]
        wandb.run.summary["final_val_precision"] = final_val["precision"]
        wandb.run.summary["final_val_recall"] = final_val["recall"]
        wandb.run.summary["final_val_f1"] = final_val["f1"]
        wandb.run.summary["final_val_mid_f1"] = final_val["f1_mid"]

    return best_path, final_val

# -------------------------
# Optuna hyperparameter tuning (ALWAYS ON)
# -------------------------

# Constants
FIXED_EPOCHS = 7
FIXED_PATIENCE = 3

def objective(trial: optuna.trial.Trial):
    params = {
        "run_name": f"{BASE_RUN_NAME}_optuna_trial_{trial.number}",
        "num_unfreeze_last_layers": trial.suggest_int("num_unfreeze_last_layers", 4, 12),
        "lr": trial.suggest_float("lr", 1e-5, 1e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 1e-7, 1e-5, log=True),
        "batch_size": trial.suggest_categorical("batch_size", [4, 8, 16, 32, 64]),
        "epochs": FIXED_EPOCHS,
        "patience": FIXED_PATIENCE,
        "trial_number": trial.number,
    }
    path, val_metrics = train_one_run(params)
    # console visibility per trial (unchanged)
    print(f"[Trial {trial.number}] f1={val_metrics['f1']:.4f} | "
          f"unfreeze_k={params['num_unfreeze_last_layers']} lr={params['lr']:.2e} "
          f"wd={params['weight_decay']:.1e} suggested_bs={params['batch_size']}")
    # report intermediate value for pruning if enabled (keep macro f1 for study objective)
    trial.report(val_metrics["f1"], step=1)
    return val_metrics["f1"]

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=TRIALS, show_progress_bar=True)
print("Best trial:", study.best_trial.number, "F1:", study.best_value)
best_params = {"run_name": f"{BASE_RUN_NAME}_best_optuna", **study.best_trial.params}

# also persist best params to a different folder/name
os.makedirs("checkpoints_midf1", exist_ok=True)
with open(os.path.join("checkpoints_midf1", "best_hparams_optuna_midf1.json"), "w") as f:
    json.dump({**best_params, "epochs": FIXED_EPOCHS, "patience": FIXED_PATIENCE}, f, indent=2)


Train/Val/Test sizes: 37039/4116/3798
[loss] Using tiered class weights; per label:
{'extremely negative': 0.6557376980781555, 'negative': 1.2295081615447998, 'neutral': 1.2295081615447998, 'positive': 1.2295081615447998, 'extremely positive': 0.6557376980781555}


C:\Users\adishalit1\AppData\Local\anaconda3\envs\dl4090\Lib\site-packages\transformers\convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
[I 2025-08-17 17:48:02,852] A new study created in memory with name: no-name-03ead3fe-dcdf-4b18-bc81-53020fce6c05
  0%|          | 0/20 [00:00<?, ?it/s]Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/mdeberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use

Trainable params: 28,945,925 / 278,813,189 (10.38%) ; unfreeze_last_k=4


C:\Users\adishalit1\AppData\Local\Temp\ipykernel_23124\32348540.py:339: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),


[e1 b1/2315] loss=1.6733 avg=1.6733 it/s=41.6
[e1 b2/2315] loss=1.6255 avg=1.6494 it/s=65.9
[e1 b232/2315] loss=1.5370 avg=1.5309 it/s=522.7
[e1 b463/2315] loss=1.3853 avg=1.4602 it/s=541.8
[e1 b694/2315] loss=1.0948 avg=1.3923 it/s=543.0
[e1 b925/2315] loss=1.0755 avg=1.3458 it/s=532.2
[e1 b1156/2315] loss=0.8271 avg=1.3049 it/s=529.5
[e1 b1387/2315] loss=0.9937 avg=1.2741 it/s=529.8
[e1 b1618/2315] loss=0.7183 avg=1.2491 it/s=529.8
[e1 b1849/2315] loss=1.2618 avg=1.2267 it/s=535.6
[e1 b2080/2315] loss=1.1084 avg=1.2071 it/s=540.1
[e1 b2311/2315] loss=1.3135 avg=1.1931 it/s=543.1


...

C:\Users\adishalit1\AppData\Local\Temp\ipykernel_23124\32348540.py:257: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Epoch 7/7 | loss=0.4107 | val_acc=0.8547 | val_f1=0.8581 | time=106.2s


epoch,▁▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇█████
lr,█▇▆▅▃▂▁
params/ratio,▁
params/total,▁
params/trainable,▁
step,▁▁▁▁▁▁▂▁▁▁▂▁▂▂▂▂▁▂▄▂▂▂▂▂▂▁▅▂▆▆▂▂▂▇▁▁▂▂▂█
time/epoch_sec,▄▁█▄▆▇▇
train/avg_loss_so_far,██▇▇▇▆▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▃▂▂▆▄▂▂▂▂▂▂▁▂▂▂▂▂▂
train/epoch_loss,█▄▃▂▂▁▁
train/items_per_sec,▁▇▇▇▇██▇▇███▇▆▆▇▇▇▇▆████▇▃▅██▇██▇▆▂▆▇▇▇▇
train/loss,██▇█▇▄▄▄▄▃▂▃▂▂▃▃▃▂▃▆▂▃▁▂▂▃▂▃▁▄▂▁▂▁▁▂▁▄▂▁


C:\Users\adishalit1\AppData\Local\Temp\ipykernel_23124\32348540.py:257: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),
Best trial: 15. Best value: 0.876329: 100%|██████████| 20/20 [3:33:15<00:00, 639.78s/it]

[Trial 19] f1=0.8615 | unfreeze_k=11 lr=1.54e-05 wd=6.3e-06 suggested_bs=8
[I 2025-08-17 21:21:18,523] Trial 19 finished with value: 0.8615373448847018 and parameters: {'num_unfreeze_last_layers': 11, 'lr': 1.5441131471553194e-05, 'weight_decay': 6.254940759615576e-06, 'batch_size': 8}. Best is trial 15 with value: 0.8763286523329038.
Best trial: 15 F1: 0.8763286523329038


## 📊 Results – Weighted Loss Experiment (mid-class emphasis)

This second run (full training, Ex.4 style + weighted loss) gave **more consistent improvements** compared to the plain loss function:

- The **best trial** reached **val/F1 = 0.879** and **mid-class F1 = 0.867**,  
  which is stronger than our earlier unweighted runs.  
- The **top trials (11, 15, 8, 13)** all sit around **0.86–0.87 F1**,  
  showing that the weighting helped stabilize the optimization and reduce variance.  
- As expected, the **best LRs are still in the range 2.5e-5 – 7.5e-5** with small weight decay (≈1e-6–1e-5).  
- Best unfreeze depth remains **10–12 layers**, similar to before.  
- Almost all successful runs used **batch size = 8**, confirming smaller batches give more stable convergence.  

---

### 🚀 Key Takeaways
- Weighted loss **did boost performance on the mid classes**, which were previously weaker.  
- Even though absolute F1 gains are modest, this approach reduced the gap between extreme vs. mid sentiments.  
- Bad trials (very high learning rate >1e-3 or shallow unfreezing <8) still collapse (F1 ~0.14),  
  but **the good region is now clearer and more reproducible**.  

---

We will now check for test results. 
  


In [4]:


with open(os.path.join("checkpoints_midf1", "best_hparams_optuna_midf1.json")) as f:
    best_hparams = json.load(f)

# give a distinct name for the final run
best_hparams["run_name"] = f"{BASE_RUN_NAME}_best_optuna_weighed_classes"

# (optional) bump epochs here; see guidance below
best_hparams["epochs"] = 15      # higher cap
best_hparams["patience"] = 4     # unchanged
# ckpt, val = train_one_run(best_hparams)

#
# best_ckpt_path, best_val = train_one_run(best_hparams)
# print("Final best checkpoint:", best_ckpt_path)
# print("Final val metrics:", best_val)


In [5]:
# Retrain best config to get a clean checkpoint
best_ckpt, _ = train_one_run(best_hparams)
best_path = best_ckpt


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/mdeberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\adishalit1\AppData\Local\Temp\ipykernel_23124\32348540.py:284: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=(DEVICE == "cuda" and USE_AMP))


Trainable params: 78,561,029 / 278,813,189 (28.18%) ; unfreeze_last_k=11
[e1 b1/2315] loss=1.6515 avg=1.6515 it/s=239.0
[e1 b2/2315] loss=1.6300 avg=1.6407 it/s=316.0


C:\Users\adishalit1\AppData\Local\Temp\ipykernel_23124\32348540.py:339: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),


[e1 b232/2315] loss=1.4546 avg=1.5346 it/s=328.2
[e1 b463/2315] loss=1.2486 avg=1.4866 it/s=346.9
[e1 b694/2315] loss=1.2882 avg=1.4268 it/s=350.5
[e1 b925/2315] loss=0.9568 avg=1.3558 it/s=352.5
[e1 b1156/2315] loss=0.5954 avg=1.2882 it/s=355.5
[e1 b1387/2315] loss=1.0275 avg=1.2351 it/s=358.5
[e1 b1618/2315] loss=0.8343 avg=1.1911 it/s=362.1
[e1 b1849/2315] loss=0.6917 avg=1.1536 it/s=358.7
[e1 b2080/2315] loss=0.7476 avg=1.1215 it/s=336.6
[e1 b2311/2315] loss=0.9310 avg=1.0926 it/s=340.8


C:\Users\adishalit1\AppData\Local\Temp\ipykernel_23124\32348540.py:257: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),


...

[e15 b232/2315] loss=0.2093 avg=0.2586 it/s=389.4
[e15 b463/2315] loss=0.2093 avg=0.2582 it/s=388.4
[e15 b694/2315] loss=0.2091 avg=0.2580 it/s=384.4
[e15 b925/2315] loss=0.2029 avg=0.2587 it/s=340.0
[e15 b1156/2315] loss=0.2098 avg=0.2601 it/s=321.4
[e15 b1387/2315] loss=0.1895 avg=0.2619 it/s=323.9
[e15 b1618/2315] loss=0.2427 avg=0.2598 it/s=327.4
[e15 b1849/2315] loss=0.5179 avg=0.2611 it/s=333.2
[e15 b2080/2315] loss=0.4498 avg=0.2607 it/s=337.4
[e15 b2311/2315] loss=0.4474 avg=0.2611 it/s=341.7


C:\Users\adishalit1\AppData\Local\Temp\ipykernel_23124\32348540.py:257: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Epoch 15/15 | loss=0.2612 | val_acc=0.8763 | val_f1=0.8795 | time=113.2s


epoch,▁▁▁▁▁▁▁▁▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▇▇▇▇▇▇▇▇▇▇▇███
lr,██▇▇▆▆▅▅▄▄▃▃▂▂▁
params/ratio,▁
params/total,▁
params/trainable,▁
step,▁▁▁▁▂▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇▁▁█▁▁▁
time/epoch_sec,▆▃▂▁▂▃▂▁▄▃▆█▅▇▆
train/avg_loss_so_far,█▆▃▄▄▃▃▃▂▃▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▂▁▁▁▁▁▁▁▁▁
train/epoch_loss,█▅▄▃▃▃▂▂▂▂▁▁▁▁▁
train/items_per_sec,▄▆▆▅▅▆▆▅▇▅█▇▇█▇▇▇▇▇▇▇▇▇▆▆▆▁▅▆▆▇▇▇▁▁▄▄▇▆█
train/loss,█▆▇▄▃▂▃▂▂▃▃▁▂▂▃▂▂▁▃▂▂▂▂▁▂▁▂▂▂▂▁▁▁▂▁▂▁▁▁▁


C:\Users\adishalit1\AppData\Local\Temp\ipykernel_23124\32348540.py:257: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),


## 🧾 Final Test Results – Weighted Loss Run  

**Overall metrics**  
`acc = 0.8626 | f1_macro = 0.8658 | precision_macro = 0.8636 | recall_macro = 0.8686`  

---

## 🔄 Comparison with Ex.4 Baseline (no weighted loss)

| Label                | Ex.4 F1 | Weighted F1 | Change |
|-----------------------|---------|-------------|--------|
| extremely negative    | 0.89    | 0.88        | -0.01  |
| negative              | 0.87    | 0.85        | -0.02  |
| neutral               | 0.86    | 0.86        | ~0.00  |
| positive              | 0.84    | 0.85        | +0.01  |
| extremely positive    | 0.88    | 0.89        | +0.01  |
| **Macro avg**         | **0.87**| **0.87**    | ~0.00  |
| **Accuracy**          | **0.87**| **0.86**    | -0.01  |

---

### 📌 Takeaways
- **Weighted loss did not drastically change overall macro F1 (~0.87 → ~0.87).**  
- Gains are seen in **positive and extremely positive**, which improved slightly.  
- The **negative class** dropped a bit, suggesting the weighting may have shifted attention slightly away from it.  
- Macro balance looks good — the mid classes (neutral, positive) now perform closer to extremes.  

✅ In short: weighted loss **kept overall performance stable** while slightly **balancing mid-class performance**.  



In [6]:
# # Retrain best config to get a clean checkpoint
# best_ckpt, _ = train_one_run(best_params)
# best_path = best_ckpt
best_params=best_hparams
# -------------------------
# Final evaluation on TEST (+ W&B logging)
# -------------------------
model = build_model(best_params["num_unfreeze_last_layers"])
model.load_state_dict(torch.load(best_path, map_location=DEVICE))
model.eval()

all_preds, all_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(DEVICE, non_blocking=True) for k, v in batch.items()}
        with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),
                      enabled=(DEVICE == "cuda" and USE_AMP)):
            logits = model(**batch).logits
        all_preds.extend(torch.argmax(logits, dim=-1).detach().cpu().tolist())
        all_labels.extend(batch["labels"].detach().cpu().tolist())

acc = accuracy_score(all_labels, all_preds)
p, r, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="macro", zero_division=0)
print(f"\nTEST | acc={acc:.4f} | f1_macro={f1:.4f} | precision_macro={p:.4f} | recall_macro={r:.4f}\n")

print("Per-class report (ids map to labels):")
print(ID2LABEL)
report = classification_report(
    all_labels, all_preds,
    target_names=[ID2LABEL[i] for i in range(len(ORDER))],
    zero_division=0, output_dict=True
)
print(classification_report(
    all_labels, all_preds,
    target_names=[ID2LABEL[i] for i in range(len(ORDER))],
    zero_division=0
))



Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/mdeberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\adishalit1\AppData\Local\Temp\ipykernel_23124\4076346695.py:16: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),



TEST | acc=0.8626 | f1_macro=0.8658 | precision_macro=0.8636 | recall_macro=0.8686

Per-class report (ids map to labels):
{0: 'extremely negative', 1: 'negative', 2: 'neutral', 3: 'positive', 4: 'extremely positive'}
                    precision    recall  f1-score   support

extremely negative       0.86      0.90      0.88       592
          negative       0.86      0.84      0.85      1041
           neutral       0.84      0.89      0.86       619
          positive       0.86      0.83      0.85       947
extremely positive       0.90      0.88      0.89       599

          accuracy                           0.86      3798
         macro avg       0.86      0.87      0.87      3798
      weighted avg       0.86      0.86      0.86      3798



In [7]:
# ---- W&B: log test metrics, per-class scores, and confusion matrix ----
test_run = wandb.init(project=PROJECT, name=f"{BASE_RUN_NAME}_test_wighted_classes", resume="allow", reinit=True)
log_payload = {
    "test/acc": acc,
    "test/precision_macro": p,
    "test/recall_macro": r,
    "test/f1_macro": f1,
}
for cls_name in ORDER:
    if cls_name in report:
        log_payload[f"test/{cls_name}/precision"] = report[cls_name]["precision"]
        log_payload[f"test/{cls_name}/recall"]    = report[cls_name]["recall"]
        log_payload[f"test/{cls_name}/f1"]        = report[cls_name]["f1-score"]

wandb.log(log_payload)

cm = confusion_matrix(all_labels, all_preds, labels=list(range(len(ORDER))))
wandb.log({
    "test/confusion_matrix": wandb.plot.confusion_matrix(
        y_true=all_labels,
        preds=all_preds,
        class_names=[ID2LABEL[i] for i in range(len(ID2LABEL))]
    )
})
test_run.summary["best_checkpoint_path"] = best_path
test_run.summary["test_f1_macro"] = f1
wandb.finish()



wandb: ERROR The nbformat package was not found. It is required to save notebook history.


test/acc,▁
test/extremely negative/f1,▁
test/extremely negative/precision,▁
test/extremely negative/recall,▁
test/extremely positive/f1,▁
test/extremely positive/precision,▁
test/extremely positive/recall,▁
test/f1_macro,▁
test/negative/f1,▁
test/negative/precision,▁
test/negative/recall,▁


## ⚖️ Extra Trials – Ratio as HP

We also tried an extended search where the **mid/extreme weight ratio** (`ratio_mid_ext`) itself was treated as a **hyperparameter**, alongside the usual learning rate, weight decay, batch size, and unfreezing depth.  

🔍 **Outcome:**  
- The results did **not show consistent improvements** beyond our fixed-weight run.  
- Best trials stayed around the same macro F1 (~0.86–0.87), without a clear advantage over the non-weighted baseline.  
- Some mid-class F1 gains were offset by drops in other classes, leaving overall performance unchanged.

---

⏳ **Decision**: Since we are limited in time and weighted models did not prove a clear benefit, we will **not include weighted-loss variants in the final ensemble**.  
We will continue with the **standard models** that showed stronger and more stable results in Ex.4/Ex.5.  


In [ ]:
# # =========================
# # ADV DL – Part B: Monolingual baseline (DeBERTa) – Exercise-4 style
# # Custom loop + early stopping + W&B + Optuna; freeze base, unfreeze last k layers
# # Focus: improve mid-class (negative/neutral/positive) via tiered class-weighted loss
# # Uses df_train / df_test with columns: OriginalTweet (str), Sentiment (str)
# # =========================

# import os, math, random, time, json
# from typing import Dict, List, Tuple

# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix
# import torch
# from torch.utils.data import Dataset, DataLoader
# from torch.cuda.amp import autocast, GradScaler
# import torch.nn.functional as F

# # ---- deps ----
# # !pip -q install transformers==4.43.3 optuna==3.6.1 wandb==0.17.5 >/dev/null

# import transformers
# from transformers import (
#     AutoTokenizer, AutoModelForSequenceClassification,
#     DataCollatorWithPadding, get_linear_schedule_with_warmup
# )

# os.environ["TRANSFORMERS_NO_TF"] = "1"
# os.environ["TRANSFORMERS_NO_FLAX"] = "1"

# import optuna
# import wandb

# # -------------------------
# # Constants (no CFG, Optuna-only workflow)
# # -------------------------
# MODEL_NAME = "microsoft/mdeberta-v3-base"
# MAX_LEN = 512
# BATCH_SIZE_DEFAULT = 16
# WARMUP_RATIO_DEFAULT = 0.06
# GRAD_CLIP_DEFAULT = 1.0
# USE_AMP = True
# FIXED_EPOCHS=8
# FIXED_PATIENTS=3
# # New W&B project & distinct run base (keeps things separate)
# PROJECT = "adv-dl-p2-deberta-midf1_new_study_2"
# BASE_RUN_NAME = "microsoft/mdeberta-v3-base_full_ex_4_midf1"

# TRIALS = 30
# SEED = 42
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# EARLY_ABORT_F1_E1 = 0.20  # if val macro-F1 after epoch 1 is below this → stop the run

# def set_seed(seed=42):
#     random.seed(seed); np.random.seed(seed)
#     torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
#     torch.backends.cudnn.deterministic = True
#     torch.backends.cudnn.benchmark = False

# set_seed(SEED)

# # ---- GPU perf toggles (Windows-safe) ----
# torch.backends.cuda.matmul.allow_tf32 = True
# torch.backends.cudnn.allow_tf32 = True
# try:
#     torch.set_float32_matmul_precision("high")
# except Exception:
#     pass

# # -------------------------
# # Label mapping (5-way sentiment)
# # -------------------------
# CANON = {
#     "extremely negative": "extremely negative",
#     "negative": "negative",
#     "neutral": "neutral",
#     "positive": "positive",
#     "extremely positive": "extremely positive",
# }
# ORDER = ["extremely negative","negative","neutral","positive","extremely positive"]
# LABEL2ID = {lab: i for i, lab in enumerate(ORDER)}
# ID2LABEL = {i: lab for lab, i in LABEL2ID.items()}

# def normalize_label(s: str) -> str:
#     s = str(s).strip().lower()
#     s = s.replace("very negative", "extremely negative")
#     s = s.replace("very positive", "extremely positive")
#     s = s.replace("extreme negative", "extremely negative")
#     s = s.replace("extreme positive", "extremely positive")
#     return CANON.get(s, s)

# # -------------------------
# # Expect df_train, df_test in memory
# # -------------------------
# assert "OriginalTweet" in df_train.columns and "Sentiment" in df_train.columns, "df_train missing required columns"
# assert "OriginalTweet" in df_test.columns and "Sentiment" in df_test.columns, "df_test missing required columns"

# def prep_df(df: pd.DataFrame) -> pd.DataFrame:
#     df = df.copy()
#     df = df.dropna(subset=["OriginalTweet", "Sentiment"])
#     df["text"] = df["OriginalTweet"].astype(str).str.strip()
#     df["label_name"] = df["Sentiment"].apply(normalize_label)
#     df = df[df["label_name"].isin(ORDER)].reset_index(drop=True)
#     df["label"] = df["label_name"].map(LABEL2ID)
#     return df[["text", "label", "label_name"]]

# dftrain_ = prep_df(df_train)
# dftest_  = prep_df(df_test)

# train_df, val_df = train_test_split(
#     dftrain_, test_size=0.1, stratify=dftrain_["label"], random_state=SEED
# )
# print(f"Train/Val/Test sizes: {len(train_df)}/{len(val_df)}/{len(dftest_)}")

# # -------------------------
# # Dataset & Collator
# # -------------------------
# class TweetDataset(Dataset):
#     def __init__(self, df: pd.DataFrame, tokenizer: transformers.PreTrainedTokenizerBase, max_len: int):
#         self.texts = df["text"].tolist()
#         self.labels = df["label"].tolist()
#         self.tok = tokenizer
#         self.max_len = max_len
#     def __len__(self): return len(self.texts)
#     def __getitem__(self, idx):
#         enc = self.tok(self.texts[idx], truncation=True, max_length=self.max_len, padding=False)
#         enc["labels"] = self.labels[idx]
#         return {k: torch.tensor(v) for k, v in enc.items()}

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
# train_ds = TweetDataset(train_df, tokenizer, MAX_LEN)
# val_ds   = TweetDataset(val_df, tokenizer, MAX_LEN)
# test_ds  = TweetDataset(dftest_, tokenizer, MAX_LEN)

# # -------------------------
# # Model & Freeze/Unfreeze strategy
# # -------------------------
# def build_model(num_unfreeze_last_layers: int = 4, dropout: float = 0.1):
#     model = AutoModelForSequenceClassification.from_pretrained(
#         MODEL_NAME, num_labels=len(ORDER), id2label=ID2LABEL, label2id=LABEL2ID
#     )
#     # apply head-level dropout knobs
#     model.config.hidden_dropout_prob = float(dropout)
#     model.config.attention_probs_dropout_prob = float(dropout)

#     base = getattr(model, "roberta", None) or getattr(model, "bert", None) or getattr(model, "deberta", None)
#     if base is not None:
#         for p in base.parameters(): p.requires_grad = False
#         if hasattr(base, "encoder") and hasattr(base.encoder, "layer"):
#             k = int(num_unfreeze_last_layers)
#             if k > 0:
#                 for layer in base.encoder.layer[-k:]:
#                     for p in layer.parameters(): p.requires_grad = True
#     for p in model.classifier.parameters(): p.requires_grad = True
#     return model.to(DEVICE)

# # -------------------------
# # Train / Eval utilities
# # -------------------------
# def get_optimizer_scheduler(model, num_training_steps: int, lr: float, weight_decay: float, warmup_ratio: float):
#     no_decay = ["bias", "LayerNorm.weight"]
#     optimizer_grouped_parameters = [
#         {"params": [p for n, p in model.named_parameters() if p.requires_grad and not any(nd in n for nd in no_decay)], "weight_decay": weight_decay},
#         {"params": [p for n, p in model.named_parameters() if p.requires_grad and any(nd in n for nd in no_decay)],  "weight_decay": 0.0},
#     ]
#     # fused AdamW on CUDA if available
#     try:
#         optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=lr, weight_decay=weight_decay, fused=(DEVICE=="cuda"))
#     except TypeError:
#         optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=lr, weight_decay=weight_decay)
#     num_warmup = int(num_training_steps * warmup_ratio)
#     scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup, num_training_steps=num_training_steps)
#     return optimizer, scheduler

# # indices and mid set
# IDX_EXT_NEG = LABEL2ID["extremely negative"]
# IDX_NEG     = LABEL2ID["negative"]
# IDX_NEU     = LABEL2ID["neutral"]
# IDX_POS     = LABEL2ID["positive"]
# IDX_EXT_POS = LABEL2ID["extremely positive"]
# MIDS = [IDX_NEG, IDX_NEU, IDX_POS]

# def make_tier_weights(ratio_mid_ext: float) -> torch.Tensor:
#     """
#     Assigns the same weight to both extremes, and a higher weight to mid classes.
#     Renormalizes so the mean weight is ≈ 1.0 (keeps loss scale stable).
#     """
#     K = len(ORDER)
#     w = np.ones(K, dtype=np.float32)                        # extremes start at 1.0
#     w[[IDX_NEG, IDX_NEU, IDX_POS]] = float(ratio_mid_ext)   # mid classes boosted
#     w = w * (K / w.sum())                                   # mean ~ 1.0
#     return torch.tensor(w, dtype=torch.float, device=DEVICE)

# def evaluate(model, loader) -> Dict[str, float]:
#     model.eval()
#     preds, labels = [], []
#     with torch.no_grad():
#         for batch in loader:
#             batch = {k: v.to(DEVICE, non_blocking=True) for k, v in batch.items()}
#             with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),
#                           enabled=(DEVICE == "cuda" and USE_AMP)):
#                 logits = model(**batch).logits
#             preds.extend(torch.argmax(logits, dim=-1).detach().cpu().tolist())
#             labels.extend(batch["labels"].detach().cpu().tolist())
#     acc = accuracy_score(labels, preds)
#     p, r, f1, _ = precision_recall_fscore_support(labels, preds, average="macro", zero_division=0)
#     # mid-class F1
#     p_mid, r_mid, f1_mid, _ = precision_recall_fscore_support(labels, preds, labels=MIDS, average="macro", zero_division=0)
#     return {"acc": acc, "precision": p, "recall": r, "f1": f1, "f1_mid": f1_mid}

# def make_loaders(batch_size: int):
#     collate_fn = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=8)
#     train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  collate_fn=collate_fn, num_workers=0, pin_memory=True, persistent_workers=False)
#     val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, collate_fn=collate_fn, num_workers=0, pin_memory=True, persistent_workers=False)
#     test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, collate_fn=collate_fn, num_workers=0, pin_memory=True, persistent_workers=False)
#     return train_loader, val_loader, test_loader

# def train_one_run(hp: Dict) -> Tuple[str, Dict[str, float]]:
#     """
#     hp keys: run_name, num_unfreeze_last_layers, lr, weight_decay, epochs, patience, trial_number,
#              ratio_mid_ext, label_smoothing, warmup_ratio, grad_clip, dropout, batch_size
#     """
#     run_name        = hp["run_name"]
#     num_unfreeze    = int(hp["num_unfreeze_last_layers"])
#     lr              = float(hp["lr"])
#     wd              = float(hp["weight_decay"])
#     epochs          = int(hp.get("epochs",   FIXED_EPOCHS))
#     patience        = int(hp.get("patience", FIXED_PATIENCE))
#     ratio_mid_ext   = float(hp.get("ratio_mid_ext", 1.6))
#     label_smoothing = float(hp.get("label_smoothing", 0.05))
#     warmup_ratio    = float(hp.get("warmup_ratio", WARMUP_RATIO_DEFAULT))
#     grad_clip       = float(hp.get("grad_clip", GRAD_CLIP_DEFAULT))
#     dropout         = float(hp.get("dropout", 0.1))
#     bs              = int(hp.get("batch_size", BATCH_SIZE_DEFAULT))

#     # model + loaders
#     model = build_model(num_unfreeze, dropout=dropout)
#     train_loader, val_loader, _ = make_loaders(bs)

#     total_steps = int(math.ceil(len(train_loader) * epochs))
#     optimizer, scheduler = get_optimizer_scheduler(model, total_steps, lr, wd, warmup_ratio)

#     scaler = GradScaler(enabled=(DEVICE == "cuda" and USE_AMP))
#     best_metric = -1.0
#     no_improve = 0

#     # save to a different folder + name to avoid collisions
#     safe_run_name = run_name.replace("/", "__").replace("\\", "__")
#     ckpt_dir = "checkpoints_midf1"
#     os.makedirs(ckpt_dir, exist_ok=True)
#     best_path = os.path.join(ckpt_dir, f"best_midf1_{safe_run_name}.pt")

#     wandb_run = wandb.init(
#         project=PROJECT,
#         name=run_name,
#         config={
#             "model": MODEL_NAME,
#             "max_len": MAX_LEN,
#             "batch_size": bs,
#             "epochs": epochs,
#             "lr": lr,
#             "weight_decay": wd,
#             "warmup_ratio": warmup_ratio,
#             "grad_clip": grad_clip,
#             "dropout": dropout,
#             "ratio_mid_ext": ratio_mid_ext,
#             "label_smoothing": label_smoothing,
#             "num_unfreeze_last_layers": num_unfreeze,
#             "trial_number": hp.get("trial_number", -1),
#         },
#         reinit=True,
#     )

#     # nicer W&B charts
#     wandb.define_metric("epoch")
#     wandb.define_metric("step")
#     wandb.define_metric("train/*", step_metric="step")
#     wandb.define_metric("val/*",   step_metric="epoch")

#     # print + log trainable params
#     total_params     = sum(p.numel() for p in model.parameters())
#     trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
#     print(f"Trainable params: {trainable_params:,} / {total_params:,} "
#           f"({100.0*trainable_params/total_params:.2f}%) ; unfreeze_last_k={num_unfreeze}")
#     wandb.log({"params/total": total_params,
#                "params/trainable": trainable_params,
#                "params/ratio": trainable_params/max(1,total_params)}, step=0)

#     # class weights for this run
#     class_weights = make_tier_weights(ratio_mid_ext)

#     for epoch in range(epochs):
#         model.train()
#         t0 = time.time()
#         running_loss = 0.0

#         for step, batch in enumerate(train_loader):
#             batch = {k: v.to(DEVICE, non_blocking=True) for k, v in batch.items()}
#             labels = batch.pop("labels")  # we compute weighted loss ourselves

#             optimizer.zero_grad(set_to_none=True)
#             with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),
#                           enabled=(DEVICE == "cuda" and USE_AMP)):
#                 outputs = model(**batch)
#                 logits = outputs.logits
#                 try:
#                     loss = F.cross_entropy(logits, labels, weight=class_weights, label_smoothing=label_smoothing)
#                 except TypeError:
#                     loss = F.cross_entropy(logits, labels, weight=class_weights)

#             scaler.scale(loss).backward()
#             if grad_clip is not None:
#                 scaler.unscale_(optimizer)
#                 torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
#             scaler.step(optimizer); scaler.update(); scheduler.step()
#             running_loss += loss.item()

#             if step % 20 == 0:
#                 wandb.log({"train/loss": loss.item(), "step": step + 1, "epoch": epoch + 1})

#             # periodic console + throughput log (about 10x per epoch)
#             if step % max(1, len(train_loader)//10) == 0 or step == 1:
#                 avg_loss = running_loss / max(1, (step + 1))
#                 elapsed  = time.time() - t0
#                 items    = (step + 1) * bs
#                 itps     = items / max(elapsed, 1e-6)
#                 print(f"[e{epoch+1} b{step+1}/{len(train_loader)}] loss={loss.item():.4f} avg={avg_loss:.4f} it/s={itps:.1f}")
#                 wandb.log({"train/avg_loss_so_far": avg_loss,
#                            "train/items_per_sec": itps,
#                            "step": (epoch * len(train_loader)) + (step + 1),
#                            "epoch": epoch + 1})

#         # epoch-end validation
#         val_metrics = evaluate(model, val_loader)
#         elapsed = time.time() - t0

#         epoch_loss = running_loss / max(1, len(train_loader))
#         current_lr = scheduler.get_last_lr()[0]

#         wandb.log({
#             "train/epoch_loss": epoch_loss,
#             "val/acc": val_metrics["acc"],
#             "val/precision": val_metrics["precision"],
#             "val/recall": val_metrics["recall"],
#             "val/f1": val_metrics["f1"],
#             "val/mid_f1": val_metrics["f1_mid"],   # extra log
#             "lr": current_lr,
#             "time/epoch_sec": elapsed,
#             "epoch": epoch + 1,
#         })
#         # --- Early abort if epoch 1 macro-F1 is too low ---
#         if epoch == 0 and val_metrics["f1"] < EARLY_ABORT_F1_E1:
#             wandb.log({"early_stop/epoch1_low_f1": val_metrics["f1"], "epoch": epoch + 1})
#             print(f"[EARLY-EXIT] epoch=1 | val_f1={val_metrics['f1']:.4f} < {EARLY_ABORT_F1_E1:.2f} → stopping trial")
#             # ensure a checkpoint exists for downstream code
#             if not os.path.exists(best_path):
#                 torch.save(model.state_dict(), best_path)
#                 wandb_run.summary["best_checkpoint_path"] = best_path
#                 wandb_run.summary["best_val_f1"] = val_metrics["f1"]
#                 wandb_run.summary["best_val_mid_f1"] = val_metrics["f1_mid"]
#             break

#         # Early stopping on mid-class F1 (prints stay the same)
#         target_metric = val_metrics["f1_mid"]
#         if target_metric > best_metric:
#             best_metric = target_metric
#             torch.save(model.state_dict(), best_path)
#             no_improve = 0
#             wandb_run.summary["best_val_f1"] = best_metric   # kept same key for compatibility
#             wandb_run.summary["best_val_mid_f1"] = best_metric
#             wandb_run.summary["best_checkpoint_path"] = best_path
#             wandb.log({"val/best_f1_so_far": best_metric, "val/best_epoch": epoch + 1})
#         else:
#             no_improve += 1
#             if no_improve >= patience:
#                 print(f"Early stopping at epoch {epoch+1}")
#                 break

#         # console print line unchanged
#         print(f"Epoch {epoch+1}/{epochs} | "
#               f"loss={epoch_loss:.4f} | "
#               f"val_acc={val_metrics['acc']:.4f} | val_f1={val_metrics['f1']:.4f} | time={elapsed:.1f}s")

#     wandb.finish()

#     # Load best and return path + metrics on val for reference
#     model.load_state_dict(torch.load(best_path, map_location=DEVICE))
#     # rebuild val loader (for safety if using different bs later)
#     _, val_loader, _ = make_loaders(bs)
#     final_val = evaluate(model, val_loader)

#     # store final val in W&B summary for quick sorting
#     if wandb.run is not None:
#         wandb.run.summary["final_val_acc"] = final_val["acc"]
#         wandb.run.summary["final_val_precision"] = final_val["precision"]
#         wandb.run.summary["final_val_recall"] = final_val["recall"]
#         wandb.run.summary["final_val_f1"] = final_val["f1"]
#         wandb.run.summary["final_val_mid_f1"] = final_val["f1_mid"]

#     return best_path, final_val

# # -------------------------
# # Optuna hyperparameter tuning (ALWAYS ON)
# # -------------------------
# FIXED_EPOCHS = 8
# FIXED_PATIENCE = 3

# def objective(trial: optuna.trial.Trial):
#     params = {
#         "run_name": f"{BASE_RUN_NAME}_optuna_trial_{trial.number}",
#         "num_unfreeze_last_layers": trial.suggest_int("num_unfreeze_last_layers", 8, 12),
#         "lr": trial.suggest_float("lr", 1e-5, 1e-4, log=True),
#         "weight_decay": trial.suggest_float("weight_decay", 7e-8, 1e-5, log=True),
#         "batch_size": trial.suggest_categorical("batch_size", [4,8, 16, 32]),
#         "warmup_ratio": trial.suggest_float("warmup_ratio", 0.02, 0.12),
#         "grad_clip": trial.suggest_float("grad_clip", 0.5, 1.5),
#         "dropout": trial.suggest_float("dropout", 0.0, 0.2),
#         "ratio_mid_ext": trial.suggest_float("ratio_mid_ext", 1.2, 2.5),
#         "label_smoothing": trial.suggest_float("label_smoothing", 0.00, 0.12),
#         "epochs": FIXED_EPOCHS,
#         "patience": FIXED_PATIENCE,
#         "trial_number": trial.number,
#     }
#     path, val_metrics = train_one_run(params)
#     print(f"[Trial {trial.number}] f1={val_metrics['f1']:.4f} | "
#           f"unfreeze_k={params['num_unfreeze_last_layers']} lr={params['lr']:.2e} "
#           f"wd={params['weight_decay']:.1e} suggested_bs={params['batch_size']}")
#     trial.report(val_metrics["f1"], step=1)  # study objective stays macro F1
#     return val_metrics["f1"]

# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=TRIALS, show_progress_bar=True)
# print("Best trial:", study.best_trial.number, "F1:", study.best_value)
# best_params = {"run_name": f"{BASE_RUN_NAME}_best_optuna", **study.best_trial.params}

# # persist best params to a separate folder/name
# os.makedirs("checkpoints_midf1", exist_ok=True)
# with open(os.path.join("checkpoints_midf1", "best_hparams_optuna_midf1_new_code.json"), "w") as f:
#     json.dump({**best_params, "epochs": FIXED_EPOCHS, "patience": FIXED_PATIENCE}, f, indent=2)

# # (Optional) retrain once on the best params to get a clean checkpoint:
# # best_ckpt, _ = train_one_run(best_params)
# # print("Best checkpoint saved to:", best_ckpt)


Train/Val/Test sizes: 37039/4116/3798


C:\Users\adishalit1\AppData\Local\anaconda3\envs\dl4090\Lib\site-packages\transformers\convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
[I 2025-08-18 09:53:20,497] A new study created in memory with name: no-name-c77d1994-8070-4841-86c9-c2ee6d7644f4
  0%|          | 0/30 [00:00<?, ?it/s]Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/mdeberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use

Trainable params: 78,561,029 / 278,813,189 (28.18%) ; unfreeze_last_k=11


C:\Users\adishalit1\AppData\Local\Temp\ipykernel_35316\3429734493.py:303: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=(torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16),


[e1 b1/1158] loss=1.6729 avg=1.6729 it/s=69.2
[e1 b2/1158] loss=1.6312 avg=1.6521 it/s=108.5
[e1 b116/1158] loss=1.5415 avg=1.6206 it/s=530.5
[e1 b231/1158] loss=1.3327 avg=1.5464 it/s=599.9
[e1 b346/1158] loss=1.3846 avg=1.5029 it/s=627.3
[e1 b461/1158] loss=1.1854 avg=1.4677 it/s=639.8
[e1 b576/1158] loss=1.3628 avg=1.4338 it/s=650.0
[e1 b691/1158] loss=1.1391 avg=1.3957 it/s=654.0
